# Analisis Base

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import friedmanchisquare
from itertools import combinations
import warnings
warnings.filterwarnings('ignore')

class DieboldMarianoTest:
    """
    Implementación del test de Diebold-Mariano para comparar pronósticos
    """
    @staticmethod
    def dm_test(errors1, errors2, h=1, crit="MSE", power=2):
        """
        Realiza el test de Diebold-Mariano
        
        Parameters:
        -----------
        errors1 : array-like
            Errores del primer modelo
        errors2 : array-like
            Errores del segundo modelo
        h : int
            Horizonte de predicción (para ajustar autocorrelación)
        crit : str
            Criterio de pérdida: "MSE", "MAE", "MAPE"
        power : int
            Potencia para la función de pérdida
            
        Returns:
        --------
        dm_stat : float
            Estadístico DM
        p_value : float
            P-valor (two-tailed)
        """
        errors1 = np.array(errors1)
        errors2 = np.array(errors2)
        
        # Calcular diferencias de pérdida
        if crit == "MSE":
            loss_diff = errors1**2 - errors2**2
        elif crit == "MAE":
            loss_diff = np.abs(errors1) - np.abs(errors2)
        elif crit == "MAPE":
            loss_diff = np.abs(errors1) - np.abs(errors2)
        else:
            loss_diff = errors1**power - errors2**power
        
        # Media de las diferencias
        mean_diff = np.mean(loss_diff)
        
        # Varianza de las diferencias (ajustada por autocorrelación)
        n = len(loss_diff)
        
        # Calcular varianza con corrección de Newey-West
        gamma0 = np.var(loss_diff, ddof=1)
        
        if h > 1:
            gamma_sum = 0
            for k in range(1, h):
                gamma_k = np.cov(loss_diff[:-k], loss_diff[k:])[0, 1]
                gamma_sum += (1 - k/h) * gamma_k
            variance = (gamma0 + 2 * gamma_sum) / n
        else:
            variance = gamma0 / n
        
        # Estadístico DM
        dm_stat = mean_diff / np.sqrt(variance) if variance > 0 else 0
        
        # P-valor (two-tailed)
        p_value = 2 * (1 - stats.norm.cdf(np.abs(dm_stat)))
        
        return dm_stat, p_value


class ModelPerformanceAnalyzer:
    """
    Clase para análisis exhaustivo de rendimiento de modelos de predicción
    en diferentes escenarios de simulación.
    """
    
    def __init__(self):
        """
        Inicializa el analizador cargando los datos de los tres escenarios.
        """
        self.models = ['AREPD', 'AV-MCPS', 'Block Bootstrapping', 'DeepAR', 
                      'EnCQR-LSTM', 'LSPM', 'LSPMW', 'MCPS', 'Sieve Bootstrap']
        
        # Cargar datos con las rutas especificadas
        print("Cargando datos...")
        
        try:
            self.df_estacionario = pd.read_excel("./Datos/estacionario.xlsx")
            self.df_estacionario['Escenario'] = 'Estacionario_Lineal'
            print(f"✓ Estacionario: {len(self.df_estacionario)} filas")
            print(f"  Columnas: {self.df_estacionario.columns.tolist()}")
            
            self.df_no_estacionario = pd.read_excel("./Datos/no_estacionario.xlsx")
            self.df_no_estacionario['Escenario'] = 'No_Estacionario_Lineal'
            print(f"✓ No Estacionario: {len(self.df_no_estacionario)} filas")
            print(f"  Columnas: {self.df_no_estacionario.columns.tolist()}")
            
            self.df_no_lineal = pd.read_excel("./Datos/no_lineal.xlsx")
            self.df_no_lineal['Escenario'] = 'No_Lineal'
            print(f"✓ No Lineal: {len(self.df_no_lineal)} filas")
            print(f"  Columnas: {self.df_no_lineal.columns.tolist()}")
            
        except FileNotFoundError as e:
            print(f"ERROR: No se encontró el archivo - {e}")
            print("Verifica que los archivos estén en la carpeta './Datos/'")
            raise
        
        # Estandarizar nombres de columnas
        self._standardize_columns()
        
        # Combinar todos los datos
        self.df_all = pd.concat([self.df_estacionario, self.df_no_estacionario, 
                                 self.df_no_lineal], ignore_index=True)
        
        # Convertir tipos de datos críticos
        self._convert_data_types()
        
        print(f"\n✓ Datos combinados: {len(self.df_all)} observaciones totales")
        print(f"✓ Columnas finales: {self.df_all.columns.tolist()}")
        
    def _standardize_columns(self):
        """Estandariza nombres de columnas entre datasets"""
        # Para estacionario
        if 'Varianza error' in self.df_estacionario.columns:
            self.df_estacionario.rename(columns={'Varianza error': 'Varianza'}, inplace=True)
        
        # Agregar columna 'Tipo de Modelo' si no existe en estacionario
        if 'Tipo de Modelo' not in self.df_estacionario.columns:
            # Crear tipo de modelo basado en valores AR y MA
            def create_model_type(row):
                ar_vals = row.get('Valores de AR', '')
                ma_vals = row.get('Valores MA', '')
                
                ar_str = str(ar_vals) if pd.notna(ar_vals) else ''
                ma_str = str(ma_vals) if pd.notna(ma_vals) else ''
                
                # Contar órdenes
                ar_order = len([x for x in ar_str.split(',') if x.strip() and x.strip() != '[]']) if ar_str else 0
                ma_order = len([x for x in ma_str.split(',') if x.strip() and x.strip() != '[]']) if ma_str else 0
                
                if ar_order > 0 and ma_order > 0:
                    return f'ARMA({ar_order},{ma_order})'
                elif ar_order > 0:
                    return f'AR({ar_order})'
                elif ma_order > 0:
                    return f'MA({ma_order})'
                else:
                    return 'Unknown'
            
            self.df_estacionario['Tipo de Modelo'] = self.df_estacionario.apply(create_model_type, axis=1)
        
        # Para no estacionario
        if 'Varianza error' in self.df_no_estacionario.columns:
            self.df_no_estacionario.rename(columns={'Varianza error': 'Varianza'}, inplace=True)
        
        # Para no lineal
        if 'Varianza error' in self.df_no_lineal.columns:
            self.df_no_lineal.rename(columns={'Varianza error': 'Varianza'}, inplace=True)
    
    def _convert_data_types(self):
        """Convierte tipos de datos para evitar errores de comparación"""
        # Convertir 'Paso' a numérico
        self.df_all['Paso'] = pd.to_numeric(self.df_all['Paso'], errors='coerce')
        
        # Convertir 'Varianza' a numérico
        self.df_all['Varianza'] = pd.to_numeric(self.df_all['Varianza'], errors='coerce')
        
        # Convertir columnas de modelos a numérico
        for model in self.models:
            self.df_all[model] = pd.to_numeric(self.df_all[model], errors='coerce')
        
        # Eliminar filas con valores NaN críticos
        critical_cols = ['Paso', 'Varianza'] + self.models
        self.df_all.dropna(subset=critical_cols, inplace=True)
        
        print(f"✓ Tipos de datos convertidos")
        print(f"✓ Filas después de limpieza: {len(self.df_all)}")
        
    def generate_full_report(self, output_dir='resultados_analisis'):
        """
        Genera reporte completo respondiendo a todas las preguntas clave.
        """
        import os
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        
        print("\n" + "="*80)
        print("INICIANDO ANÁLISIS COMPREHENSIVO DE MODELOS")
        print("="*80)
        
        # Crear archivo de reporte
        report_file = f"{output_dir}/reporte_completo.txt"
        with open(report_file, 'w', encoding='utf-8') as f:
            f.write("REPORTE COMPLETO DE ANÁLISIS DE MODELOS DE PREDICCIÓN\n")
            f.write("="*80 + "\n\n")
        
        # 1. ANÁLISIS POR CARACTERÍSTICAS DEL DGP
        print("\n1. Analizando características del proceso generador...")
        self.analyze_dgp_characteristics(output_dir)
        
        # 2. ANÁLISIS POR DISTRIBUCIÓN DE ERRORES
        print("\n2. Analizando efecto de distribuciones...")
        self.analyze_distribution_effects(output_dir)
        
        # 3. ANÁLISIS POR HORIZONTE DE PREDICCIÓN
        print("\n3. Analizando horizonte de predicción...")
        self.analyze_horizon_effects(output_dir)
        
        # 4. ANÁLISIS DE INTERACCIONES COMPLEJAS
        print("\n4. Analizando interacciones complejas...")
        self.analyze_interactions(output_dir)
        
        # 5. ANÁLISIS DE ROBUSTEZ Y ESTABILIDAD
        print("\n5. Analizando robustez y estabilidad...")
        self.analyze_robustness(output_dir)
        
        # 6. ANÁLISIS DE SIGNIFICANCIA ESTADÍSTICA (DIEBOLD-MARIANO)
        print("\n6. Realizando tests de Diebold-Mariano...")
        self.analyze_statistical_significance_dm(output_dir)
        
        # 7. ANÁLISIS POR MODELO INDIVIDUAL
        print("\n7. Generando perfiles por modelo...")
        self.analyze_individual_models(output_dir)
        
        # 8. RECOMENDACIONES Y CONCLUSIONES
        print("\n8. Generando recomendaciones...")
        self.generate_recommendations(output_dir)
        
        print(f"\n{'='*80}")
        print(f"ANÁLISIS COMPLETO. Resultados guardados en: {output_dir}/")
        print(f"{'='*80}")
        
    def analyze_dgp_characteristics(self, output_dir):
        """
        1. ANÁLISIS DE CARACTERÍSTICAS DEL PROCESO GENERADOR
        """
        results = []
        
        # 1.1 Efecto de estacionaridad
        print("  - Analizando efecto de estacionaridad...")
        for model in self.models:
            est_mean = self.df_estacionario[model].mean()
            no_est_mean = self.df_no_estacionario[model].mean()
            diff = no_est_mean - est_mean
            pct_change = (diff / est_mean) * 100 if est_mean != 0 else 0
            
            results.append({
                'Modelo': model,
                'ECRPS_Estacionario': est_mean,
                'ECRPS_No_Estacionario': no_est_mean,
                'Diferencia': diff,
                'Cambio_%': pct_change
            })
        
        df_estacionaridad = pd.DataFrame(results)
        df_estacionaridad = df_estacionaridad.sort_values('Cambio_%')
        df_estacionaridad.to_csv(f'{output_dir}/1_efecto_estacionaridad.csv', index=False)
        
        # Visualización
        fig, axes = plt.subplots(1, 2, figsize=(15, 6))
        
        # Gráfico de barras comparativas
        x = np.arange(len(self.models))
        width = 0.35
        axes[0].bar(x - width/2, df_estacionaridad['ECRPS_Estacionario'], 
                   width, label='Estacionario', alpha=0.8)
        axes[0].bar(x + width/2, df_estacionaridad['ECRPS_No_Estacionario'], 
                   width, label='No Estacionario', alpha=0.8)
        axes[0].set_xlabel('Modelo')
        axes[0].set_ylabel('ECRPS Promedio')
        axes[0].set_title('Rendimiento: Estacionario vs No Estacionario')
        axes[0].set_xticks(x)
        axes[0].set_xticklabels(df_estacionaridad['Modelo'], rotation=45, ha='right')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)
        
        # Gráfico de cambio porcentual
        colors = ['green' if x < 0 else 'red' for x in df_estacionaridad['Cambio_%']]
        axes[1].barh(df_estacionaridad['Modelo'], df_estacionaridad['Cambio_%'], color=colors, alpha=0.7)
        axes[1].set_xlabel('Cambio Porcentual (%)')
        axes[1].set_title('Impacto de No Estacionaridad\n(Negativo = Mejor en No Estacionario)')
        axes[1].axvline(x=0, color='black', linestyle='--', linewidth=0.8)
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(f'{output_dir}/1_estacionaridad.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        # 1.2 Efecto de no linealidad
        print("  - Analizando efecto de no linealidad...")
        results_nl = []
        for model in self.models:
            lin_mean = self.df_estacionario[model].mean()
            nl_mean = self.df_no_lineal[model].mean()
            diff = nl_mean - lin_mean
            pct_change = (diff / lin_mean) * 100 if lin_mean != 0 else 0
            
            results_nl.append({
                'Modelo': model,
                'ECRPS_Lineal': lin_mean,
                'ECRPS_No_Lineal': nl_mean,
                'Diferencia': diff,
                'Cambio_%': pct_change
            })
        
        df_linealidad = pd.DataFrame(results_nl)
        df_linealidad = df_linealidad.sort_values('Cambio_%')
        df_linealidad.to_csv(f'{output_dir}/1_efecto_no_linealidad.csv', index=False)
        
        # Visualización no linealidad
        fig, axes = plt.subplots(1, 2, figsize=(15, 6))
        
        x = np.arange(len(self.models))
        axes[0].bar(x - width/2, df_linealidad['ECRPS_Lineal'], 
                   width, label='Lineal', alpha=0.8)
        axes[0].bar(x + width/2, df_linealidad['ECRPS_No_Lineal'], 
                   width, label='No Lineal', alpha=0.8)
        axes[0].set_xlabel('Modelo')
        axes[0].set_ylabel('ECRPS Promedio')
        axes[0].set_title('Rendimiento: Lineal vs No Lineal')
        axes[0].set_xticks(x)
        axes[0].set_xticklabels(df_linealidad['Modelo'], rotation=45, ha='right')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)
        
        colors = ['green' if x < 0 else 'red' for x in df_linealidad['Cambio_%']]
        axes[1].barh(df_linealidad['Modelo'], df_linealidad['Cambio_%'], color=colors, alpha=0.7)
        axes[1].set_xlabel('Cambio Porcentual (%)')
        axes[1].set_title('Impacto de No Linealidad\n(Negativo = Mejor en No Lineal)')
        axes[1].axvline(x=0, color='black', linestyle='--', linewidth=0.8)
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(f'{output_dir}/1_no_linealidad.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        # 1.3 Análisis por tipo de modelo
        print("  - Analizando efecto del tipo de modelo...")
        self.analyze_model_type_effect(output_dir)
        
    def analyze_model_type_effect(self, output_dir):
        """Analiza el efecto del tipo de modelo en el rendimiento"""
        
        # Análisis para datos estacionarios
        if 'Tipo de Modelo' in self.df_estacionario.columns:
            results_type = []
            for model in self.models:
                for model_type in self.df_estacionario['Tipo de Modelo'].unique():
                    subset = self.df_estacionario[self.df_estacionario['Tipo de Modelo'] == model_type]
                    if len(subset) > 0:
                        results_type.append({
                            'Modelo_Predictor': model,
                            'Tipo_Proceso': model_type,
                            'ECRPS_Mean': subset[model].mean(),
                            'ECRPS_Std': subset[model].std(),
                            'N_Obs': len(subset)
                        })
            
            df_type = pd.DataFrame(results_type)
            df_type.to_csv(f'{output_dir}/1_efecto_tipo_modelo.csv', index=False)
            
            # Crear heatmap para tipos más comunes
            common_types = df_type['Tipo_Proceso'].value_counts().head(10).index
            df_type_filtered = df_type[df_type['Tipo_Proceso'].isin(common_types)]
            
            if len(df_type_filtered) > 0:
                pivot = df_type_filtered.pivot_table(
                    index='Modelo_Predictor', 
                    columns='Tipo_Proceso', 
                    values='ECRPS_Mean'
                )
                
                fig, ax = plt.subplots(figsize=(14, 8))
                sns.heatmap(pivot, annot=True, fmt='.4f', cmap='RdYlGn_r', ax=ax, 
                           cbar_kws={'label': 'ECRPS'})
                ax.set_title('Rendimiento por Modelo Predictor y Tipo de Proceso', fontsize=14)
                ax.set_xlabel('Tipo de Proceso')
                ax.set_ylabel('Modelo Predictor')
                plt.tight_layout()
                plt.savefig(f'{output_dir}/1_heatmap_tipo_modelo.png', dpi=300, bbox_inches='tight')
                plt.close()
        
    def analyze_distribution_effects(self, output_dir):
        """
        2. ANÁLISIS DE EFECTOS DE DISTRIBUCIÓN
        """
        print("  - Analizando efectos de distribuciones...")
        
        results_dist = []
        for model in self.models:
            for dist in self.df_all['Distribución'].unique():
                if pd.notna(dist):
                    subset = self.df_all[self.df_all['Distribución'] == dist]
                    if len(subset) > 0:
                        results_dist.append({
                            'Modelo': model,
                            'Distribución': dist,
                            'ECRPS_Mean': subset[model].mean(),
                            'ECRPS_Std': subset[model].std(),
                            'ECRPS_Min': subset[model].min(),
                            'ECRPS_Max': subset[model].max()
                        })
        
        df_dist = pd.DataFrame(results_dist)
        df_dist.to_csv(f'{output_dir}/2_efecto_distribucion.csv', index=False)
        
        # Heatmap
        if len(df_dist) > 0:
            pivot = df_dist.pivot(index='Modelo', columns='Distribución', values='ECRPS_Mean')
            
            fig, ax = plt.subplots(figsize=(10, 8))
            sns.heatmap(pivot, annot=True, fmt='.4f', cmap='RdYlGn_r', ax=ax, cbar_kws={'label': 'ECRPS'})
            ax.set_title('Rendimiento por Modelo y Distribución', fontsize=14)
            plt.tight_layout()
            plt.savefig(f'{output_dir}/2_heatmap_distribucion.png', dpi=300, bbox_inches='tight')
            plt.close()
        
        # Análisis por varianza
        print("  - Analizando efectos de varianza...")
        results_var = []
        varianzas_unicas = sorted([v for v in self.df_all['Varianza'].unique() if pd.notna(v)])
        
        for model in self.models:
            for var in varianzas_unicas:
                subset = self.df_all[self.df_all['Varianza'] == var]
                if len(subset) > 0:
                    results_var.append({
                        'Modelo': model,
                        'Varianza': var,
                        'ECRPS_Mean': subset[model].mean(),
                        'ECRPS_Std': subset[model].std()
                    })
        
        df_var = pd.DataFrame(results_var)
        df_var.to_csv(f'{output_dir}/2_efecto_varianza.csv', index=False)
        
        # Gráfico de líneas por varianza
        if len(df_var) > 0:
            fig, ax = plt.subplots(figsize=(12, 8))
            for model in self.models:
                data = df_var[df_var['Modelo'] == model].sort_values('Varianza')
                if len(data) > 0:
                    ax.plot(data['Varianza'], data['ECRPS_Mean'], marker='o', label=model, linewidth=2)
            
            ax.set_xlabel('Varianza', fontsize=12)
            ax.set_ylabel('ECRPS Promedio', fontsize=12)
            ax.set_title('Rendimiento según Nivel de Varianza', fontsize=14)
            ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
            ax.grid(True, alpha=0.3)
            plt.tight_layout()
            plt.savefig(f'{output_dir}/2_efecto_varianza.png', dpi=300, bbox_inches='tight')
            plt.close()
        
    def analyze_horizon_effects(self, output_dir):
        """
        3. ANÁLISIS DE HORIZONTE DE PREDICCIÓN
        """
        print("  - Analizando deterioro por horizonte...")
        
        results_horizon = []
        pasos_unicos = sorted([p for p in self.df_all['Paso'].unique() if pd.notna(p)])
        
        for model in self.models:
            for paso in pasos_unicos:
                subset = self.df_all[self.df_all['Paso'] == paso]
                if len(subset) > 0:
                    mean_val = subset[model].mean()
                    std_val = subset[model].std()
                    cv_val = std_val / mean_val if mean_val != 0 and pd.notna(mean_val) else 0
                    
                    results_horizon.append({
                        'Modelo': model,
                        'Paso': int(paso),
                        'ECRPS_Mean': mean_val,
                        'ECRPS_Std': std_val,
                        'ECRPS_CV': cv_val
                    })
        
        df_horizon = pd.DataFrame(results_horizon)
        df_horizon.to_csv(f'{output_dir}/3_efecto_horizonte.csv', index=False)
        
        # Gráfico de deterioro
        fig, axes = plt.subplots(1, 2, figsize=(16, 6))
        
        # ECRPS promedio por paso
        for model in self.models:
            data = df_horizon[df_horizon['Modelo'] == model].sort_values('Paso')
            if len(data) > 0:
                axes[0].plot(data['Paso'], data['ECRPS_Mean'], marker='o', label=model, linewidth=2)
        
        axes[0].set_xlabel('Paso de Predicción', fontsize=12)
        axes[0].set_ylabel('ECRPS Promedio', fontsize=12)
        axes[0].set_title('Deterioro del Rendimiento por Horizonte', fontsize=14)
        axes[0].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        axes[0].grid(True, alpha=0.3)
        
        # Tasa de deterioro
        deterioro = []
        for model in self.models:
            data = df_horizon[df_horizon['Modelo'] == model].sort_values('Paso')
            if len(data) >= 2:
                paso_values = data['Paso'].tolist()
                ecrps_paso1 = data.iloc[0]['ECRPS_Mean']
                ecrps_paso_final = data.iloc[-1]['ECRPS_Mean']
                
                if pd.notna(ecrps_paso1) and pd.notna(ecrps_paso_final) and ecrps_paso1 != 0:
                    tasa = ((ecrps_paso_final - ecrps_paso1) / ecrps_paso1) * 100
                    deterioro.append({'Modelo': model, 'Deterioro_%': tasa})
        
        if deterioro:
            df_deterioro = pd.DataFrame(deterioro).sort_values('Deterioro_%')
            colors = ['green' if x < df_deterioro['Deterioro_%'].median() else 'red' 
                     for x in df_deterioro['Deterioro_%']]
            axes[1].barh(df_deterioro['Modelo'], df_deterioro['Deterioro_%'], color=colors, alpha=0.7)
            axes[1].set_xlabel(f'Deterioro Paso {pasos_unicos[0]}→{pasos_unicos[-1]} (%)', fontsize=12)
            axes[1].set_title('Tasa de Deterioro por Modelo', fontsize=14)
            axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(f'{output_dir}/3_horizonte_prediccion.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        # Análisis de consistencia de ranking
        print("  - Analizando consistencia de ranking...")
        ranking_consistency = []
        for paso in pasos_unicos:
            subset = self.df_all[self.df_all['Paso'] == paso]
            if len(subset) > 0:
                ranks = subset[self.models].mean().rank()
                rank_dict = ranks.to_dict()
                rank_dict['Paso'] = int(paso)
                ranking_consistency.append(rank_dict)
        
        df_ranks = pd.DataFrame(ranking_consistency)
        df_ranks.to_csv(f'{output_dir}/3_ranking_por_paso.csv', index=False)
        
    def analyze_interactions(self, output_dir):
        """
        4. ANÁLISIS DE INTERACCIONES COMPLEJAS
        """
        print("  - Analizando interacciones Escenario × Distribución...")
        
        results_int = []
        for model in self.models:
            for escenario in self.df_all['Escenario'].unique():
                for dist in self.df_all['Distribución'].unique():
                    subset = self.df_all[(self.df_all['Escenario'] == escenario) & 
                                        (self.df_all['Distribución'] == dist)]
                    if len(subset) > 0:
                        results_int.append({
                            'Modelo': model,
                            'Escenario': escenario,
                            'Distribución': dist,
                            'ECRPS_Mean': subset[model].mean()
                        })
        
        df_int = pd.DataFrame(results_int)
        df_int.to_csv(f'{output_dir}/4_interacciones.csv', index=False)
        
        # Heatmap de interacciones para cada modelo
        for model in self.models[:3]:  # Solo primeros 3 por espacio
            model_data = df_int[df_int['Modelo'] == model]
            if len(model_data) > 0:
                pivot = model_data.pivot(
                    index='Escenario', columns='Distribución', values='ECRPS_Mean')
                
                fig, ax = plt.subplots(figsize=(10, 6))
                sns.heatmap(pivot, annot=True, fmt='.4f', cmap='RdYlGn_r', ax=ax)
                ax.set_title(f'Interacción Escenario × Distribución: {model}', fontsize=12)
                plt.tight_layout()
                plt.savefig(f'{output_dir}/4_interaccion_{model.replace(" ", "_")}.png', 
                           dpi=300, bbox_inches='tight')
                plt.close()
        
        # Interacción triple: Escenario × Varianza × Paso
        print("  - Analizando interacción triple...")
        results_triple = []
        
        varianzas_unicas = sorted([v for v in self.df_all['Varianza'].unique() if pd.notna(v)])
        pasos_unicos = sorted([p for p in self.df_all['Paso'].unique() if pd.notna(p)])
        
        for model in self.models:
            for escenario in self.df_all['Escenario'].unique():
                for var in varianzas_unicas:
                    for paso in pasos_unicos:
                        subset = self.df_all[
                            (self.df_all['Escenario'] == escenario) & 
                            (self.df_all['Varianza'] == var) &
                            (self.df_all['Paso'] == paso)
                        ]
                        if len(subset) > 0:
                            results_triple.append({
                                'Modelo': model,
                                'Escenario': escenario,
                                'Varianza': var,
                                'Paso': int(paso),
                                'ECRPS_Mean': subset[model].mean()
                            })
        
        df_triple = pd.DataFrame(results_triple)
        df_triple.to_csv(f'{output_dir}/4_interaccion_triple.csv', index=False)
        
    def analyze_robustness(self, output_dir):
        """
        5. ANÁLISIS DE ROBUSTEZ Y ESTABILIDAD
        """
        print("  - Calculando métricas de robustez...")
        
        results_robust = []
        for model in self.models:
            ecrps_values = self.df_all[model]
            
            results_robust.append({
                'Modelo': model,
                'ECRPS_Mean': ecrps_values.mean(),
                'ECRPS_Std': ecrps_values.std(),
                'ECRPS_CV': ecrps_values.std() / ecrps_values.mean() if ecrps_values.mean() != 0 else 0,
                'ECRPS_Min': ecrps_values.min(),
                'ECRPS_Q25': ecrps_values.quantile(0.25),
                'ECRPS_Median': ecrps_values.median(),
                'ECRPS_Q75': ecrps_values.quantile(0.75),
                'ECRPS_Max': ecrps_values.max(),
                'ECRPS_IQR': ecrps_values.quantile(0.75) - ecrps_values.quantile(0.25)
            })
        
        df_robust = pd.DataFrame(results_robust)
        df_robust = df_robust.sort_values('ECRPS_CV')
        df_robust.to_csv(f'{output_dir}/5_robustez.csv', index=False)
        
        # Gráfico de robustez
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        
        # Coeficiente de variación
        axes[0, 0].barh(df_robust['Modelo'], df_robust['ECRPS_CV'], alpha=0.7)
        axes[0, 0].set_xlabel('Coeficiente de Variación')
        axes[0, 0].set_title('Estabilidad (Menor CV = Más Estable)')
        axes[0, 0].grid(True, alpha=0.3)
        
        # Rango intercuartílico
        axes[0, 1].barh(df_robust['Modelo'], df_robust['ECRPS_IQR'], alpha=0.7, color='coral')
        axes[0, 1].set_xlabel('Rango Intercuartílico')
        axes[0, 1].set_title('Variabilidad (Menor IQR = Más Consistente)')
        axes[0, 1].grid(True, alpha=0.3)
        
        # Boxplot comparativo
        data_box = [self.df_all[model] for model in self.models]
        bp = axes[1, 0].boxplot(data_box, labels=self.models, patch_artist=True)
        for patch in bp['boxes']:
            patch.set_facecolor('lightblue')
        axes[1, 0].set_ylabel('ECRPS')
        axes[1, 0].set_title('Distribución de ECRPS por Modelo')
        axes[1, 0].tick_params(axis='x', rotation=45)
        axes[1, 0].grid(True, alpha=0.3)
        
        # Scatter: Media vs Variabilidad
        axes[1, 1].scatter(df_robust['ECRPS_Mean'], df_robust['ECRPS_Std'], 
                          s=100, alpha=0.6, c=range(len(df_robust)), cmap='viridis')
        for idx, row in df_robust.iterrows():
            axes[1, 1].annotate(row['Modelo'], 
                               (row['ECRPS_Mean'], row['ECRPS_Std']),
                               fontsize=8, alpha=0.7)
        axes[1, 1].set_xlabel('ECRPS Promedio')
        axes[1, 1].set_ylabel('Desviación Estándar')
        axes[1, 1].set_title('Trade-off Rendimiento vs Estabilidad')
        axes[1, 1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.savefig(f'{output_dir}/5_robustez.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        # Análisis de peores casos
        print("  - Identificando peores casos...")
        worst_cases = []
        for model in self.models:
            df_temp = self.df_all.copy()
            df_temp['ECRPS'] = df_temp[model]
            worst = df_temp.nlargest(10, 'ECRPS')[
                ['Escenario', 'Tipo de Modelo', 'Distribución', 'Varianza', 'Paso', 'ECRPS']
            ]
            worst['Modelo_Predictor'] = model
            worst_cases.append(worst)
        
        df_worst = pd.concat(worst_cases, ignore_index=True)
        df_worst.to_csv(f'{output_dir}/5_peores_casos.csv', index=False)
        
    def analyze_statistical_significance_dm(self, output_dir):
        """
        6. ANÁLISIS DE SIGNIFICANCIA ESTADÍSTICA CON DIEBOLD-MARIANO
        """
        print("  - Realizando tests de Diebold-Mariano...")
        
        # Test de Friedman por escenario (para comparación general)
        results_friedman = []
        for escenario in self.df_all['Escenario'].unique():
            subset = self.df_all[self.df_all['Escenario'] == escenario]
            data_matrix = subset[self.models].values
            
            try:
                statistic, p_value = friedmanchisquare(*[data_matrix[:, i] for i in range(len(self.models))])
                
                results_friedman.append({
                    'Escenario': escenario,
                    'Friedman_Statistic': statistic,
                    'P_Value': p_value,
                    'Significativo': 'Sí' if p_value < 0.05 else 'No'
                })
            except Exception as e:
                print(f"    Advertencia: Error en test de Friedman para {escenario}: {e}")
        
        if results_friedman:
            df_friedman = pd.DataFrame(results_friedman)
            df_friedman.to_csv(f'{output_dir}/6_test_friedman.csv', index=False)
        
        # Tests de Diebold-Mariano pareados
        print("  - Realizando tests pareados de Diebold-Mariano...")
        pairs = list(combinations(self.models, 2))
        dm_results = []
        
        for model1, model2 in pairs:
            # Calcular errores (usamos ECRPS directamente como métrica de pérdida)
            errors1 = self.df_all[model1].values
            errors2 = self.df_all[model2].values
            
            # Test de Diebold-Mariano
            dm_stat, p_value = DieboldMarianoTest.dm_test(errors1, errors2, h=1, crit="MSE")
            
            mean_diff = self.df_all[model1].mean() - self.df_all[model2].mean()
            
            # Determinar ganador
            if p_value < 0.05:
                if mean_diff < 0:
                    ganador = model1
                else:
                    ganador = model2
            else:
                ganador = 'Empate'
            
            dm_results.append({
                'Modelo_1': model1,
                'Modelo_2': model2,
                'Diferencia_Media': mean_diff,
                'DM_Statistic': dm_stat,
                'P_Value': p_value,
                'Significativo_0.05': 'Sí' if p_value < 0.05 else 'No',
                'Significativo_0.01': 'Sí' if p_value < 0.01 else 'No',
                'Ganador': ganador
            })
        
        df_dm = pd.DataFrame(dm_results)
        df_dm = df_dm.sort_values('P_Value')
        df_dm.to_csv(f'{output_dir}/6_tests_diebold_mariano.csv', index=False)
        
        # Matriz de p-valores (Diebold-Mariano)
        print("  - Creando matriz de p-valores...")
        p_matrix = np.ones((len(self.models), len(self.models)))
        for i, model1 in enumerate(self.models):
            for j, model2 in enumerate(self.models):
                if i != j:
                    errors1 = self.df_all[model1].values
                    errors2 = self.df_all[model2].values
                    _, p_val = DieboldMarianoTest.dm_test(errors1, errors2, h=1, crit="MSE")
                    p_matrix[i, j] = p_val
        
        fig, ax = plt.subplots(figsize=(12, 10))
        sns.heatmap(p_matrix, annot=True, fmt='.3f', cmap='RdYlGn', 
                   xticklabels=self.models, yticklabels=self.models, 
                   ax=ax, vmin=0, vmax=0.1, cbar_kws={'label': 'P-valor'})
        ax.set_title('Matriz de P-valores (Test de Diebold-Mariano)\nVerde = Diferencia Significativa', 
                    fontsize=14)
        plt.tight_layout()
        plt.savefig(f'{output_dir}/6_matriz_pvalores_dm.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        # Dominancia estadística con Diebold-Mariano
        print("  - Analizando dominancia estadística...")
        dominance = []
        for model in self.models:
            wins = 0
            losses = 0
            ties = 0
            for other_model in self.models:
                if model != other_model:
                    errors1 = self.df_all[model].values
                    errors2 = self.df_all[other_model].values
                    _, p_val = DieboldMarianoTest.dm_test(errors1, errors2, h=1, crit="MSE")
                    mean_diff = self.df_all[model].mean() - self.df_all[other_model].mean()
                    
                    if p_val < 0.05:
                        if mean_diff < 0:  # modelo es mejor (menor ECRPS)
                            wins += 1
                        else:
                            losses += 1
                    else:
                        ties += 1
            
            dominance.append({
                'Modelo': model,
                'Victorias_Significativas': wins,
                'Derrotas_Significativas': losses,
                'Empates': ties,
                'Score_Neto': wins - losses
            })
        
        df_dominance = pd.DataFrame(dominance)
        df_dominance = df_dominance.sort_values('Score_Neto', ascending=False)
        df_dominance.to_csv(f'{output_dir}/6_dominancia_estadistica_dm.csv', index=False)
        
        # Visualización de dominancia
        fig, ax = plt.subplots(figsize=(12, 6))
        x = np.arange(len(df_dominance))
        width = 0.25
        
        ax.bar(x - width, df_dominance['Victorias_Significativas'], 
               width, label='Victorias', color='green', alpha=0.7)
        ax.bar(x, df_dominance['Empates'], 
               width, label='Empates', color='gray', alpha=0.7)
        ax.bar(x + width, df_dominance['Derrotas_Significativas'], 
               width, label='Derrotas', color='red', alpha=0.7)
        
        ax.set_xlabel('Modelo')
        ax.set_ylabel('Número de Comparaciones')
        ax.set_title('Dominancia Estadística (Test Diebold-Mariano)')
        ax.set_xticks(x)
        ax.set_xticklabels(df_dominance['Modelo'], rotation=45, ha='right')
        ax.legend()
        ax.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.savefig(f'{output_dir}/6_dominancia_dm.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        # Análisis de Diebold-Mariano por escenario
        print("  - Analizando DM por escenario...")
        dm_by_scenario = []
        for escenario in self.df_all['Escenario'].unique():
            subset = self.df_all[self.df_all['Escenario'] == escenario]
            
            for model1, model2 in combinations(self.models, 2):
                errors1 = subset[model1].values
                errors2 = subset[model2].values
                
                if len(errors1) > 0 and len(errors2) > 0:
                    dm_stat, p_value = DieboldMarianoTest.dm_test(errors1, errors2, h=1, crit="MSE")
                    mean_diff = subset[model1].mean() - subset[model2].mean()
                    
                    dm_by_scenario.append({
                        'Escenario': escenario,
                        'Modelo_1': model1,
                        'Modelo_2': model2,
                        'DM_Statistic': dm_stat,
                        'P_Value': p_value,
                        'Diferencia_Media': mean_diff,
                        'Significativo': 'Sí' if p_value < 0.05 else 'No'
                    })
        
        df_dm_scenario = pd.DataFrame(dm_by_scenario)
        df_dm_scenario.to_csv(f'{output_dir}/6_dm_por_escenario.csv', index=False)
    
    def analyze_individual_models(self, output_dir):
        """
        7. PERFILES INDIVIDUALES POR MODELO
        """
        print("  - Generando perfiles individuales...")
        
        for model in self.models:
            print(f"    > Analizando {model}...")
            
            # Crear subdirectorio para el modelo
            model_dir = f"{output_dir}/perfiles_modelos/{model.replace(' ', '_')}"
            import os
            os.makedirs(model_dir, exist_ok=True)
            
            # Reporte del modelo
            report = []
            report.append(f"="*80)
            report.append(f"PERFIL DETALLADO: {model}")
            report.append(f"="*80)
            report.append("")
            
            # Estadísticas generales
            report.append("1. ESTADÍSTICAS GENERALES")
            report.append("-" * 40)
            report.append(f"ECRPS Promedio Global: {self.df_all[model].mean():.6f}")
            report.append(f"Desviación Estándar: {self.df_all[model].std():.6f}")
            cv = self.df_all[model].std()/self.df_all[model].mean() if self.df_all[model].mean() != 0 else 0
            report.append(f"Coeficiente de Variación: {cv:.4f}")
            report.append(f"Mínimo: {self.df_all[model].min():.6f}")
            report.append(f"Mediana: {self.df_all[model].median():.6f}")
            report.append(f"Máximo: {self.df_all[model].max():.6f}")
            report.append("")
            
            # Ranking general
            mean_scores = self.df_all[self.models].mean()
            ranking = mean_scores.rank().astype(int)
            report.append(f"Ranking General: {ranking[model]}° de {len(self.models)}")
            report.append("")
            
            # Mejor escenario
            report.append("2. MEJORES ESCENARIOS")
            report.append("-" * 40)
            best_idx = self.df_all[model].idxmin()
            best_row = self.df_all.loc[best_idx]
            report.append(f"Mejor ECRPS: {best_row[model]:.6f}")
            report.append(f"  - Escenario: {best_row['Escenario']}")
            if 'Tipo de Modelo' in best_row:
                report.append(f"  - Tipo Modelo: {best_row['Tipo de Modelo']}")
            report.append(f"  - Distribución: {best_row['Distribución']}")
            report.append(f"  - Varianza: {best_row['Varianza']}")
            report.append(f"  - Paso: {best_row['Paso']}")
            report.append("")
            
            # Peor escenario
            report.append("3. PEORES ESCENARIOS")
            report.append("-" * 40)
            worst_idx = self.df_all[model].idxmax()
            worst_row = self.df_all.loc[worst_idx]
            report.append(f"Peor ECRPS: {worst_row[model]:.6f}")
            report.append(f"  - Escenario: {worst_row['Escenario']}")
            if 'Tipo de Modelo' in worst_row:
                report.append(f"  - Tipo Modelo: {worst_row['Tipo de Modelo']}")
            report.append(f"  - Distribución: {worst_row['Distribución']}")
            report.append(f"  - Varianza: {worst_row['Varianza']}")
            report.append(f"  - Paso: {worst_row['Paso']}")
            report.append("")
            
            # Rendimiento por escenario
            report.append("4. RENDIMIENTO POR ESCENARIO")
            report.append("-" * 40)
            for escenario in ['Estacionario_Lineal', 'No_Estacionario_Lineal', 'No_Lineal']:
                subset = self.df_all[self.df_all['Escenario'] == escenario]
                if len(subset) > 0:
                    mean_val = subset[model].mean()
                    rank = subset[self.models].mean().rank()[model]
                    report.append(f"{escenario}:")
                    report.append(f"  ECRPS: {mean_val:.6f} (Ranking: {int(rank)}°)")
            report.append("")
            
            # Fortalezas y debilidades
            report.append("5. FORTALEZAS Y DEBILIDADES")
            report.append("-" * 40)
            
            # Por distribución
            report.append("Por Distribución:")
            dist_performance = []
            for dist in self.df_all['Distribución'].unique():
                subset = self.df_all[self.df_all['Distribución'] == dist]
                if len(subset) > 0:
                    mean_val = subset[model].mean()
                    rank = subset[self.models].mean().rank()[model]
                    dist_performance.append((dist, mean_val, rank))
            
            if dist_performance:
                dist_performance.sort(key=lambda x: x[2])
                report.append(f"  Mejor: {dist_performance[0][0]} (Ranking {int(dist_performance[0][2])}°)")
                report.append(f"  Peor: {dist_performance[-1][0]} (Ranking {int(dist_performance[-1][2])}°)")
            report.append("")
            
            # Por varianza
            report.append("Por Varianza:")
            var_performance = []
            for var in sorted(self.df_all['Varianza'].unique()):
                subset = self.df_all[self.df_all['Varianza'] == var]
                if len(subset) > 0:
                    mean_val = subset[model].mean()
                    rank = subset[self.models].mean().rank()[model]
                    var_performance.append((var, mean_val, rank))
            
            if var_performance:
                var_performance.sort(key=lambda x: x[2])
                report.append(f"  Mejor: Varianza {var_performance[0][0]} (Ranking {int(var_performance[0][2])}°)")
                report.append(f"  Peor: Varianza {var_performance[-1][0]} (Ranking {int(var_performance[-1][2])}°)")
            report.append("")
            
            # Comparaciones con Diebold-Mariano
            report.append("6. COMPARACIONES ESTADÍSTICAS (DIEBOLD-MARIANO)")
            report.append("-" * 40)
            
            wins = 0
            losses = 0
            for other_model in self.models:
                if model != other_model:
                    errors1 = self.df_all[model].values
                    errors2 = self.df_all[other_model].values
                    _, p_val = DieboldMarianoTest.dm_test(errors1, errors2, h=1, crit="MSE")
                    mean_diff = self.df_all[model].mean() - self.df_all[other_model].mean()
                    
                    if p_val < 0.05:
                        if mean_diff < 0:
                            wins += 1
                        else:
                            losses += 1
            
            report.append(f"Victorias significativas: {wins}")
            report.append(f"Derrotas significativas: {losses}")
            report.append(f"Score neto: {wins - losses}")
            report.append("")
            
            # Guardar reporte
            with open(f"{model_dir}/perfil_{model.replace(' ', '_')}.txt", 'w', encoding='utf-8') as f:
                f.write('\n'.join(report))
            
            # Visualizaciones del modelo
            self._create_model_visualizations(model, model_dir)
    
    def _create_model_visualizations(self, model, model_dir):
        """Crea visualizaciones específicas para un modelo"""
        
        # 1. Distribución de ECRPS
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        
        # Histograma
        axes[0, 0].hist(self.df_all[model], bins=50, alpha=0.7, color='steelblue', edgecolor='black')
        axes[0, 0].axvline(self.df_all[model].mean(), color='red', linestyle='--', 
                          linewidth=2, label=f'Media: {self.df_all[model].mean():.4f}')
        axes[0, 0].axvline(self.df_all[model].median(), color='green', linestyle='--', 
                          linewidth=2, label=f'Mediana: {self.df_all[model].median():.4f}')
        axes[0, 0].set_xlabel('ECRPS')
        axes[0, 0].set_ylabel('Frecuencia')
        axes[0, 0].set_title(f'Distribución de ECRPS - {model}')
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)
        
        # Boxplot por escenario
        data_by_scenario = [self.df_all[self.df_all['Escenario'] == esc][model] 
                           for esc in ['Estacionario_Lineal', 'No_Estacionario_Lineal', 'No_Lineal']]
        bp = axes[0, 1].boxplot(data_by_scenario, labels=['Est. Lin.', 'No Est. Lin.', 'No Lin.'], 
                               patch_artist=True)
        for patch, color in zip(bp['boxes'], ['lightblue', 'lightcoral', 'lightgreen']):
            patch.set_facecolor(color)
        axes[0, 1].set_ylabel('ECRPS')
        axes[0, 1].set_title(f'ECRPS por Escenario - {model}')
        axes[0, 1].grid(True, alpha=0.3)
        
        # Rendimiento por paso
        paso_data = []
        for p in sorted(self.df_all['Paso'].unique()):
            subset = self.df_all[self.df_all['Paso'] == p]
            if len(subset) > 0:
                paso_data.append((p, subset[model].mean()))
        
        if paso_data:
            pasos, means = zip(*paso_data)
            axes[1, 0].plot(pasos, means, marker='o', linewidth=2, markersize=8, color='darkblue')
            axes[1, 0].set_xlabel('Paso de Predicción')
            axes[1, 0].set_ylabel('ECRPS Promedio')
            axes[1, 0].set_title(f'Rendimiento por Horizonte - {model}')
            axes[1, 0].grid(True, alpha=0.3)
        
        # Heatmap: Distribución × Varianza
        pivot_data = []
        dist_labels = []
        var_labels = sorted(self.df_all['Varianza'].unique())
        
        for dist in self.df_all['Distribución'].unique():
            row = []
            for var in var_labels:
                subset = self.df_all[(self.df_all['Distribución'] == dist) & 
                                    (self.df_all['Varianza'] == var)]
                if len(subset) > 0:
                    row.append(subset[model].mean())
                else:
                    row.append(np.nan)
            if not all(np.isnan(row)):
                pivot_data.append(row)
                dist_labels.append(dist)
        
        if pivot_data:
            pivot_df = pd.DataFrame(pivot_data, index=dist_labels, columns=var_labels)
            
            sns.heatmap(pivot_df, annot=True, fmt='.4f', cmap='RdYlGn_r', ax=axes[1, 1],
                       cbar_kws={'label': 'ECRPS'})
            axes[1, 1].set_title(f'ECRPS: Distribución × Varianza - {model}')
            axes[1, 1].set_xlabel('Varianza')
            axes[1, 1].set_ylabel('Distribución')
        
        plt.tight_layout()
        plt.savefig(f'{model_dir}/visualizaciones_{model.replace(" ", "_")}.png', 
                   dpi=300, bbox_inches='tight')
        plt.close()
        
        # 2. Comparación con otros modelos
        fig, ax = plt.subplots(figsize=(12, 8))
        
        means = self.df_all[self.models].mean().sort_values()
        colors = ['red' if m == model else 'steelblue' for m in means.index]
        bars = ax.barh(means.index, means.values, color=colors, alpha=0.7)
        
        # Destacar el modelo actual
        for i, bar in enumerate(bars):
            if means.index[i] == model:
                bar.set_edgecolor('black')
                bar.set_linewidth(3)
        
        ax.set_xlabel('ECRPS Promedio')
        ax.set_title(f'Comparación Global - {model} (Destacado en Rojo)')
        ax.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.savefig(f'{model_dir}/comparacion_{model.replace(" ", "_")}.png', 
                   dpi=300, bbox_inches='tight')
        plt.close()
    
    def generate_recommendations(self, output_dir):
        """
        8. GENERACIÓN DE RECOMENDACIONES
        """
        print("  - Generando recomendaciones estratégicas...")
        
        recommendations = []
        recommendations.append("="*80)
        recommendations.append("RECOMENDACIONES Y CONCLUSIONES")
        recommendations.append("="*80)
        recommendations.append("")
        
        # 1. Modelo campeón general
        overall_best = self.df_all[self.models].mean().idxmin()
        overall_worst = self.df_all[self.models].mean().idxmax()
        
        recommendations.append("1. MODELO CAMPEÓN GENERAL")
        recommendations.append("-" * 40)
        recommendations.append(f"Mejor rendimiento promedio: {overall_best}")
        recommendations.append(f"ECRPS: {self.df_all[overall_best].mean():.6f}")
        recommendations.append(f"Desviación Estándar: {self.df_all[overall_best].std():.6f}")
        recommendations.append("")
        recommendations.append(f"Peor rendimiento promedio: {overall_worst}")
        recommendations.append(f"ECRPS: {self.df_all[overall_worst].mean():.6f}")
        recommendations.append("")
        
        # 2. Modelos por escenario
        recommendations.append("2. RECOMENDACIONES POR ESCENARIO")
        recommendations.append("-" * 40)
        
        for escenario in ['Estacionario_Lineal', 'No_Estacionario_Lineal', 'No_Lineal']:
            subset = self.df_all[self.df_all['Escenario'] == escenario]
            if len(subset) > 0:
                best_model = subset[self.models].mean().idxmin()
                best_score = subset[best_model].mean()
                
                recommendations.append(f"\n{escenario}:")
                recommendations.append(f"  Modelo Recomendado: {best_model}")
                recommendations.append(f"  ECRPS Promedio: {best_score:.6f}")
        
        recommendations.append("")
        
        # 3. Modelos por distribución
        recommendations.append("3. RECOMENDACIONES POR DISTRIBUCIÓN DE ERRORES")
        recommendations.append("-" * 40)
        
        for dist in self.df_all['Distribución'].unique():
            subset = self.df_all[self.df_all['Distribución'] == dist]
            if len(subset) > 0:
                best_model = subset[self.models].mean().idxmin()
                best_score = subset[best_model].mean()
                
                recommendations.append(f"\nDistribución {dist}:")
                recommendations.append(f"  Modelo Recomendado: {best_model}")
                recommendations.append(f"  ECRPS Promedio: {best_score:.6f}")
        
        recommendations.append("")
        
        # 4. Modelos más robustos
        recommendations.append("4. MODELOS MÁS ROBUSTOS (MENOR VARIABILIDAD)")
        recommendations.append("-" * 40)
        
        cv_scores = {model: self.df_all[model].std() / self.df_all[model].mean() 
                    for model in self.models if self.df_all[model].mean() != 0}
        cv_sorted = sorted(cv_scores.items(), key=lambda x: x[1])
        
        for i, (model, cv) in enumerate(cv_sorted[:3], 1):
            recommendations.append(f"{i}. {model}: CV = {cv:.4f}")
        
        recommendations.append("")
        
        # 5. Modelos por horizonte
        recommendations.append("5. RECOMENDACIONES POR HORIZONTE DE PREDICCIÓN")
        recommendations.append("-" * 40)
        
        pasos_unicos = sorted(self.df_all['Paso'].unique())
        for paso in [pasos_unicos[0], pasos_unicos[len(pasos_unicos)//2], pasos_unicos[-1]]:
            subset = self.df_all[self.df_all['Paso'] == paso]
            if len(subset) > 0:
                best_model = subset[self.models].mean().idxmin()
                best_score = subset[best_model].mean()
                
                recommendations.append(f"\nPaso {paso}:")
                recommendations.append(f"  Modelo Recomendado: {best_model}")
                recommendations.append(f"  ECRPS Promedio: {best_score:.6f}")
        
        recommendations.append("")
        
        # 6. Estrategia de ensamble
        recommendations.append("6. ESTRATEGIA DE ENSAMBLE SUGERIDA")
        recommendations.append("-" * 40)
        
        # Top 3 modelos complementarios
        top3 = self.df_all[self.models].mean().nsmallest(3)
        recommendations.append("Combinar los siguientes modelos:")
        for i, (model, score) in enumerate(top3.items(), 1):
            recommendations.append(f"{i}. {model} (ECRPS: {score:.6f})")
        
        recommendations.append("")
        recommendations.append("Justificación:")
        recommendations.append("  - Estos modelos muestran el mejor rendimiento promedio")
        recommendations.append("  - Un ensamble puede capturar fortalezas complementarias")
        recommendations.append("  - Reduce el riesgo de seleccionar un modelo subóptimo")
        
        recommendations.append("")
        
        # 7. Modelos con dominancia estadística
        recommendations.append("7. MODELOS CON DOMINANCIA ESTADÍSTICA")
        recommendations.append("-" * 40)
        
        dominance_scores = []
        for model in self.models:
            wins = 0
            for other_model in self.models:
                if model != other_model:
                    errors1 = self.df_all[model].values
                    errors2 = self.df_all[other_model].values
                    _, p_val = DieboldMarianoTest.dm_test(errors1, errors2, h=1, crit="MSE")
                    mean_diff = self.df_all[model].mean() - self.df_all[other_model].mean()
                    
                    if p_val < 0.05 and mean_diff < 0:
                        wins += 1
            
            dominance_scores.append((model, wins))
        
        dominance_scores.sort(key=lambda x: x[1], reverse=True)
        
        recommendations.append("Modelos estadísticamente superiores (test Diebold-Mariano):")
        for i, (model, wins) in enumerate(dominance_scores[:5], 1):
            recommendations.append(f"{i}. {model}: {wins} victorias significativas")
        
        recommendations.append("")
        
        # 8. Reglas de decisión
        recommendations.append("8. REGLAS DE DECISIÓN SUGERIDAS")
        recommendations.append("-" * 40)
        recommendations.append("")
        
        # Reglas por escenario
        for escenario in ['Estacionario_Lineal', 'No_Estacionario_Lineal', 'No_Lineal']:
            subset = self.df_all[self.df_all['Escenario'] == escenario]
            if len(subset) > 0:
                top2 = subset[self.models].mean().nsmallest(2)
                
                if escenario == 'Estacionario_Lineal':
                    recommendations.append("SI el proceso es ESTACIONARIO y LINEAL:")
                elif escenario == 'No_Estacionario_Lineal':
                    recommendations.append("SI el proceso es NO ESTACIONARIO y LINEAL:")
                else:
                    recommendations.append("SI el proceso es NO LINEAL:")
                
                recommendations.append(f"  → Primera opción: {top2.index[0]}")
                recommendations.append(f"  → Segunda opción: {top2.index[1]}")
                recommendations.append("")
        
        # Reglas por distribución
        recommendations.append("SI la distribución de errores:")
        for dist in self.df_all['Distribución'].unique():
            subset = self.df_all[self.df_all['Distribución'] == dist]
            if len(subset) > 0:
                best = subset[self.models].mean().idxmin()
                recommendations.append(f"  • Es {dist} → Usar {best}")
        
        recommendations.append("")
        
        # Reglas por varianza
        recommendations.append("SI el nivel de varianza:")
        variances = sorted(self.df_all['Varianza'].unique())
        if len(variances) >= 2:
            low_var = variances[0]
            high_var = variances[-1]
            
            subset_low = self.df_all[self.df_all['Varianza'] == low_var]
            subset_high = self.df_all[self.df_all['Varianza'] == high_var]
            
            best_low = subset_low[self.models].mean().idxmin()
            best_high = subset_high[self.models].mean().idxmin()
            
            recommendations.append(f"  • Es bajo ({low_var}) → Usar {best_low}")
            recommendations.append(f"  • Es alto ({high_var}) → Usar {best_high}")
        
        recommendations.append("")
        
        # 9. Conclusiones finales
        recommendations.append("9. CONCLUSIONES PRINCIPALES")
        recommendations.append("-" * 40)
        recommendations.append("")
        recommendations.append(f"• El modelo {overall_best} muestra el mejor rendimiento general")
        recommendations.append(f"  con ECRPS promedio de {self.df_all[overall_best].mean():.6f}")
        recommendations.append("")
        
        # Análisis de robustez
        most_robust = min(cv_scores.items(), key=lambda x: x[1])[0]
        recommendations.append(f"• El modelo más robusto (menor CV) es {most_robust}")
        recommendations.append("")
        
        # Comparación estacionario vs no estacionario
        est_best = self.df_estacionario[self.models].mean().idxmin()
        no_est_best = self.df_no_estacionario[self.models].mean().idxmin()
        
        if est_best == no_est_best:
            recommendations.append(f"• {est_best} es consistentemente superior en procesos")
            recommendations.append("  estacionarios y no estacionarios")
        else:
            recommendations.append(f"• Para procesos estacionarios: preferir {est_best}")
            recommendations.append(f"• Para procesos no estacionarios: preferir {no_est_best}")
        recommendations.append("")
        
        # Análisis de no linealidad
        nl_best = self.df_no_lineal[self.models].mean().idxmin()
        recommendations.append(f"• Para procesos no lineales: {nl_best} es la mejor opción")
        recommendations.append("")
        
        # Recomendación de ensamble
        recommendations.append("• Se recomienda implementar un ENSAMBLE de los top 3 modelos")
        recommendations.append("  para maximizar robustez y rendimiento")
        recommendations.append("")
        
        # Consideraciones prácticas
        recommendations.append("10. CONSIDERACIONES PRÁCTICAS")
        recommendations.append("-" * 40)
        recommendations.append("")
        recommendations.append("Factores a considerar en la selección:")
        recommendations.append("  1. Costo computacional vs ganancia en precisión")
        recommendations.append("  2. Robustez ante cambios en la distribución de errores")
        recommendations.append("  3. Consistencia a través de horizontes de predicción")
        recommendations.append("  4. Facilidad de interpretación y explicabilidad")
        recommendations.append("  5. Disponibilidad de recursos para implementación")
        recommendations.append("")
        
        # Trade-offs identificados
        recommendations.append("Trade-offs identificados:")
        
        # Mejor vs más robusto
        if overall_best != most_robust:
            recommendations.append(f"  • Rendimiento vs Robustez: {overall_best} (mejor) vs {most_robust} (más robusto)")
        
        # Modelos especializados
        recommendations.append("  • Algunos modelos son especialistas en escenarios específicos")
        recommendations.append("  • Otros modelos son generalistas con buen rendimiento global")
        recommendations.append("")
        
        # Guardar recomendaciones
        with open(f'{output_dir}/8_recomendaciones.txt', 'w', encoding='utf-8') as f:
            f.write('\n'.join(recommendations))
        
        print('\n'.join(recommendations))


# ============================================================================
# CÓDIGO DE EJECUCIÓN PRINCIPAL
# ============================================================================

def main():
    """
    Función principal para ejecutar el análisis completo
    """
    print("\n" + "="*80)
    print("ANÁLISIS COMPREHENSIVO DE MODELOS DE PREDICCIÓN PROBABILÍSTICA")
    print("="*80 + "\n")
    
    # Crear analizador
    try:
        analyzer = ModelPerformanceAnalyzer()
    except FileNotFoundError:
        print("\nERROR: No se encontraron los archivos de datos")
        print("Verifica que existan los siguientes archivos:")
        print("  - ./Datos/estacionario.xlsx")
        print("  - ./Datos/no_estacionario.xlsx")
        print("  - ./Datos/no_lineal.xlsx")
        return
    except Exception as e:
        print(f"\nERROR al cargar datos: {e}")
        import traceback
        traceback.print_exc()
        return
    
    # Ejecutar análisis completo
    output_directory = 'resultados_analisis_completo'
    
    try:
        analyzer.generate_full_report(output_dir=output_directory)
        
        print(f"\n{'='*80}")
        print(f"✓ Análisis completado exitosamente")
        print(f"✓ Todos los resultados guardados en: {output_directory}/")
        print(f"{'='*80}\n")
        
        print("Archivos generados:")
        print("  📊 Análisis de características del DGP")
        print("  📈 Efectos de distribución y varianza")
        print("  🎯 Análisis de horizonte de predicción")
        print("  🔄 Interacciones complejas")
        print("  💪 Métricas de robustez")
        print("  📉 Tests de Diebold-Mariano")
        print("  👤 Perfiles individuales por modelo")
        print("  💡 Recomendaciones estratégicas")
        print("")
        
    except Exception as e:
        print(f"\n❌ ERROR durante el análisis: {e}")
        import traceback
        traceback.print_exc()


if __name__ == "__main__":
    main()


ANÁLISIS COMPREHENSIVO DE MODELOS DE PREDICCIÓN PROBABILÍSTICA

Cargando datos...
✓ Estacionario: 1320 filas
  Columnas: ['Paso', 'Valores de AR', 'Valores MA', 'Distribución', 'Varianza error', 'AREPD', 'AV-MCPS', 'Block Bootstrapping', 'DeepAR', 'EnCQR-LSTM', 'LSPM', 'LSPMW', 'MCPS', 'Sieve Bootstrap', 'Mejor Modelo', 'Escenario']
✓ No Estacionario: 840 filas
  Columnas: ['Paso', 'Tipo de Modelo', 'Valores de AR', 'Valores MA', 'Distribución', 'Varianza error', 'AREPD', 'AV-MCPS', 'Block Bootstrapping', 'DeepAR', 'EnCQR-LSTM', 'LSPM', 'LSPMW', 'MCPS', 'Sieve Bootstrap', 'Mejor Modelo', 'Escenario']
✓ No Lineal: 840 filas
  Columnas: ['Paso', 'Tipo de Modelo', 'Distribución', 'Varianza error', 'AREPD', 'AV-MCPS', 'Block Bootstrapping', 'DeepAR', 'EnCQR-LSTM', 'LSPM', 'LSPMW', 'MCPS', 'Sieve Bootstrap', 'Mejor Modelo', 'Escenario']
✓ Tipos de datos convertidos
✓ Filas después de limpieza: 2600

✓ Datos combinados: 2600 observaciones totales
✓ Columnas finales: ['Paso', 'Valores de AR'

# Pre analisis

In [12]:
import pandas as pd
import re
    
estacionario = pd.read_excel("./Datos/estacionario.xlsx")

estacionario = estacionario.drop_duplicates()
estacionario = estacionario[estacionario["Paso"] != "Promedio"]

def determinar_tipo_modelo_mejorado(row):
    """
    Determina el tipo de modelo (AR, MA, ARMA) y su orden a partir de los valores
    en las columnas 'Valores de AR' y 'Valores MA'.
    """
    ar_str = str(row['Valores de AR'])
    ma_str = str(row['Valores MA'])
    
    # Expresión regular para encontrar números (enteros o decimales, positivos o negativos)
    regex_numeros = r'-?\d+\.?\d*'
    
    # Cuenta cuántos números válidos hay en cada string
    p = len(re.findall(regex_numeros, ar_str))
    q = len(re.findall(regex_numeros, ma_str))
    
    if p > 0 and q == 0:
        return f"AR({p})"
    elif p == 0 and q > 0:
        return f"MA({q})"
    elif p > 0 and q > 0:
        return f"ARMA({p},{q})"
    else:
        return None # O "Ruido Blanco" si p=0 y q=0

# Aplica la función mejorada para crear la columna "Tipo de Modelo"
estacionario['Tipo de Modelo'] = estacionario.apply(determinar_tipo_modelo_mejorado, axis=1)

# Imprime los valores únicos de la columna Tipo de modelo para verificar
print("Valores únicos encontrados en 'Tipo de Modelo':")
print(estacionario['Tipo de Modelo'].unique())

# Ordena las columnas 'Paso' y 'Tipo de modelo' al inicio
cols = estacionario.columns.tolist()
# Aseguramos que las columnas existan antes de moverlas
if 'Paso' in cols:
    cols.insert(0, cols.pop(cols.index('Paso')))
if 'Tipo de Modelo' in cols:
    cols.insert(1, cols.pop(cols.index('Tipo de Modelo')))

estacionario = estacionario.reindex(columns=cols)


# Borra las columnas originales 'Valores de AR' y 'Valores MA'
estacionario = estacionario.drop(columns=['Valores de AR', 'Valores MA'])
estacionario["Escenario"] = "Estacionario_Lineal"

# Muestra el DataFrame resultante
estacionario

Valores únicos encontrados en 'Tipo de Modelo':
['AR(1)' 'AR(2)' 'MA(1)' 'MA(2)' 'ARMA(1,1)' 'ARMA(2,2)']


,Paso,Tipo de Modelo,Distribución,Varianza error,AREPD,AV-MCPS,Block Bootstrapping,DeepAR,EnCQR-LSTM,LSPM,LSPMW,MCPS,Sieve Bootstrap,Mejor Modelo,Escenario
0,1,AR(1),normal,0.2,0.294667,0.355344,0.248447,0.263419,0.306622,0.440706,0.431452,0.285427,0.248691,Block Bootstrapping,Estacionario_Lineal
2,2,AR(1),normal,0.2,0.604540,0.307449,0.254264,0.273001,0.565522,0.470424,0.474111,0.285430,0.254193,Sieve Bootstrap,Estacionario_Lineal
4,3,AR(1),normal,0.2,0.273622,0.276230,0.258388,0.315765,0.269452,0.520070,0.517876,0.337990,0.258039,Sieve Bootstrap,Estacionario_Lineal
6,4,AR(1),normal,0.2,0.261423,0.279697,0.254453,0.289443,0.269285,0.287989,0.288111,0.282999,0.254655,Block Bootstrapping,Estacionario_Lineal
8,5,AR(1),normal,0.2,0.626252,0.273680,0.254842,0.272827,0.639437,0.763960,0.753066,0.308347,0.254952,Block Bootstrapping,Estacionario_Lineal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1309,1,"ARMA(2,2)",mixture,3.0,1.082513,0.999066,0.953857,1.116455,1.053269,2.030504,2.165650,0.990087,0.954156,Block Bootstrapping,Estacionario_Lineal
1311,2,"ARMA(2,2)",mixture,3.0,1.903173,0.971148,0.954440,1.005615,1.518301,1.431610,1.522051,1.141614,0.954065,Sieve Bootstrap,Estacionario_Lineal
1313,3,"ARMA(2,2)",mixture,3.0,2.310542,1.021845,0.976235,1.002865,1.615073,1.026140,1.036051,1.484601,0.962417,Sieve Bootstrap,Estacionario_Lineal
1315,4,"ARMA(2,2)",mixture,3.0,1.324103,0.968827,0.961514,0.977739,1.072897,1.453428,1.530595,1.125230,0.960919,Sieve Bootstrap,Estacionario_Lineal


In [20]:
no_estacionario = pd.read_excel("./Datos/no_estacionario.xlsx")
no_estacionario.drop(columns=['Valores de AR', 'Valores MA'], inplace=True)
no_estacionario["Escenario"] = "No_Estacionario_Lineal"
no_estacionario = no_estacionario[no_estacionario["Paso"] != "Promedio"]
no_estacionario

,Paso,Tipo de Modelo,Distribución,Varianza error,AREPD,AV-MCPS,Block Bootstrapping,DeepAR,EnCQR-LSTM,LSPM,LSPMW,MCPS,Sieve Bootstrap,Mejor Modelo,Escenario
0,1,"ARIMA(0,1,0)",normal,0.2,1.860823,0.258474,0.253635,0.319481,0.488711,0.367279,0.360494,0.270816,0.273828,Block Bootstrapping,No_Estacionario_Lineal
1,2,"ARIMA(0,1,0)",normal,0.2,1.244128,0.528968,0.275061,0.438099,0.322919,0.426187,0.430296,0.576792,0.272952,Sieve Bootstrap,No_Estacionario_Lineal
2,3,"ARIMA(0,1,0)",normal,0.2,1.799818,0.864295,0.272406,0.291500,0.396481,0.642530,0.639134,0.269655,0.275661,MCPS,No_Estacionario_Lineal
3,4,"ARIMA(0,1,0)",normal,0.2,1.912421,0.481159,0.255186,0.291577,0.495882,0.341570,0.341227,0.533788,0.275948,Block Bootstrapping,No_Estacionario_Lineal
4,5,"ARIMA(0,1,0)",normal,0.2,2.822771,0.792130,0.257461,0.658698,1.291283,0.981902,0.969842,1.455485,0.338116,Block Bootstrapping,No_Estacionario_Lineal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
834,1,"ARIMA(2,1,2)",mixture,3.0,76.766114,5.668568,0.965836,7.254422,13.176312,4.421885,4.173484,20.231134,2.612414,Block Bootstrapping,No_Estacionario_Lineal
835,2,"ARIMA(2,1,2)",mixture,3.0,80.630681,6.161741,0.974398,8.767931,9.287902,1.733689,1.596168,21.251698,1.956761,Block Bootstrapping,No_Estacionario_Lineal
836,3,"ARIMA(2,1,2)",mixture,3.0,86.539087,10.452450,0.982561,24.631292,18.639842,6.195609,5.953120,23.480752,3.623684,Block Bootstrapping,No_Estacionario_Lineal
837,4,"ARIMA(2,1,2)",mixture,3.0,93.057798,13.911382,0.958507,27.567728,17.852720,7.288522,6.952830,28.286507,4.681807,Block Bootstrapping,No_Estacionario_Lineal


In [21]:
no_lineal = pd.read_excel("./Datos/no_lineal.xlsx")
no_lineal = no_lineal[no_lineal["Paso"] != "Promedio"]
no_lineal["Escenario"] = "No_Lineal_Estacionario"
no_lineal

,Paso,Tipo de Modelo,Distribución,Varianza error,AREPD,AV-MCPS,Block Bootstrapping,DeepAR,EnCQR-LSTM,LSPM,LSPMW,MCPS,Sieve Bootstrap,Mejor Modelo,Escenario
0,1,"SETAR(2,1)",normal,0.2,0.257043,0.253521,0.251524,0.263274,0.257984,0.285655,0.282110,0.257015,0.251188,Sieve Bootstrap,No_Lineal_Estacionario
1,2,"SETAR(2,1)",normal,0.2,0.305723,0.383340,0.288529,0.297164,0.324101,0.316846,0.319675,0.347319,0.290022,Block Bootstrapping,No_Lineal_Estacionario
2,3,"SETAR(2,1)",normal,0.2,0.292055,0.258555,0.287265,0.275374,0.278881,0.320347,0.320181,0.270736,0.262183,AV-MCPS,No_Lineal_Estacionario
3,4,"SETAR(2,1)",normal,0.2,0.298469,0.269290,0.263802,0.255605,0.270449,0.290893,0.290581,0.329900,0.258734,DeepAR,No_Lineal_Estacionario
4,5,"SETAR(2,1)",normal,0.2,0.298007,0.368342,0.501202,0.323900,0.348571,0.326254,0.329508,0.423889,0.442319,AREPD,No_Lineal_Estacionario
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
834,1,"SETAR(2,3)",mixture,3.0,1.164445,0.992519,0.962026,0.989297,1.046459,0.971555,1.076860,1.003262,0.961513,Sieve Bootstrap,No_Lineal_Estacionario
835,2,"SETAR(2,3)",mixture,3.0,1.191648,1.034591,0.986347,1.077081,0.972263,0.957417,0.986525,0.963721,0.984072,LSPM,No_Lineal_Estacionario
836,3,"SETAR(2,3)",mixture,3.0,1.193252,1.387456,1.012627,0.981861,0.955903,0.987603,0.977201,1.041540,1.009812,EnCQR-LSTM,No_Lineal_Estacionario
837,4,"SETAR(2,3)",mixture,3.0,1.229893,1.182221,1.124342,0.983326,0.960088,1.036372,0.978720,1.029875,1.103310,EnCQR-LSTM,No_Lineal_Estacionario


In [22]:
# Une los tres DataFrames en uno solo uno debajo de otro
df_all = pd.concat([estacionario, no_estacionario, no_lineal], ignore_index=True)
# Guarda el DataFrame combinado en un archivo Excel
df_all.to_excel("./Datos/datos_combinados.xlsx", index=False)

# Analisis con la correcion del profe

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os
from scipy import stats
from itertools import combinations

# ============================================================================
# CONFIGURACIÓN
# ============================================================================
RUTA_DATOS = "./Datos/datos_combinados.xlsx"
CARPETA_RESULTADOS = "resultados_completos_media_mediana"
MODELOS = ['AREPD', 'AV-MCPS', 'Block Bootstrapping', 'DeepAR', 
           'EnCQR-LSTM', 'LSPM', 'LSPMW', 'MCPS', 'Sieve Bootstrap']
ESCENARIOS_ESTACIONARIOS = ['Estacionario_Lineal', 'No_Lineal_Estacionario']
ESCENARIOS_NO_ESTACIONARIOS = ['No_Estacionario_Lineal']
ESCENARIOS_LINEALES = ['Estacionario_Lineal', 'No_Estacionario_Lineal']
ESCENARIOS_NO_LINEALES = ['No_Lineal_Estacionario']

# ============================================================================
# CLASE PARA TEST ESTADÍSTICO
# ============================================================================
class DieboldMarianoTest:
    @staticmethod
    def dm_test(errors1, errors2, h=1, power=2):
        # Implementación del test... (sin cambios)
        errors1, errors2 = np.array(errors1), np.array(errors2)
        loss_diff = (errors1**power) - (errors2**power)
        mean_diff = np.mean(loss_diff)
        n = len(loss_diff)
        gamma0 = np.var(loss_diff, ddof=1)
        if h > 1:
            gamma_sum = sum((1 - k/h) * np.cov(loss_diff[:-k], loss_diff[k:])[0, 1] for k in range(1, h))
            variance = (gamma0 + 2 * gamma_sum) / n
        else:
            variance = gamma0 / n
        dm_stat = mean_diff / np.sqrt(variance) if variance > 0 else 0
        p_value = 2 * (1 - stats.norm.cdf(np.abs(dm_stat)))
        return dm_stat, p_value

# ============================================================================
# FUNCIONES DE ANÁLISIS Y VISUALIZACIÓN (MODIFICADAS)
# ============================================================================
def crear_directorio_resultados(nombre_carpeta):
    if not os.path.exists(nombre_carpeta):
        os.makedirs(nombre_carpeta)
        print(f"Directorio '{nombre_carpeta}' creado.")

def guardar_grafico(nombre_archivo):
    ruta_completa = os.path.join(CARPETA_RESULTADOS, nombre_archivo)
    plt.savefig(ruta_completa, dpi=300, bbox_inches='tight')
    plt.close()

def graficar_comparacion_barras(promedios1, promedios2, orden, etiqueta1, etiqueta2, agg_method, nombre_archivo):
    """Grafica la comparación de barras para media o mediana."""
    fig, ax = plt.subplots(figsize=(14, 8))
    x = np.arange(len(orden))
    width = 0.35
    bars1 = ax.bar(x - width/2, promedios1[orden], width, label=etiqueta1, alpha=0.8, color='#3498db')
    bars2 = ax.bar(x + width/2, promedios2[orden], width, label=etiqueta2, alpha=0.8, color='#e74c3c')
    
    ylabel = f'ECRPS {"Promedio" if agg_method == "mean" else "Mediano"} (menor es mejor)'
    titulo = f'Comparación de Desempeño ({agg_method.capitalize()})'
    
    ax.set_xlabel('Modelos', fontsize=12, fontweight='bold')
    ax.set_ylabel(ylabel, fontsize=12, fontweight='bold')
    ax.set_title(titulo, fontsize=14, fontweight='bold', pad=20)
    ax.set_xticks(x)
    ax.set_xticklabels(orden, rotation=45, ha='right')
    ax.legend(fontsize=11)
    ax.grid(axis='y', alpha=0.3, linestyle='--')
    for bars in [bars1, bars2]:
        for bar in bars:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width() / 2., height, f'{height:.3f}', ha='center', va='bottom', fontsize=8)
    plt.tight_layout()
    guardar_grafico(nombre_archivo)

def generar_heatmap(data, agg_method, titulo_sufijo, nombre_archivo, figsize=(14, 8)):
    """Genera un heatmap basado en media o mediana."""
    fig, ax = plt.subplots(figsize=figsize)
    cbar_label = f'ECRPS {"Promedio" if agg_method == "mean" else "Mediano"}'
    titulo = f'Heatmap: {titulo_sufijo} ({agg_method.capitalize()})'
    
    sns.heatmap(data, annot=True, fmt='.3f', cmap='RdYlGn_r',
                cbar_kws={'label': cbar_label},
                linewidths=0.5, linecolor='gray', ax=ax)
    ax.set_title(titulo, fontsize=14, fontweight='bold', pad=20)
    ax.set_xlabel('Modelos', fontsize=12, fontweight='bold')
    ax.set_ylabel(data.index.name, fontsize=12, fontweight='bold')
    plt.tight_layout()
    guardar_grafico(nombre_archivo)

def graficar_evolucion_metrica_por_tipo(df, metrica_eje_x, agg_method, xlabel, nombre_archivo_sufijo):
    """Genera gráficos de evolución para cada tipo de modelo, usando media o mediana."""
    valores_unicos = sorted(df[metrica_eje_x].unique())
    tipos_modelo_unicos = df['Tipo de Modelo'].unique()
    
    for tipo in tipos_modelo_unicos:
        df_tipo = df[df['Tipo de Modelo'] == tipo]
        fig, ax = plt.subplots(figsize=(12, 7))
        
        for modelo in MODELOS:
            agregados = [df_tipo[df_tipo[metrica_eje_x] == val][modelo].agg(agg_method) for val in valores_unicos]
            if not all(np.isnan(agregados)):
                ax.plot(valores_unicos, agregados, marker='o', linewidth=2, markersize=8, label=modelo, alpha=0.8)
        
        ylabel = f'ECRPS {"Promedio" if agg_method == "mean" else "Mediano"}'
        titulo = f'ECRPS vs {metrica_eje_x} ({agg_method.capitalize()}) - Tipo: {tipo}'
        
        ax.set_xlabel(xlabel, fontsize=12, fontweight='bold')
        ax.set_ylabel(ylabel, fontsize=12, fontweight='bold')
        ax.set_title(titulo, fontsize=13, fontweight='bold', pad=15)
        ax.legend(fontsize=9, loc='best', ncol=2)
        ax.grid(True, alpha=0.3, linestyle='--')
        if metrica_eje_x == 'Paso':
            ax.set_xticks(valores_unicos)
            
        plt.tight_layout()
        nombre_archivo_tipo = f'ecrps_vs_{nombre_archivo_sufijo}_tipo_{tipo.replace(" ", "_").lower()}_{agg_method}.png'
        guardar_grafico(nombre_archivo_tipo)

def analizar_robustez_estabilidad(df, agg_method):
    """Calcula y grafica métricas de robustez y estabilidad."""
    print(f" -> Analizando robustez y estabilidad (basado en {agg_method})...")
    
    if agg_method == 'mean':
        # Análisis basado en la media (como antes)
        metrics = [{'Modelo': m, 'Centralidad': df[m].mean(), 'Dispersion': df[m].std()} for m in MODELOS]
        df_robust = pd.DataFrame(metrics)
        label_centralidad = 'ECRPS Promedio (Rendimiento)'
        label_dispersion = 'Desviación Estándar (Estabilidad)'
        titulo_compromiso = 'Compromiso Rendimiento vs. Estabilidad (Media vs Std)'
        
    else: # agg_method == 'median'
        # Análisis basado en la mediana (más robusto a outliers)
        metrics = [{'Modelo': m, 'Centralidad': df[m].median(), 'Dispersion': df[m].quantile(0.75) - df[m].quantile(0.25)} for m in MODELOS]
        df_robust = pd.DataFrame(metrics)
        label_centralidad = 'ECRPS Mediano (Rendimiento Típico)'
        label_dispersion = 'Rango Intercuartílico (IQR - Estabilidad Robusta)'
        titulo_compromiso = 'Compromiso Rendimiento vs. Estabilidad (Mediana vs IQR)'

    # Gráfico de dispersión Rendimiento vs Estabilidad
    fig, ax = plt.subplots(figsize=(12, 8))
    sns.scatterplot(data=df_robust, x='Centralidad', y='Dispersion', hue='Modelo', s=150, alpha=0.8, ax=ax)
    for _, row in df_robust.iterrows():
        ax.text(row['Centralidad'], row['Dispersion'], row['Modelo'], fontsize=9, ha='left', va='bottom')
    ax.set_xlabel(label_centralidad, fontweight='bold')
    ax.set_ylabel(label_dispersion, fontweight='bold')
    ax.set_title(titulo_compromiso, fontsize=14, fontweight='bold')
    ax.grid(True, linestyle='--', alpha=0.6)
    ax.legend(title='Modelos', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    guardar_grafico(f"6_compromiso_rendimiento_estabilidad_{agg_method}.png")

# --- Funciones que no dependen de la agregación (se ejecutan una sola vez) ---
def graficar_densidades_individuales(df):
    """Crea un gráfico de densidad individual para cada modelo."""
    print(" -> Generando gráficos de densidad individuales (análisis único)...")
    all_ecrps_values = df[MODELOS].values.flatten()
    xlim_max = np.quantile(all_ecrps_values[~np.isnan(all_ecrps_values)], 0.995)
    for modelo in MODELOS:
        fig, ax = plt.subplots(figsize=(8, 5))
        sns.kdeplot(df[modelo].dropna(), fill=True, color='teal', ax=ax, lw=2.5)
        mean_val, median_val = df[modelo].mean(), df[modelo].median()
        ax.axvline(mean_val, color='red', linestyle='--', label=f'Media: {mean_val:.3f}')
        ax.axvline(median_val, color='green', linestyle=':', label=f'Mediana: {median_val:.3f}')
        ax.set_title(f'Distribución del ECRPS - Modelo: {modelo}', fontsize=14, fontweight='bold')
        ax.set_xlabel('ECRPS', fontweight='bold')
        ax.set_ylabel('Densidad', fontweight='bold')
        ax.set_xlim(left=0, right=xlim_max)
        ax.legend()
        ax.grid(True, linestyle='--', alpha=0.5)
        plt.tight_layout()
        guardar_grafico(f"7_densidad_{modelo.replace(' ', '_').lower()}.png")

def realizar_test_diebold_mariano(df):
    """Realiza el test de Diebold-Mariano con corrección de Bonferroni."""
    print(" -> Realizando Test de Diebold-Mariano (análisis único)...")
    pairs = list(combinations(MODELOS, 2))
    alpha_bonferroni = 0.05 / len(pairs)
    dm_results = []
    for m1, m2 in pairs:
        e1, e2 = df[m1].dropna(), df[m2].dropna()
        min_len = min(len(e1), len(e2))
        _, p_value = DieboldMarianoTest.dm_test(e1[:min_len], e2[:min_len])
        winner = 'Empate' if p_value >= alpha_bonferroni else (m1 if df[m1].mean() < df[m2].mean() else m2)
        dm_results.append({'Modelo_1': m1, 'Modelo_2': m2, 'Ganador_Bonferroni': winner})
    
    # Heatmap de resultados
    result_matrix = pd.DataFrame(index=MODELOS, columns=MODELOS, data=0)
    for _, row in pd.DataFrame(dm_results).iterrows():
        if row['Ganador_Bonferroni'] == row['Modelo_1']:
            result_matrix.loc[row['Modelo_1'], row['Modelo_2']], result_matrix.loc[row['Modelo_2'], row['Modelo_1']] = 1, -1
        elif row['Ganador_Bonferroni'] == row['Modelo_2']:
            result_matrix.loc[row['Modelo_1'], row['Modelo_2']], result_matrix.loc[row['Modelo_2'], row['Modelo_1']] = -1, 1

    annot_matrix = result_matrix.applymap(lambda x: {1: 'Gana', -1: 'Pierde', 0: 'Empate'}[x])
    fig, ax = plt.subplots(figsize=(12, 10))
    sns.heatmap(result_matrix.astype(float), annot=annot_matrix, fmt='s', cmap=['red', 'lightgray', 'green'], cbar=False, ax=ax)
    ax.set_title('Resultado Test Diebold-Mariano (con corrección de Bonferroni)', fontweight='bold')
    guardar_grafico("8_dm_heatmap_bonferroni.png")

# ============================================================================
# SCRIPT PRINCIPAL
# ============================================================================
def main():
    crear_directorio_resultados(CARPETA_RESULTADOS)
    try:
        df = pd.read_excel(RUTA_DATOS)
        print("✓ Datos cargados exitosamente.")
    except FileNotFoundError:
        print(f"ERROR: No se encontró el archivo en la ruta '{RUTA_DATOS}'.")
        return

    # Bucle principal para ejecutar análisis por media y mediana
    for agg_method in ['mean', 'median']:
        print(f"\n{'='*80}\n--- INICIANDO ANÁLISIS BASADO EN LA {agg_method.upper()} ---\n{'='*80}")

        # --- ANÁLISIS 1: ESTACIONARIEDAD ---
        print(f" -> 1. Analizando por estacionariedad ({agg_method})...")
        df_est = df[df['Escenario'].isin(ESCENARIOS_ESTACIONARIOS)]
        df_no_est = df[df['Escenario'].isin(ESCENARIOS_NO_ESTACIONARIOS)]
        agregados_est = df_est[MODELOS].agg(agg_method)
        agregados_no_est = df_no_est[MODELOS].agg(agg_method)
        orden_est = (agregados_est + agregados_no_est).sort_values().index
        graficar_comparacion_barras(agregados_est, agregados_no_est, orden_est, 'Estacionarios', 'No Estacionarios', agg_method, f'1_comparacion_estacionariedad_{agg_method}.png')
        
        # --- ANÁLISIS 2: LINEALIDAD ---
        print(f" -> 2. Analizando por linealidad ({agg_method})...")
        df_lin = df[df['Escenario'].isin(ESCENARIOS_LINEALES)]
        df_no_lin = df[df['Escenario'].isin(ESCENARIOS_NO_LINEALES)]
        agregados_lin = df_lin[MODELOS].agg(agg_method)
        agregados_no_lin = df_no_lin[MODELOS].agg(agg_method)
        orden_lin = (agregados_lin + agregados_no_lin).sort_values().index
        graficar_comparacion_barras(agregados_lin, agregados_no_lin, orden_lin, 'Lineales', 'No Lineales', agg_method, f'2_comparacion_linealidad_{agg_method}.png')

        # --- ANÁLISIS 3: HEATMAPS GENERALES ---
        print(f" -> 3. Generando heatmaps generales ({agg_method})...")
        heatmap_esc_df = df.groupby('Escenario')[MODELOS].agg(agg_method)
        generar_heatmap(heatmap_esc_df, agg_method, 'Desempeño por Escenario', f'3_heatmap_escenario_{agg_method}.png', figsize=(14, 6))
        heatmap_dist_df = df.groupby('Distribución')[MODELOS].agg(agg_method)
        generar_heatmap(heatmap_dist_df, agg_method, 'Desempeño por Distribución', f'3_heatmap_distribucion_{agg_method}.png')

        # --- ANÁLISIS 4 & 5: EVOLUCIÓN VS VARIANZA Y PASO ---
        print(f" -> 4. Analizando ECRPS vs Varianza ({agg_method})...")
        graficar_evolucion_metrica_por_tipo(df, 'Varianza error', agg_method, 'Varianza error', 'varianza')
        print(f" -> 5. Analizando ECRPS vs Paso ({agg_method})...")
        graficar_evolucion_metrica_por_tipo(df, 'Paso', agg_method, 'Paso (Horizonte)', 'paso')
        
        # --- ANÁLISIS 6: ROBUSTEZ Y ESTABILIDAD ---
        analizar_robustez_estabilidad(df, agg_method)

    # --- ANÁLISIS QUE SE EJECUTAN UNA SOLA VEZ ---
    print(f"\n{'='*80}\n--- INICIANDO ANÁLISIS INDEPENDIENTES DE AGREGACIÓN ---\n{'='*80}")
    # --- ANÁLISIS 7: DENSIDAD DE ERRORES ---
    graficar_densidades_individuales(df)
    
    # --- ANÁLISIS 8: TEST DE DIEBOLD-MARIANO ---
    realizar_test_diebold_mariano(df)
    
    print(f"\n✓ Análisis completo. Resultados guardados en la carpeta '{CARPETA_RESULTADOS}'.")

if __name__ == "__main__":
    main()

Directorio 'resultados_completos_media_mediana' creado.
✓ Datos cargados exitosamente.

--- INICIANDO ANÁLISIS BASADO EN LA MEAN ---
 -> 1. Analizando por estacionariedad (mean)...
 -> 2. Analizando por linealidad (mean)...
 -> 3. Generando heatmaps generales (mean)...
 -> 4. Analizando ECRPS vs Varianza (mean)...
 -> 5. Analizando ECRPS vs Paso (mean)...
 -> Analizando robustez y estabilidad (basado en mean)...

--- INICIANDO ANÁLISIS BASADO EN LA MEDIAN ---
 -> 1. Analizando por estacionariedad (median)...
 -> 2. Analizando por linealidad (median)...
 -> 3. Generando heatmaps generales (median)...
 -> 4. Analizando ECRPS vs Varianza (median)...
 -> 5. Analizando ECRPS vs Paso (median)...
 -> Analizando robustez y estabilidad (basado en median)...

--- INICIANDO ANÁLISIS INDEPENDIENTES DE AGREGACIÓN ---
 -> Generando gráficos de densidad individuales (análisis único)...
 -> Realizando Test de Diebold-Mariano (análisis único)...


C:\Users\pleal\AppData\Local\Temp\ipykernel_16608\4105642524.py:199: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  annot_matrix = result_matrix.applymap(lambda x: {1: 'Gana', -1: 'Pierde', 0: 'Empate'}[x])



✓ Análisis completo. Resultados guardados en la carpeta 'resultados_completos_media_mediana'.


# Machine Learning

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# ============================================================================
# CONFIGURACIÓN
# ============================================================================
RUTA_DATOS = "./Datos/datos_combinados.xlsx"
CARPETA_RESULTADOS = "resultados_meta_modelo"
MODELOS = ['AREPD', 'AV-MCPS', 'Block Bootstrapping', 'DeepAR', 
           'EnCQR-LSTM', 'LSPM', 'LSPMW', 'MCPS', 'Sieve Bootstrap']

# ============================================================================
# FUNCIONES AUXILIARES
# ============================================================================
def crear_directorio_resultados(nombre_carpeta):
    """Crea la carpeta de resultados si no existe."""
    if not os.path.exists(nombre_carpeta):
        os.makedirs(nombre_carpeta)
        print(f"Directorio '{nombre_carpeta}' creado.")

def guardar_grafico(nombre_archivo):
    """Guarda la figura actual en un archivo y la cierra."""
    ruta_completa = os.path.join(CARPETA_RESULTADOS, nombre_archivo)
    plt.savefig(ruta_completa, dpi=300, bbox_inches='tight')
    print(f" -> Gráfico guardado en: {ruta_completa}")
    plt.close()

def plot_feature_importance(model, feature_names, model_name):
    """Grafica la importancia de las características del modelo."""
    importances = model.feature_importances_
    df_importance = pd.DataFrame({
        'Característica': feature_names,
        'Importancia': importances
    }).sort_values(by='Importancia', ascending=True)

    fig, ax = plt.subplots(figsize=(10, 8))
    ax.barh(df_importance['Característica'], df_importance['Importancia'], color='steelblue')
    ax.set_xlabel('Importancia')
    ax.set_title(f'Importancia de Características - {model_name}')
    plt.tight_layout()
    guardar_grafico(f"feature_importance_{model_name.replace(' ', '_').lower()}.png")

def plot_confusion_matrix(y_true, y_pred, model_name, class_labels):
    """Grafica la matriz de confusión normalizada."""
    cm = confusion_matrix(y_true, y_pred, labels=class_labels, normalize='true')
    df_cm = pd.DataFrame(cm, index=class_labels, columns=class_labels)

    fig, ax = plt.subplots(figsize=(14, 12))
    sns.heatmap(df_cm, annot=True, fmt='.2f', cmap='Blues', ax=ax)
    ax.set_xlabel('Predicción del Recomendador', fontweight='bold')
    ax.set_ylabel('Mejor Modelo Real', fontweight='bold')
    ax.set_title(f'Matriz de Confusión Normalizada - {model_name}', fontsize=16, fontweight='bold')
    plt.tight_layout()
    guardar_grafico(f"confusion_matrix_{model_name.replace(' ', '_').lower()}.png")

# ============================================================================
# SCRIPT PRINCIPAL
# ============================================================================
def main():
    """Función principal para crear y analizar el meta-modelo."""
    crear_directorio_resultados(CARPETA_RESULTADOS)
    
    # 1. Cargar y preparar los datos
    print("1. Cargando y preparando los datos para el meta-modelo...")
    try:
        df = pd.read_excel(RUTA_DATOS)
    except FileNotFoundError:
        print(f"ERROR: No se encontró el archivo en la ruta '{RUTA_DATOS}'.")
        return

    features = ['Escenario', 'Distribución', 'Varianza error', 'Paso', 'Tipo de Modelo']
    df_meta = df[features + MODELOS].copy()
    df_meta['Mejor_Modelo'] = df_meta[MODELOS].idxmin(axis=1)
    df_meta.dropna(subset=features, inplace=True)

    X = df_meta[features]
    y = df_meta['Mejor_Modelo']
    
    # 2. Preprocesamiento de características
    print("2. Realizando preprocesamiento (One-Hot Encoding)...")
    categorical_features = ['Escenario', 'Distribución', 'Tipo de Modelo']
    encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    X_encoded_cats = encoder.fit_transform(X[categorical_features])
    
    # Obtener los nombres de las nuevas columnas codificadas
    encoded_feature_names = encoder.get_feature_names_out(categorical_features)
    
    # Combinar características numéricas y codificadas
    X_numeric = X.drop(columns=categorical_features)
    X_processed = np.hstack((X_numeric.values, X_encoded_cats))
    
    # Nombres de todas las características finales
    final_feature_names = list(X_numeric.columns) + list(encoded_feature_names)

    # 3. Dividir en conjuntos de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(
        X_processed, y, test_size=0.3, random_state=42, stratify=y
    )
    
    # 4. Definir y entrenar los modelos
    print("\n3. Entrenando y evaluando los modelos recomendadores...")
    models_to_train = {
        "Árbol de Decisión": DecisionTreeClassifier(max_depth=5, min_samples_leaf=20, random_state=42),
        "Gradient Boosting": GradientBoostingClassifier(n_estimators=100, max_depth=4, learning_rate=0.1, random_state=42)
    }
    
    class_labels = sorted(y.unique())

    for name, model in models_to_train.items():
        print(f"\n--- Analizando: {name} ---")
        
        # Entrenar
        model.fit(X_train, y_train)
        
        # Predecir
        y_pred = model.predict(X_test)
        
        # Evaluar y mostrar reporte
        accuracy = accuracy_score(y_test, y_pred)
        print(f"Precisión: {accuracy:.2%}")
        print("Reporte de Clasificación:")
        print(classification_report(y_test, y_pred, labels=class_labels))
        
        # Generar visualizaciones
        print("Generando visualizaciones...")
        plot_feature_importance(model, final_feature_names, name)
        plot_confusion_matrix(y_test, y_pred, name, class_labels)
        
        # Visualizar el árbol de decisión si corresponde
        if name == "Árbol de Decisión":
            fig, ax = plt.subplots(figsize=(25, 15))
            plot_tree(model, feature_names=final_feature_names, class_names=class_labels, 
                      filled=True, rounded=True, fontsize=10, ax=ax)
            ax.set_title("Árbol de Decisión para Recomendación de Modelos", fontsize=20)
            guardar_grafico("decision_tree_visualization.png")

    print("\n✓ Análisis del meta-modelo completado.")

if __name__ == "__main__":
    main()

Directorio 'resultados_meta_modelo' creado.
1. Cargando y preparando los datos para el meta-modelo...
2. Realizando preprocesamiento (One-Hot Encoding)...

3. Entrenando y evaluando los modelos recomendadores...

--- Analizando: Árbol de Decisión ---
Precisión: 45.83%
Reporte de Clasificación:
                     precision    recall  f1-score   support

              AREPD       0.00      0.00      0.00         9
            AV-MCPS       0.00      0.00      0.00        22
Block Bootstrapping       0.50      0.88      0.64       286
             DeepAR       0.00      0.00      0.00        31
         EnCQR-LSTM       0.18      0.29      0.22        35
               LSPM       0.00      0.00      0.00        42
              LSPMW       0.00      0.00      0.00        16
               MCPS       0.00      0.00      0.00        20
    Sieve Bootstrap       0.29      0.09      0.13       139

           accuracy                           0.46       600
          macro avg       0.11  

c:\Users\pleal\Documents\Unal\Tesis\Codigo\Prediccion_Probabilistica\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pleal\Documents\Unal\Tesis\Codigo\Prediccion_Probabilistica\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\pleal\Documents\Unal\Tesis\Codigo\Prediccion_Probabilistica\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division`

 -> Gráfico guardado en: resultados_meta_modelo\feature_importance_árbol_de_decisión.png
 -> Gráfico guardado en: resultados_meta_modelo\confusion_matrix_árbol_de_decisión.png
 -> Gráfico guardado en: resultados_meta_modelo\decision_tree_visualization.png

--- Analizando: Gradient Boosting ---
Precisión: 43.17%
Reporte de Clasificación:
                     precision    recall  f1-score   support

              AREPD       0.00      0.00      0.00         9
            AV-MCPS       0.00      0.00      0.00        22
Block Bootstrapping       0.58      0.71      0.64       286
             DeepAR       0.21      0.26      0.23        31
         EnCQR-LSTM       0.21      0.20      0.20        35
               LSPM       0.18      0.10      0.12        42
              LSPMW       0.12      0.06      0.08        16
               MCPS       0.00      0.00      0.00        20
    Sieve Bootstrap       0.30      0.26      0.28       139

           accuracy                           0.4

# Analisis Escalonado

## Analisis Especifico

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from scipy import stats
from itertools import combinations
import warnings
warnings.filterwarnings('ignore')

# Configuración de estilo
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Ruta de datos
RUTA_DATOS = "./Datos/datos_combinados.xlsx"

# Modelos a analizar
MODELOS = ['AREPD', 'AV-MCPS', 'Block Bootstrapping', 'DeepAR', 
           'EnCQR-LSTM', 'LSPM', 'LSPMW', 'MCPS', 'Sieve Bootstrap']

# Escenarios
ESCENARIOS = ['Estacionario_Lineal', 'No_Lineal_Estacionario', 'No_Estacionario_Lineal']

# Características de simulación
CARACTERISTICAS = ['Paso', 'Tipo de Modelo', 'Distribución', 'Varianza error']


def diebold_mariano_test(errores1, errores2, h=1, alternative='two-sided'):
    """
    Test de Diebold-Mariano para comparar la precisión de dos pronósticos
    
    Args:
        errores1: Errores del modelo 1 (valores menores = mejor)
        errores2: Errores del modelo 2 (valores menores = mejor)
        h: Horizonte de predicción
        alternative: 'two-sided', 'less' (modelo1 < modelo2), 'greater' (modelo1 > modelo2)
    
    Returns:
        dict con estadístico DM, p-valor y conclusión
    """
    # Asegurar que son arrays numpy
    e1 = np.asarray(errores1)
    e2 = np.asarray(errores2)
    
    # Verificar misma longitud
    if len(e1) != len(e2):
        raise ValueError("Los vectores de errores deben tener la misma longitud")
    
    n = len(e1)
    
    # Calcular diferencias de pérdidas (loss differential)
    d = e1 - e2  # Si d < 0, modelo 1 es mejor
    
    # Media de diferencias
    d_mean = np.mean(d)
    
    # Calcular varianza con corrección de autocorrelación (Harvey, Leybourne y Newbold, 1997)
    gamma_0 = np.var(d, ddof=1)
    
    # Autocovarianzas hasta lag h
    gamma_sum = 0
    for k in range(1, h):
        if k < n:
            gamma_k = np.mean((d[:-k] - d_mean) * (d[k:] - d_mean))
            gamma_sum += 2 * gamma_k
    
    # Varianza de largo plazo
    var_d = (gamma_0 + gamma_sum) / n
    
    # Corrección de Harvey-Leybourne-Newbold para muestras pequeñas
    hlnc = np.sqrt((n + 1 - 2*h + h*(h-1)/n) / n)
    
    # Estadístico DM
    if var_d > 0:
        dm_stat = d_mean / np.sqrt(var_d)
        dm_stat_corrected = dm_stat * hlnc
    else:
        dm_stat = 0
        dm_stat_corrected = 0
    
    # P-valor usando distribución t con n-1 grados de libertad
    if alternative == 'two-sided':
        p_value = 2 * (1 - stats.t.cdf(abs(dm_stat_corrected), df=n-1))
    elif alternative == 'less':
        p_value = stats.t.cdf(dm_stat_corrected, df=n-1)
    elif alternative == 'greater':
        p_value = 1 - stats.t.cdf(dm_stat_corrected, df=n-1)
    else:
        raise ValueError("alternative debe ser 'two-sided', 'less' o 'greater'")
    
    return {
        'dm_statistic': dm_stat,
        'dm_statistic_corrected': dm_stat_corrected,
        'p_value': p_value,
        'mean_diff': d_mean,
        'modelo1_mejor': d_mean < 0,
        'n': n
    }


def comparaciones_multiples_dm(df, modelos, alpha=0.05):
    """
    Realiza comparaciones múltiples usando test Diebold-Mariano 
    con corrección de Bonferroni
    """
    n_comparaciones = len(list(combinations(modelos, 2)))
    alpha_bonferroni = alpha / n_comparaciones
    
    print(f"\n🔬 TEST DE DIEBOLD-MARIANO CON CORRECCIÓN DE BONFERRONI")
    print(f"   Número de comparaciones: {n_comparaciones}")
    print(f"   Alpha original: {alpha}")
    print(f"   Alpha corregido (Bonferroni): {alpha_bonferroni:.6f}")
    print("-" * 70)
    
    resultados = []
    
    for modelo1, modelo2 in combinations(modelos, 2):
        try:
            dm_result = diebold_mariano_test(
                df[modelo1].values, 
                df[modelo2].values,
                h=1,
                alternative='two-sided'
            )
            
            significativo = dm_result['p_value'] < alpha_bonferroni
            
            if significativo:
                if dm_result['mean_diff'] < 0:
                    ganador = modelo1
                    interpretacion = f"{modelo1} significativamente mejor"
                else:
                    ganador = modelo2
                    interpretacion = f"{modelo2} significativamente mejor"
            else:
                ganador = "No hay diferencia"
                interpretacion = "Sin diferencia significativa"
            
            resultados.append({
                'Modelo_1': modelo1,
                'Modelo_2': modelo2,
                'DM_Statistic': dm_result['dm_statistic_corrected'],
                'p_value': dm_result['p_value'],
                'p_value_bonferroni': alpha_bonferroni,
                'Significativo': significativo,
                'Ganador': ganador,
                'Diff_Media': dm_result['mean_diff'],
                'Interpretacion': interpretacion
            })
            
        except Exception as e:
            print(f"⚠️  Error comparando {modelo1} vs {modelo2}: {str(e)}")
            continue
    
    df_resultados = pd.DataFrame(resultados)
    return df_resultados, alpha_bonferroni


def crear_matriz_superioridad(df_comparaciones, modelos):
    """Crea una matriz mostrando qué modelo es superior a cuál"""
    n = len(modelos)
    matriz = pd.DataFrame(np.zeros((n, n)), index=modelos, columns=modelos)
    
    for _, row in df_comparaciones.iterrows():
        m1, m2 = row['Modelo_1'], row['Modelo_2']
        
        if row['Significativo']:
            if row['Ganador'] == m1:
                matriz.loc[m1, m2] = 1
                matriz.loc[m2, m1] = -1
            elif row['Ganador'] == m2:
                matriz.loc[m2, m1] = 1
                matriz.loc[m1, m2] = -1
    
    return matriz


def calcular_ranking_dm(df_comparaciones, modelos):
    """Calcula ranking basado en resultados de Diebold-Mariano"""
    matriz_sup = crear_matriz_superioridad(df_comparaciones, modelos)
    
    ranking_data = []
    
    for modelo in modelos:
        victorias = (matriz_sup.loc[modelo] == 1).sum()
        derrotas = (matriz_sup.loc[modelo] == -1).sum()
        empates = (matriz_sup.loc[modelo] == 0).sum() - 1
        
        score = victorias - derrotas
        
        total_comparaciones = victorias + derrotas + empates
        pct_victorias = (victorias / total_comparaciones * 100) if total_comparaciones > 0 else 0
        
        ranking_data.append({
            'Modelo': modelo,
            'Victorias': int(victorias),
            'Derrotas': int(derrotas),
            'Empates': int(empates),
            'Score': int(score),
            'Pct_Victorias': round(pct_victorias, 2)
        })
    
    df_ranking = pd.DataFrame(ranking_data)
    df_ranking = df_ranking.sort_values('Score', ascending=False).reset_index(drop=True)
    df_ranking['Rank'] = range(1, len(df_ranking) + 1)
    
    return df_ranking, matriz_sup


class AnalizadorModelos:
    """Clase para analizar el desempeño de modelos de predicción"""
    
    def __init__(self, ruta_datos):
        """Inicializa el analizador cargando los datos"""
        self.df = pd.read_excel(ruta_datos)
        self.resultados_analisis = {}
        print(f"✓ Datos cargados: {self.df.shape[0]} filas, {self.df.shape[1]} columnas")
        print(f"\nEscenarios encontrados: {self.df['Escenario'].unique()}")
        print(f"Modelos a analizar: {len(MODELOS)}")
        
    def analizar_modelo_escenario(self, escenario, modelo):
        """
        Realiza análisis completo de un modelo en un escenario específico
        con gráficas individuales
        """
        print(f"\n{'='*80}")
        print(f"ANÁLISIS: {modelo} en escenario {escenario}")
        print(f"{'='*80}\n")
        
        # Filtrar datos
        df_filtrado = self.df[self.df['Escenario'] == escenario].copy()
        
        if df_filtrado.empty:
            print(f"⚠ No hay datos para el escenario {escenario}")
            return
        
        # Crear directorio para guardar resultados
        dir_salida = Path(f"./Resultados/{escenario}/{modelo}")
        dir_salida.mkdir(parents=True, exist_ok=True)
        
        resultados = {
            'escenario': escenario,
            'modelo': modelo,
            'n_observaciones': len(df_filtrado)
        }
        
        # ===== GRÁFICAS INDIVIDUALES =====
        
        # 1. Barras de rendimiento por distribución
        self._grafica_rendimiento_distribucion(df_filtrado, modelo, dir_salida)
        
        # 2. Barras de rendimiento por Tipo de Modelo (Generador)
        self._grafica_rendimiento_tipo_generador(df_filtrado, modelo, dir_salida)
        
        # 3. Rendimiento por Paso
        self._grafica_rendimiento_paso(df_filtrado, modelo, dir_salida)
        
        # 4. Rendimiento por Varianza
        self._grafica_rendimiento_varianza(df_filtrado, modelo, dir_salida)
        
        # 5-10. Interacciones (6 gráficas)
        self._grafica_interaccion_dist_tipo(df_filtrado, modelo, dir_salida)
        self._grafica_interaccion_dist_paso(df_filtrado, modelo, dir_salida)
        self._grafica_interaccion_dist_varianza(df_filtrado, modelo, dir_salida)
        self._grafica_interaccion_tipo_paso(df_filtrado, modelo, dir_salida)
        self._grafica_interaccion_tipo_varianza(df_filtrado, modelo, dir_salida)
        self._grafica_interaccion_varianza_paso(df_filtrado, modelo, dir_salida)
        
        # 11. Características que más afectan el rendimiento
        self._grafica_importancia_caracteristicas(df_filtrado, modelo, dir_salida, resultados)
        
        # 12. Distribución general del rendimiento
        self._grafica_distribucion_general(df_filtrado, modelo, dir_salida)
        
        # 13. Análisis de outliers
        self._grafica_analisis_outliers(df_filtrado, modelo, dir_salida)
        
        # 14. Mapa de calor de configuraciones
        self._grafica_heatmap_configuraciones(df_filtrado, modelo, dir_salida)
        
        # 15. Análisis de estabilidad
        self._grafica_estabilidad(df_filtrado, modelo, dir_salida)
        
        # Guardar estadísticas y resumen
        self._guardar_estadisticas_completas(df_filtrado, modelo, dir_salida, resultados)
        
        return resultados
    
    def _grafica_rendimiento_distribucion(self, df, modelo, dir_salida):
        """Gráfica 1: Barras de rendimiento por distribución"""
        print("📊 Generando: Rendimiento por Distribución...")
        
        fig, ax = plt.subplots(figsize=(12, 7))
        
        # Calcular estadísticas
        stats_dist = df.groupby('Distribución')[modelo].agg(['mean', 'std', 'count'])
        stats_dist = stats_dist.sort_values('mean')
        
        # Crear barras con error
        x_pos = np.arange(len(stats_dist))
        colors = plt.cm.RdYlGn_r(np.linspace(0.3, 0.9, len(stats_dist)))
        
        bars = ax.bar(x_pos, stats_dist['mean'], yerr=stats_dist['std'], 
                     capsize=8, alpha=0.8, color=colors, edgecolor='black', linewidth=1.5)
        
        # Etiquetas y título
        ax.set_xlabel('Distribución', fontsize=13, fontweight='bold')
        ax.set_ylabel('Rendimiento Promedio', fontsize=13, fontweight='bold')
        ax.set_title(f'{modelo} - Rendimiento por Distribución\n(Barras de error: ±1 std)', 
                    fontsize=15, fontweight='bold', pad=20)
        ax.set_xticks(x_pos)
        ax.set_xticklabels(stats_dist.index, rotation=45, ha='right', fontsize=11)
        
        # Añadir valores sobre las barras
        for i, (bar, mean, count) in enumerate(zip(bars, stats_dist['mean'], stats_dist['count'])):
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height,
                   f'{mean:.4f}\n(n={int(count)})',
                   ha='center', va='bottom', fontsize=9, fontweight='bold')
        
        ax.grid(True, alpha=0.3, axis='y', linestyle='--')
        plt.tight_layout()
        plt.savefig(dir_salida / '01_rendimiento_distribucion.png', dpi=300, bbox_inches='tight')
        plt.close()
        print("✓ Completado\n")
    
    def _grafica_rendimiento_tipo_generador(self, df, modelo, dir_salida):
        """Gráfica 2: Barras de rendimiento por Tipo de Generador"""
        print("📊 Generando: Rendimiento por Tipo de Generador...")
        
        fig, ax = plt.subplots(figsize=(14, 7))
        
        # Calcular estadísticas
        stats_tipo = df.groupby('Tipo de Modelo')[modelo].agg(['mean', 'std', 'count', 'median'])
        stats_tipo = stats_tipo.sort_values('mean')
        
        # Crear barras con error
        x_pos = np.arange(len(stats_tipo))
        colors = plt.cm.viridis(np.linspace(0.2, 0.9, len(stats_tipo)))
        
        bars = ax.bar(x_pos, stats_tipo['mean'], yerr=stats_tipo['std'], 
                     capsize=8, alpha=0.8, color=colors, edgecolor='black', linewidth=1.5)
        
        # Añadir línea de mediana
        ax.plot(x_pos, stats_tipo['median'], 'ro-', linewidth=2, markersize=8, 
               label='Mediana', zorder=5)
        
        # Etiquetas y título
        ax.set_xlabel('Tipo de Generador', fontsize=13, fontweight='bold')
        ax.set_ylabel('Rendimiento', fontsize=13, fontweight='bold')
        ax.set_title(f'{modelo} - Rendimiento por Tipo de Generador\n(Media ± std, Línea roja: Mediana)', 
                    fontsize=15, fontweight='bold', pad=20)
        ax.set_xticks(x_pos)
        ax.set_xticklabels(stats_tipo.index, rotation=45, ha='right', fontsize=10)
        
        # Añadir valores
        for i, (bar, mean, count) in enumerate(zip(bars, stats_tipo['mean'], stats_tipo['count'])):
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height,
                   f'{mean:.4f}\n(n={int(count)})',
                   ha='center', va='bottom', fontsize=8, fontweight='bold')
        
        ax.legend(fontsize=11)
        ax.grid(True, alpha=0.3, axis='y', linestyle='--')
        plt.tight_layout()
        plt.savefig(dir_salida / '02_rendimiento_tipo_generador.png', dpi=300, bbox_inches='tight')
        plt.close()
        print("✓ Completado\n")
    
    def _grafica_rendimiento_paso(self, df, modelo, dir_salida):
        """Gráfica 3: Rendimiento por Paso"""
        print("📊 Generando: Rendimiento por Paso...")
        
        fig, ax = plt.subplots(figsize=(12, 7))
        
        # Calcular estadísticas por paso
        paso_stats = df.groupby('Paso')[modelo].agg(['mean', 'std', 'min', 'max', 'median'])
        
        # Gráfica de línea con banda de confianza
        x = paso_stats.index
        ax.plot(x, paso_stats['mean'], 'o-', linewidth=3, markersize=10, 
               label='Media', color='darkblue')
        ax.fill_between(x, 
                        paso_stats['mean'] - paso_stats['std'],
                        paso_stats['mean'] + paso_stats['std'],
                        alpha=0.3, label='±1 std', color='lightblue')
        
        # Añadir min y max
        ax.plot(x, paso_stats['min'], 's--', linewidth=2, markersize=7, 
               label='Mínimo', color='green', alpha=0.7)
        ax.plot(x, paso_stats['max'], '^--', linewidth=2, markersize=7, 
               label='Máximo', color='red', alpha=0.7)
        
        # Etiquetas y título
        ax.set_xlabel('Paso', fontsize=13, fontweight='bold')
        ax.set_ylabel('Rendimiento', fontsize=13, fontweight='bold')
        ax.set_title(f'{modelo} - Evolución del Rendimiento por Paso', 
                    fontsize=15, fontweight='bold', pad=20)
        
        # Añadir valores sobre los puntos
        for i, (paso, mean) in enumerate(zip(x, paso_stats['mean'])):
            ax.annotate(f'{mean:.4f}', 
                       xy=(paso, mean), 
                       xytext=(0, 10), 
                       textcoords='offset points',
                       ha='center', 
                       fontsize=9, 
                       fontweight='bold',
                       bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.7))
        
        ax.legend(fontsize=11, loc='best')
        ax.grid(True, alpha=0.3, linestyle='--')
        plt.tight_layout()
        plt.savefig(dir_salida / '03_rendimiento_paso.png', dpi=300, bbox_inches='tight')
        plt.close()
        print("✓ Completado\n")
    
    def _grafica_rendimiento_varianza(self, df, modelo, dir_salida):
        """Gráfica 4: Rendimiento por Varianza"""
        print("📊 Generando: Rendimiento por Varianza...")
        
        fig, ax = plt.subplots(figsize=(12, 7))
        
        # Calcular estadísticas por varianza
        var_stats = df.groupby('Varianza error')[modelo].agg(['mean', 'std', 'count'])
        
        # Crear scatter plot con todos los puntos
        for var in df['Varianza error'].unique():
            df_var = df[df['Varianza error'] == var]
            ax.scatter([var] * len(df_var), df_var[modelo], 
                      alpha=0.4, s=50, label=f'Var={var}')
        
        # Línea de tendencia
        x = var_stats.index
        ax.plot(x, var_stats['mean'], 'ro-', linewidth=3, markersize=12, 
               label='Media', zorder=10)
        
        # Banda de confianza
        ax.fill_between(x, 
                        var_stats['mean'] - var_stats['std'],
                        var_stats['mean'] + var_stats['std'],
                        alpha=0.2, color='red', label='±1 std')
        
        # Etiquetas y título
        ax.set_xlabel('Varianza del Error', fontsize=13, fontweight='bold')
        ax.set_ylabel('Rendimiento', fontsize=13, fontweight='bold')
        ax.set_title(f'{modelo} - Rendimiento vs Varianza del Error', 
                    fontsize=15, fontweight='bold', pad=20)
        
        # Añadir valores y conteos
        for var, mean, count in zip(x, var_stats['mean'], var_stats['count']):
            ax.annotate(f'{mean:.4f}\n(n={int(count)})', 
                       xy=(var, mean), 
                       xytext=(15, 15), 
                       textcoords='offset points',
                       ha='left', 
                       fontsize=9, 
                       fontweight='bold',
                       bbox=dict(boxstyle='round,pad=0.4', facecolor='yellow', alpha=0.7),
                       arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'))
        
        # Calcular correlación
        corr = df[['Varianza error', modelo]].corr().iloc[0, 1]
        ax.text(0.02, 0.98, f'Correlación: {corr:.4f}', 
               transform=ax.transAxes, fontsize=12, fontweight='bold',
               verticalalignment='top',
               bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
        
        ax.legend(fontsize=10, loc='best', ncol=2)
        ax.grid(True, alpha=0.3, linestyle='--')
        plt.tight_layout()
        plt.savefig(dir_salida / '04_rendimiento_varianza.png', dpi=300, bbox_inches='tight')
        plt.close()
        print("✓ Completado\n")
    
    def _grafica_interaccion_dist_tipo(self, df, modelo, dir_salida):
        """Gráfica 5: Interacción Distribución vs Tipo de Generador"""
        print("📊 Generando: Interacción Distribución vs Tipo de Generador...")
        
        fig, ax = plt.subplots(figsize=(14, 8))
        
        # Crear gráfica de interacción
        for dist in df['Distribución'].unique():
            df_dist = df[df['Distribución'] == dist]
            tipo_mean = df_dist.groupby('Tipo de Modelo')[modelo].mean().sort_index()
            ax.plot(range(len(tipo_mean)), tipo_mean.values, 
                   marker='o', linewidth=2.5, markersize=8, label=dist)
        
        # Etiquetas
        tipos = sorted(df['Tipo de Modelo'].unique())
        ax.set_xticks(range(len(tipos)))
        ax.set_xticklabels(tipos, rotation=45, ha='right', fontsize=10)
        ax.set_xlabel('Tipo de Generador', fontsize=13, fontweight='bold')
        ax.set_ylabel('Rendimiento Promedio', fontsize=13, fontweight='bold')
        ax.set_title(f'{modelo} - Interacción: Distribución × Tipo de Generador', 
                    fontsize=15, fontweight='bold', pad=20)
        
        ax.legend(title='Distribución', fontsize=11, title_fontsize=12, loc='best')
        ax.grid(True, alpha=0.3, linestyle='--')
        plt.tight_layout()
        plt.savefig(dir_salida / '05_interaccion_dist_tipo.png', dpi=300, bbox_inches='tight')
        plt.close()
        print("✓ Completado\n")
    
    def _grafica_interaccion_dist_paso(self, df, modelo, dir_salida):
        """Gráfica 6: Interacción Distribución vs Paso"""
        print("📊 Generando: Interacción Distribución vs Paso...")
        
        fig, ax = plt.subplots(figsize=(12, 7))
        
        for dist in df['Distribución'].unique():
            df_dist = df[df['Distribución'] == dist]
            paso_mean = df_dist.groupby('Paso')[modelo].mean()
            ax.plot(paso_mean.index, paso_mean.values, 
                   marker='s', linewidth=2.5, markersize=9, label=dist)
        
        ax.set_xlabel('Paso', fontsize=13, fontweight='bold')
        ax.set_ylabel('Rendimiento Promedio', fontsize=13, fontweight='bold')
        ax.set_title(f'{modelo} - Interacción: Distribución × Paso', 
                    fontsize=15, fontweight='bold', pad=20)
        
        ax.legend(title='Distribución', fontsize=11, title_fontsize=12, loc='best')
        ax.grid(True, alpha=0.3, linestyle='--')
        plt.tight_layout()
        plt.savefig(dir_salida / '06_interaccion_dist_paso.png', dpi=300, bbox_inches='tight')
        plt.close()
        print("✓ Completado\n")
    
    def _grafica_interaccion_dist_varianza(self, df, modelo, dir_salida):
        """Gráfica 7: Interacción Distribución vs Varianza"""
        print("📊 Generando: Interacción Distribución vs Varianza...")
        
        fig, ax = plt.subplots(figsize=(12, 7))
        
        for dist in df['Distribución'].unique():
            df_dist = df[df['Distribución'] == dist]
            var_mean = df_dist.groupby('Varianza error')[modelo].mean()
            ax.plot(var_mean.index, var_mean.values, 
                   marker='^', linewidth=2.5, markersize=9, label=dist)
        
        ax.set_xlabel('Varianza del Error', fontsize=13, fontweight='bold')
        ax.set_ylabel('Rendimiento Promedio', fontsize=13, fontweight='bold')
        ax.set_title(f'{modelo} - Interacción: Distribución × Varianza', 
                    fontsize=15, fontweight='bold', pad=20)
        
        ax.legend(title='Distribución', fontsize=11, title_fontsize=12, loc='best')
        ax.grid(True, alpha=0.3, linestyle='--')
        plt.tight_layout()
        plt.savefig(dir_salida / '07_interaccion_dist_varianza.png', dpi=300, bbox_inches='tight')
        plt.close()
        print("✓ Completado\n")
    
    def _grafica_interaccion_tipo_paso(self, df, modelo, dir_salida):
        """Gráfica 8: Interacción Tipo de Generador vs Paso"""
        print("📊 Generando: Interacción Tipo de Generador vs Paso...")
        
        fig, ax = plt.subplots(figsize=(12, 7))
        
        tipos = df['Tipo de Modelo'].unique()
        colors = plt.cm.tab10(np.linspace(0, 1, len(tipos)))
        
        for tipo, color in zip(tipos, colors):
            df_tipo = df[df['Tipo de Modelo'] == tipo]
            paso_mean = df_tipo.groupby('Paso')[modelo].mean()
            ax.plot(paso_mean.index, paso_mean.values, 
                   marker='o', linewidth=2, markersize=8, label=tipo, color=color)
        
        ax.set_xlabel('Paso', fontsize=13, fontweight='bold')
        ax.set_ylabel('Rendimiento Promedio', fontsize=13, fontweight='bold')
        ax.set_title(f'{modelo} - Interacción: Tipo de Generador × Paso', 
                    fontsize=15, fontweight='bold', pad=20)
        
        ax.legend(title='Tipo de Generador', fontsize=9, title_fontsize=11, 
                 loc='best', ncol=2)
        ax.grid(True, alpha=0.3, linestyle='--')
        plt.tight_layout()
        plt.savefig(dir_salida / '08_interaccion_tipo_paso.png', dpi=300, bbox_inches='tight')
        plt.close()
        print("✓ Completado\n")
    
    def _grafica_interaccion_tipo_varianza(self, df, modelo, dir_salida):
        """Gráfica 9: Interacción Tipo de Generador vs Varianza"""
        print("📊 Generando: Interacción Tipo de Generador vs Varianza...")
        
        fig, ax = plt.subplots(figsize=(12, 7))
        
        tipos = df['Tipo de Modelo'].unique()
        colors = plt.cm.Set2(np.linspace(0, 1, len(tipos)))
        
        for tipo, color in zip(tipos, colors):
            df_tipo = df[df['Tipo de Modelo'] == tipo]
            var_mean = df_tipo.groupby('Varianza error')[modelo].mean()
            ax.plot(var_mean.index, var_mean.values, 
                   marker='D', linewidth=2, markersize=8, label=tipo, color=color)
        
        ax.set_xlabel('Varianza del Error', fontsize=13, fontweight='bold')
        ax.set_ylabel('Rendimiento Promedio', fontsize=13, fontweight='bold')
        ax.set_title(f'{modelo} - Interacción: Tipo de Generador × Varianza', 
                    fontsize=15, fontweight='bold', pad=20)
        
        ax.legend(title='Tipo de Generador', fontsize=9, title_fontsize=11, 
                 loc='best', ncol=2)
        ax.grid(True, alpha=0.3, linestyle='--')
        plt.tight_layout()
        plt.savefig(dir_salida / '09_interaccion_tipo_varianza.png', dpi=300, bbox_inches='tight')
        plt.close()
        print("✓ Completado\n")
    
    def _grafica_interaccion_varianza_paso(self, df, modelo, dir_salida):
        """Gráfica 10: Interacción Varianza vs Paso"""
        print("📊 Generando: Interacción Varianza vs Paso...")
        
        fig, ax = plt.subplots(figsize=(12, 7))
        
        varianzas = sorted(df['Varianza error'].unique())
        colors = plt.cm.plasma(np.linspace(0, 0.9, len(varianzas)))
        
        for var, color in zip(varianzas, colors):
            df_var = df[df['Varianza error'] == var]
            paso_mean = df_var.groupby('Paso')[modelo].mean()
            ax.plot(paso_mean.index, paso_mean.values, 
                   marker='o', linewidth=2.5, markersize=9, 
                   label=f'Varianza={var}', color=color)
        
        ax.set_xlabel('Paso', fontsize=13, fontweight='bold')
        ax.set_ylabel('Rendimiento Promedio', fontsize=13, fontweight='bold')
        ax.set_title(f'{modelo} - Interacción: Varianza × Paso', 
                    fontsize=15, fontweight='bold', pad=20)
        
        ax.legend(title='Varianza del Error', fontsize=11, title_fontsize=12, loc='best')
        ax.grid(True, alpha=0.3, linestyle='--')
        plt.tight_layout()
        plt.savefig(dir_salida / '10_interaccion_varianza_paso.png', dpi=300, bbox_inches='tight')
        plt.close()
        print("✓ Completado\n")
    
    def _grafica_importancia_caracteristicas(self, df, modelo, dir_salida, resultados):
        """Gráfica 11: Características que más afectan el rendimiento"""
        print("📊 Generando: Importancia de Características...")
        
        fig, ax = plt.subplots(figsize=(12, 8))
        
        # Calcular importancia basada en variación del rendimiento
        importancia = {}
        
        # 1. Por Distribución
        dist_var = df.groupby('Distribución')[modelo].var().mean()
        dist_rango = df.groupby('Distribución')[modelo].mean().max() - df.groupby('Distribución')[modelo].mean().min()
        importancia['Distribución'] = dist_rango
        
        # 2. Por Tipo de Modelo
        tipo_var = df.groupby('Tipo de Modelo')[modelo].var().mean()
        tipo_rango = df.groupby('Tipo de Modelo')[modelo].mean().max() - df.groupby('Tipo de Modelo')[modelo].mean().min()
        importancia['Tipo de Generador'] = tipo_rango
        
        # 3. Por Paso
        paso_var = df.groupby('Paso')[modelo].var().mean()
        paso_rango = df.groupby('Paso')[modelo].mean().max() - df.groupby('Paso')[modelo].mean().min()
        importancia['Paso'] = paso_rango
        
        # 4. Por Varianza
        var_var = df.groupby('Varianza error')[modelo].var().mean()
        var_rango = df.groupby('Varianza error')[modelo].mean().max() - df.groupby('Varianza error')[modelo].mean().min()
        importancia['Varianza Error'] = var_rango
        
        # Calcular también correlaciones absolutas
        correlaciones = {}
        correlaciones['Paso'] = abs(df['Paso'].corr(df[modelo]))
        correlaciones['Varianza Error'] = abs(df['Varianza error'].corr(df[modelo]))
        
        # Convertir variables categóricas a numéricas para correlación
        df_temp = df.copy()
        df_temp['Dist_num'] = pd.Categorical(df_temp['Distribución']).codes
        df_temp['Tipo_num'] = pd.Categorical(df_temp['Tipo de Modelo']).codes
        correlaciones['Distribución'] = abs(df_temp['Dist_num'].corr(df_temp[modelo]))
        correlaciones['Tipo de Generador'] = abs(df_temp['Tipo_num'].corr(df_temp[modelo]))
        
        # Normalizar importancias (0-100)
        max_importancia = max(importancia.values())
        importancia_norm = {k: (v/max_importancia)*100 for k, v in importancia.items()}
        
        # Ordenar por importancia
        items_ordenados = sorted(importancia_norm.items(), key=lambda x: x[1], reverse=True)
        caracteristicas = [item[0] for item in items_ordenados]
        valores = [item[1] for item in items_ordenados]
        
        # Crear barras
        colors = plt.cm.RdYlGn_r(np.linspace(0.3, 0.9, len(caracteristicas)))
        bars = ax.barh(caracteristicas, valores, color=colors, edgecolor='black', linewidth=2)
        
        # Añadir valores y correlaciones
        for i, (bar, car, val) in enumerate(zip(bars, caracteristicas, valores)):
            width = bar.get_width()
            corr = correlaciones.get(car, 0)
            ax.text(width, i, f'  {val:.1f}%\n  (corr={corr:.3f})', 
                   va='center', fontsize=10, fontweight='bold')
        
        ax.set_xlabel('Importancia Relativa (%)', fontsize=13, fontweight='bold')
        ax.set_title(f'{modelo} - Características que más Afectan el Rendimiento\n' + 
                    '(Basado en rango de variación de medias)',
                    fontsize=15, fontweight='bold', pad=20)
        ax.set_xlim(0, 110)
        ax.grid(True, alpha=0.3, axis='x', linestyle='--')
        
        plt.tight_layout()
        plt.savefig(dir_salida / '11_importancia_caracteristicas.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        # Guardar en resultados
        resultados['importancia_caracteristicas'] = dict(items_ordenados)
        resultados['correlaciones'] = correlaciones
        
        print("✓ Completado\n")
    
    def _grafica_distribucion_general(self, df, modelo, dir_salida):
        """Gráfica 12: Distribución general del rendimiento"""
        print("📊 Generando: Distribución General del Rendimiento...")
        
        fig, ax = plt.subplots(figsize=(12, 7))
        
        # Histograma con curva de densidad
        n, bins, patches = ax.hist(df[modelo], bins=40, density=True, 
                                   alpha=0.7, color='skyblue', edgecolor='black')
        
        # Añadir KDE
        from scipy.stats import gaussian_kde
        kde = gaussian_kde(df[modelo])
        x_range = np.linspace(df[modelo].min(), df[modelo].max(), 200)
        ax.plot(x_range, kde(x_range), 'r-', linewidth=3, label='KDE')
        
        # Líneas de estadísticos
        media = df[modelo].mean()
        mediana = df[modelo].median()
        ax.axvline(media, color='darkblue', linestyle='--', linewidth=2.5, 
                  label=f'Media: {media:.4f}')
        ax.axvline(mediana, color='green', linestyle='--', linewidth=2.5, 
                  label=f'Mediana: {mediana:.4f}')
        
        # Cuartiles
        q1 = df[modelo].quantile(0.25)
        q3 = df[modelo].quantile(0.75)
        ax.axvline(q1, color='orange', linestyle=':', linewidth=2, 
                  label=f'Q1: {q1:.4f}')
        ax.axvline(q3, color='orange', linestyle=':', linewidth=2, 
                  label=f'Q3: {q3:.4f}')
        
        ax.set_xlabel('Rendimiento', fontsize=13, fontweight='bold')
        ax.set_ylabel('Densidad', fontsize=13, fontweight='bold')
        ax.set_title(f'{modelo} - Distribución General del Rendimiento\n' +
                    f'(n={len(df)}, std={df[modelo].std():.4f})',
                    fontsize=15, fontweight='bold', pad=20)
        
        ax.legend(fontsize=11, loc='best')
        ax.grid(True, alpha=0.3, linestyle='--')
        plt.tight_layout()
        plt.savefig(dir_salida / '12_distribucion_general.png', dpi=300, bbox_inches='tight')
        plt.close()
        print("✓ Completado\n")
    
    def _grafica_analisis_outliers(self, df, modelo, dir_salida):
        """Gráfica 13: Análisis de outliers"""
        print("📊 Generando: Análisis de Outliers...")
        
        fig, ax = plt.subplots(figsize=(12, 7))
        
        # Boxplot detallado
        bp = ax.boxplot([df[modelo]], labels=[modelo], vert=True, patch_artist=True,
                       widths=0.5, showmeans=True, meanline=True)
        
        # Colorear
        bp['boxes'][0].set_facecolor('lightblue')
        bp['boxes'][0].set_alpha(0.7)
        bp['medians'][0].set_color('red')
        bp['medians'][0].set_linewidth(2.5)
        bp['means'][0].set_color('darkblue')
        bp['means'][0].set_linewidth(2.5)
        
        # Identificar outliers usando IQR
        Q1 = df[modelo].quantile(0.25)
        Q3 = df[modelo].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        outliers = df[(df[modelo] < lower_bound) | (df[modelo] > upper_bound)]
        n_outliers = len(outliers)
        pct_outliers = (n_outliers / len(df)) * 100
        
        # Scatter de todos los puntos con outliers resaltados
        y_normal = df[(df[modelo] >= lower_bound) & (df[modelo] <= upper_bound)][modelo]
        x_normal = np.random.normal(1, 0.04, size=len(y_normal))
        ax.scatter(x_normal, y_normal, alpha=0.3, s=30, color='blue', label='Datos normales')
        
        if n_outliers > 0:
            y_outliers = outliers[modelo]
            x_outliers = np.random.normal(1, 0.04, size=len(y_outliers))
            ax.scatter(x_outliers, y_outliers, alpha=0.8, s=80, color='red', 
                      marker='*', label=f'Outliers ({n_outliers})', zorder=10)
        
        # Líneas de límites
        ax.axhline(lower_bound, color='orange', linestyle='--', linewidth=2, 
                  label=f'Límite inferior: {lower_bound:.4f}')
        ax.axhline(upper_bound, color='orange', linestyle='--', linewidth=2, 
                  label=f'Límite superior: {upper_bound:.4f}')
        
        ax.set_ylabel('Rendimiento', fontsize=13, fontweight='bold')
        ax.set_title(f'{modelo} - Análisis de Outliers (Método IQR)\n' +
                    f'Outliers detectados: {n_outliers} ({pct_outliers:.2f}%)',
                    fontsize=15, fontweight='bold', pad=20)
        ax.set_xticks([])
        ax.legend(fontsize=11, loc='best')
        ax.grid(True, alpha=0.3, axis='y', linestyle='--')
        plt.tight_layout()
        plt.savefig(dir_salida / '13_analisis_outliers.png', dpi=300, bbox_inches='tight')
        plt.close()
        print("✓ Completado\n")
    
    def _grafica_heatmap_configuraciones(self, df, modelo, dir_salida):
        """Gráfica 14: Mapa de calor de configuraciones"""
        print("📊 Generando: Heatmap de Configuraciones...")
        
        fig, axes = plt.subplots(1, 2, figsize=(18, 7))
        
        # Heatmap 1: Tipo de Modelo vs Distribución
        pivot1 = df.pivot_table(values=modelo, 
                                index='Tipo de Modelo', 
                                columns='Distribución', 
                                aggfunc='mean')
        
        sns.heatmap(pivot1, annot=True, fmt='.4f', cmap='RdYlGn_r', 
                   ax=axes[0], cbar_kws={'label': 'Rendimiento'},
                   linewidths=0.5, linecolor='gray')
        axes[0].set_title('Tipo de Generador × Distribución', 
                         fontsize=13, fontweight='bold', pad=15)
        axes[0].set_xlabel('Distribución', fontsize=11, fontweight='bold')
        axes[0].set_ylabel('Tipo de Generador', fontsize=11, fontweight='bold')
        
        # Heatmap 2: Paso vs Varianza
        pivot2 = df.pivot_table(values=modelo, 
                                index='Paso', 
                                columns='Varianza error', 
                                aggfunc='mean')
        
        sns.heatmap(pivot2, annot=True, fmt='.4f', cmap='RdYlGn_r', 
                   ax=axes[1], cbar_kws={'label': 'Rendimiento'},
                   linewidths=0.5, linecolor='gray')
        axes[1].set_title('Paso × Varianza del Error', 
                         fontsize=13, fontweight='bold', pad=15)
        axes[1].set_xlabel('Varianza del Error', fontsize=11, fontweight='bold')
        axes[1].set_ylabel('Paso', fontsize=11, fontweight='bold')
        
        fig.suptitle(f'{modelo} - Mapas de Calor de Configuraciones', 
                    fontsize=16, fontweight='bold', y=1.02)
        
        plt.tight_layout()
        plt.savefig(dir_salida / '14_heatmap_configuraciones.png', dpi=300, bbox_inches='tight')
        plt.close()
        print("✓ Completado\n")
    
    def _grafica_estabilidad(self, df, modelo, dir_salida):
        """Gráfica 15: Análisis de estabilidad"""
        print("📊 Generando: Análisis de Estabilidad...")
        
        fig, axes = plt.subplots(2, 2, figsize=(16, 12))
        
        # 1. Estabilidad por Paso
        paso_std = df.groupby('Paso')[modelo].std()
        paso_cv = (df.groupby('Paso')[modelo].std() / df.groupby('Paso')[modelo].mean()) * 100
        
        ax1 = axes[0, 0]
        ax1_twin = ax1.twinx()
        
        bars = ax1.bar(paso_std.index, paso_std.values, alpha=0.7, 
                      color='steelblue', label='Desv. Estándar')
        line = ax1_twin.plot(paso_cv.index, paso_cv.values, 'ro-', 
                            linewidth=2.5, markersize=8, label='Coef. Variación (%)')
        
        ax1.set_xlabel('Paso', fontsize=11, fontweight='bold')
        ax1.set_ylabel('Desviación Estándar', fontsize=11, fontweight='bold', color='steelblue')
        ax1_twin.set_ylabel('Coeficiente de Variación (%)', fontsize=11, fontweight='bold', color='red')
        ax1.set_title('Estabilidad por Paso', fontsize=12, fontweight='bold')
        ax1.tick_params(axis='y', labelcolor='steelblue')
        ax1_twin.tick_params(axis='y', labelcolor='red')
        ax1.grid(True, alpha=0.3)
        
        # 2. Estabilidad por Tipo
        tipo_std = df.groupby('Tipo de Modelo')[modelo].std().sort_values()
        
        ax2 = axes[0, 1]
        colors = plt.cm.RdYlGn(np.linspace(0.3, 0.9, len(tipo_std)))
        ax2.barh(range(len(tipo_std)), tipo_std.values, color=colors, 
                edgecolor='black', linewidth=1.5)
        ax2.set_yticks(range(len(tipo_std)))
        ax2.set_yticklabels(tipo_std.index, fontsize=9)
        ax2.set_xlabel('Desviación Estándar', fontsize=11, fontweight='bold')
        ax2.set_title('Estabilidad por Tipo de Generador\n(menor = más estable)', 
                     fontsize=12, fontweight='bold')
        ax2.grid(True, alpha=0.3, axis='x')
        
        # 3. Estabilidad por Distribución
        dist_std = df.groupby('Distribución')[modelo].std().sort_values()
        
        ax3 = axes[1, 0]
        colors = plt.cm.RdYlGn(np.linspace(0.3, 0.9, len(dist_std)))
        ax3.barh(range(len(dist_std)), dist_std.values, color=colors, 
                edgecolor='black', linewidth=1.5)
        ax3.set_yticks(range(len(dist_std)))
        ax3.set_yticklabels(dist_std.index, fontsize=10)
        ax3.set_xlabel('Desviación Estándar', fontsize=11, fontweight='bold')
        ax3.set_title('Estabilidad por Distribución\n(menor = más estable)', 
                     fontsize=12, fontweight='bold')
        ax3.grid(True, alpha=0.3, axis='x')
        
        # 4. Estabilidad por Varianza
        var_std = df.groupby('Varianza error')[modelo].std()
        var_mean = df.groupby('Varianza error')[modelo].mean()
        
        ax4 = axes[1, 1]
        ax4.plot(var_std.index, var_std.values, 'bs-', 
                linewidth=2.5, markersize=10, label='Desv. Estándar')
        ax4.plot(var_mean.index, var_mean.values, 'ro-', 
                linewidth=2.5, markersize=10, label='Media')
        ax4.set_xlabel('Varianza del Error', fontsize=11, fontweight='bold')
        ax4.set_ylabel('Valor', fontsize=11, fontweight='bold')
        ax4.set_title('Estabilidad vs Varianza del Error', fontsize=12, fontweight='bold')
        ax4.legend(fontsize=10)
        ax4.grid(True, alpha=0.3)
        
        fig.suptitle(f'{modelo} - Análisis de Estabilidad del Modelo', 
                    fontsize=16, fontweight='bold', y=0.995)
        
        plt.tight_layout()
        plt.savefig(dir_salida / '15_analisis_estabilidad.png', dpi=300, bbox_inches='tight')
        plt.close()
        print("✓ Completado\n")
    
    def _guardar_estadisticas_completas(self, df, modelo, dir_salida, resultados):
        """Guarda estadísticas completas en archivo de texto"""
        print("💾 Guardando estadísticas completas...")
        
        with open(dir_salida / 'estadisticas_completas.txt', 'w', encoding='utf-8') as f:
            f.write("="*80 + "\n")
            f.write(f"ESTADÍSTICAS COMPLETAS: {modelo}\n")
            f.write(f"Escenario: {resultados['escenario']}\n")
            f.write(f"Observaciones: {resultados['n_observaciones']}\n")
            f.write("="*80 + "\n\n")
            
            # Estadísticas generales
            f.write("ESTADÍSTICAS GENERALES\n")
            f.write("-"*80 + "\n")
            f.write(f"Media: {df[modelo].mean():.6f}\n")
            f.write(f"Mediana: {df[modelo].median():.6f}\n")
            f.write(f"Desv. Estándar: {df[modelo].std():.6f}\n")
            f.write(f"Mínimo: {df[modelo].min():.6f}\n")
            f.write(f"Máximo: {df[modelo].max():.6f}\n")
            f.write(f"Rango: {df[modelo].max() - df[modelo].min():.6f}\n")
            f.write(f"Q1: {df[modelo].quantile(0.25):.6f}\n")
            f.write(f"Q3: {df[modelo].quantile(0.75):.6f}\n")
            f.write(f"IQR: {df[modelo].quantile(0.75) - df[modelo].quantile(0.25):.6f}\n\n")
            
            # Por Distribución
            f.write("\nESTADÍSTICAS POR DISTRIBUCIÓN\n")
            f.write("-"*80 + "\n")
            stats_dist = df.groupby('Distribución')[modelo].describe()
            f.write(stats_dist.to_string() + "\n\n")
            
            # Por Tipo de Modelo
            f.write("\nESTADÍSTICAS POR TIPO DE GENERADOR\n")
            f.write("-"*80 + "\n")
            stats_tipo = df.groupby('Tipo de Modelo')[modelo].describe()
            f.write(stats_tipo.to_string() + "\n\n")
            
            # Por Paso
            f.write("\nESTADÍSTICAS POR PASO\n")
            f.write("-"*80 + "\n")
            stats_paso = df.groupby('Paso')[modelo].describe()
            f.write(stats_paso.to_string() + "\n\n")
            
            # Por Varianza
            f.write("\nESTADÍSTICAS POR VARIANZA\n")
            f.write("-"*80 + "\n")
            stats_var = df.groupby('Varianza error')[modelo].describe()
            f.write(stats_var.to_string() + "\n\n")
            
            # Importancia de características
            if 'importancia_caracteristicas' in resultados:
                f.write("\nIMPORTANCIA DE CARACTERÍSTICAS\n")
                f.write("-"*80 + "\n")
                for car, val in resultados['importancia_caracteristicas'].items():
                    f.write(f"{car}: {val:.2f}%\n")
                f.write("\n")
            
            # Correlaciones
            if 'correlaciones' in resultados:
                f.write("\nCORRELACIONES\n")
                f.write("-"*80 + "\n")
                for car, val in resultados['correlaciones'].items():
                    f.write(f"{car}: {val:.4f}\n")
        
        print("✓ Completado\n")
    
    def ejecutar_analisis_completo(self):
        """Ejecuta el análisis completo para todos los modelos y escenarios"""
        print("\n" + "="*80)
        print("INICIANDO ANÁLISIS COMPLETO DE MODELOS")
        print("="*80 + "\n")
        
        total_analisis = len(ESCENARIOS) * len(MODELOS)
        contador = 0
        
        for escenario in ESCENARIOS:
            for modelo in MODELOS:
                contador += 1
                print(f"\n[{contador}/{total_analisis}] Procesando...")
                
                try:
                    resultado = self.analizar_modelo_escenario(escenario, modelo)
                    key = f"{escenario}_{modelo}"
                    self.resultados_analisis[key] = resultado
                except Exception as e:
                    print(f"❌ Error en {modelo} - {escenario}: {str(e)}")
                    import traceback
                    traceback.print_exc()
                    continue
            
            # Generar ranking DM después de cada escenario
            print(f"\n{'='*80}")
            print(f"GENERANDO RANKING PARA ESCENARIO: {escenario}")
            print(f"{'='*80}\n")
            self._generar_ranking_escenario(escenario)
        
        # Generar resumen comparativo global
        self._generar_resumen_comparativo_global()
        
        print("\n" + "="*80)
        print("✅ ANÁLISIS COMPLETO FINALIZADO")
        print(f"   Total de análisis realizados: {contador}")
        print(f"   Resultados guardados en: ./Resultados/")
        print("="*80 + "\n")
    
    def _generar_ranking_escenario(self, escenario):
        """Genera ranking con test DM para un escenario específico"""
        print(f"📊 Generando ranking con Test Diebold-Mariano para {escenario}...")
        
        dir_salida = Path(f"./Resultados/{escenario}")
        dir_salida.mkdir(parents=True, exist_ok=True)
        
        df_esc = self.df[self.df['Escenario'] == escenario].copy()
        
        # Realizar comparaciones DM
        df_comparaciones, alpha_bonf = comparaciones_multiples_dm(df_esc, MODELOS, alpha=0.05)
        
        # Calcular ranking
        df_ranking, matriz_sup = calcular_ranking_dm(df_comparaciones, MODELOS)
        
        print(f"\n🏆 RANKING (Test Diebold-Mariano):")
        print(df_ranking.to_string(index=False))
        
        # Guardar archivos
        df_comparaciones.to_excel(dir_salida / f'comparaciones_dm_{escenario}.xlsx', index=False)
        df_ranking.to_excel(dir_salida / f'ranking_dm_{escenario}.xlsx', index=False)
        
        # GRÁFICA ÚNICA DEL RANKING
        fig = plt.figure(figsize=(16, 10))
        gs = fig.add_gridspec(2, 2, hspace=0.3, wspace=0.3)
        
        # 1. Ranking principal (más grande)
        ax1 = fig.add_subplot(gs[0, :])
        colors_rank = plt.cm.RdYlGn_r(np.linspace(0.2, 0.9, len(df_ranking)))
        bars = ax1.barh(df_ranking['Modelo'], df_ranking['Score'], 
                       color=colors_rank, edgecolor='black', linewidth=2)
        
        # Añadir valores
        for i, (bar, score, rank, vic, der) in enumerate(zip(bars, df_ranking['Score'], 
                                                              df_ranking['Rank'], 
                                                              df_ranking['Victorias'],
                                                              df_ranking['Derrotas'])):
            width = bar.get_width()
            ax1.text(width, i, f'  #{rank} | Score: {int(score)} (V:{int(vic)} D:{int(der)})', 
                    va='center', fontsize=11, fontweight='bold')
        
        ax1.set_xlabel('Score (Victorias - Derrotas)', fontsize=14, fontweight='bold')
        ax1.set_title(f'🏆 RANKING DE MODELOS - {escenario}\n(Test Diebold-Mariano con corrección Bonferroni)', 
                     fontsize=16, fontweight='bold', pad=20)
        ax1.axvline(0, color='black', linestyle='-', linewidth=1.5)
        ax1.grid(True, alpha=0.3, axis='x', linestyle='--')
        ax1.set_ylabel('Modelo', fontsize=13, fontweight='bold')
        
        # 2. Matriz de superioridad
        ax2 = fig.add_subplot(gs[1, 0])
        sns.heatmap(matriz_sup, annot=True, fmt='.0f', cmap='RdYlGn', 
                   center=0, ax=ax2, cbar_kws={'label': 'Superioridad'},
                   vmin=-1, vmax=1, linewidths=0.5, linecolor='gray')
        ax2.set_title('Matriz de Superioridad\n(1: gana, -1: pierde, 0: empate)', 
                     fontsize=12, fontweight='bold')
        ax2.set_xlabel('Modelo Comparado', fontsize=10)
        ax2.set_ylabel('Modelo', fontsize=10)
        ax2.tick_params(labelsize=9)
        
        # 3. Victorias vs Derrotas
        ax3 = fig.add_subplot(gs[1, 1])
        x = np.arange(len(df_ranking))
        width_bar = 0.35
        
        bars_v = ax3.bar(x - width_bar/2, df_ranking['Victorias'], width_bar, 
                        label='Victorias', color='green', alpha=0.8, edgecolor='black')
        bars_d = ax3.bar(x + width_bar/2, df_ranking['Derrotas'], width_bar, 
                        label='Derrotas', color='red', alpha=0.8, edgecolor='black')
        
        # Añadir valores sobre barras
        for bar in bars_v:
            height = bar.get_height()
            ax3.text(bar.get_x() + bar.get_width()/2., height,
                    f'{int(height)}', ha='center', va='bottom', fontsize=9, fontweight='bold')
        
        for bar in bars_d:
            height = bar.get_height()
            ax3.text(bar.get_x() + bar.get_width()/2., height,
                    f'{int(height)}', ha='center', va='bottom', fontsize=9, fontweight='bold')
        
        ax3.set_xlabel('Modelo', fontsize=11, fontweight='bold')
        ax3.set_ylabel('Cantidad', fontsize=11, fontweight='bold')
        ax3.set_title('Victorias vs Derrotas\n(Comparaciones significativas)', 
                     fontsize=12, fontweight='bold')
        ax3.set_xticks(x)
        ax3.set_xticklabels(df_ranking['Modelo'], rotation=45, ha='right', fontsize=9)
        ax3.legend(fontsize=10)
        ax3.grid(True, alpha=0.3, axis='y')
        
        plt.tight_layout()
        plt.savefig(dir_salida / f'RANKING_{escenario}.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        print(f"✅ Ranking guardado: RANKING_{escenario}.png\n")
    
    def _generar_resumen_comparativo_global(self):
        """Genera resumen comparativo global entre todos los escenarios"""
        print("\n" + "="*80)
        print("GENERANDO RESUMEN COMPARATIVO GLOBAL")
        print("="*80 + "\n")
        
        dir_salida = Path("./Resultados/Comparativo_Global")
        dir_salida.mkdir(parents=True, exist_ok=True)
        
        # ============================================================
        # RANKING GLOBAL CON DM TEST
        # ============================================================
        print("📊 Calculando ranking global (todos los escenarios)...\n")
        
        df_comparaciones_global, alpha_bonf = comparaciones_multiples_dm(
            self.df, MODELOS, alpha=0.05
        )
        
        df_ranking_global, matriz_sup_global = calcular_ranking_dm(
            df_comparaciones_global, MODELOS
        )
        
        print(f"\n🏆 RANKING GLOBAL (TODOS LOS ESCENARIOS):")
        print(df_ranking_global.to_string(index=False))
        
        # Guardar resultados globales
        df_comparaciones_global.to_excel(
            dir_salida / 'comparaciones_dm_global.xlsx', 
            index=False
        )
        df_ranking_global.to_excel(
            dir_salida / 'ranking_dm_global.xlsx', 
            index=False
        )
        
        # GRÁFICA ÚNICA DEL RANKING GLOBAL
        fig = plt.figure(figsize=(18, 12))
        gs = fig.add_gridspec(3, 2, hspace=0.35, wspace=0.3)
        
        # 1. Ranking global principal
        ax1 = fig.add_subplot(gs[0, :])
        colors_rank = plt.cm.RdYlGn_r(np.linspace(0.2, 0.9, len(df_ranking_global)))
        bars = ax1.barh(df_ranking_global['Modelo'], df_ranking_global['Score'], 
                       color=colors_rank, edgecolor='black', linewidth=2.5)
        
        # Añadir información detallada
        for i, (bar, score, rank, vic, der, pct) in enumerate(zip(bars, 
                                                                   df_ranking_global['Score'],
                                                                   df_ranking_global['Rank'],
                                                                   df_ranking_global['Victorias'],
                                                                   df_ranking_global['Derrotas'],
                                                                   df_ranking_global['Pct_Victorias'])):
            width = bar.get_width()
            ax1.text(width, i, 
                    f'  #{rank} | Score: {int(score)} | V:{int(vic)} D:{int(der)} | {pct:.1f}%', 
                    va='center', fontsize=10, fontweight='bold')
        
        ax1.set_xlabel('Score Global (Victorias - Derrotas)', fontsize=14, fontweight='bold')
        ax1.set_title('🏆 RANKING GLOBAL DE MODELOS (Todos los Escenarios)\n' +
                     'Test Diebold-Mariano con corrección de Bonferroni', 
                     fontsize=17, fontweight='bold', pad=20)
        ax1.axvline(0, color='black', linestyle='-', linewidth=2)
        ax1.grid(True, alpha=0.3, axis='x', linestyle='--')
        ax1.set_ylabel('Modelo', fontsize=13, fontweight='bold')
        
        # 2. Matriz de superioridad global
        ax2 = fig.add_subplot(gs[1, :])
        sns.heatmap(matriz_sup_global, annot=True, fmt='.0f', cmap='RdYlGn', 
                   center=0, ax=ax2, cbar_kws={'label': 'Superioridad Global'},
                   vmin=-1, vmax=1, linewidths=1, linecolor='gray')
        ax2.set_title('Matriz de Superioridad Global\n(1: superior, -1: inferior, 0: sin diferencia)', 
                     fontsize=13, fontweight='bold', pad=15)
        ax2.set_xlabel('Modelo Comparado', fontsize=11, fontweight='bold')
        ax2.set_ylabel('Modelo', fontsize=11, fontweight='bold')
        ax2.tick_params(labelsize=10)
        
        # 3. Rendimiento promedio por escenario
        ax3 = fig.add_subplot(gs[2, 0])
        
        rendimientos_esc = []
        for escenario in ESCENARIOS:
            df_esc = self.df[self.df['Escenario'] == escenario]
            medias = df_esc[MODELOS].mean()
            rendimientos_esc.append(medias)
        
        df_rend = pd.DataFrame(rendimientos_esc, index=ESCENARIOS, columns=MODELOS)
        
        # Ordenar por ranking global
        modelos_ordenados = df_ranking_global.sort_values('Rank')['Modelo'].tolist()
        df_rend = df_rend[modelos_ordenados]
        
        # Plotear líneas
        for escenario in ESCENARIOS:
            ax3.plot(range(len(modelos_ordenados)), df_rend.loc[escenario], 
                    marker='o', linewidth=2.5, markersize=8, label=escenario)
        
        ax3.set_xlabel('Modelos (ordenados por ranking)', fontsize=11, fontweight='bold')
        ax3.set_ylabel('Rendimiento Promedio', fontsize=11, fontweight='bold')
        ax3.set_title('Rendimiento por Escenario', fontsize=12, fontweight='bold')
        ax3.set_xticks(range(len(modelos_ordenados)))
        ax3.set_xticklabels(modelos_ordenados, rotation=45, ha='right', fontsize=9)
        ax3.legend(fontsize=9)
        ax3.grid(True, alpha=0.3)
        
        # 4. Victorias totales
        ax4 = fig.add_subplot(gs[2, 1])
        
        colors_vic = plt.cm.RdYlGn_r(np.linspace(0.3, 0.9, len(df_ranking_global)))
        bars_vic = ax4.bar(range(len(df_ranking_global)), df_ranking_global['Victorias'], 
                          color=colors_vic, edgecolor='black', linewidth=1.5, alpha=0.8)
        
        # Añadir porcentajes
        for i, (bar, vic, pct) in enumerate(zip(bars_vic, 
                                                 df_ranking_global['Victorias'],
                                                 df_ranking_global['Pct_Victorias'])):
            height = bar.get_height()
            ax4.text(i, height, f'{int(vic)}\n({pct:.1f}%)', 
                    ha='center', va='bottom', fontsize=9, fontweight='bold')
        
        ax4.set_xlabel('Modelo', fontsize=11, fontweight='bold')
        ax4.set_ylabel('Número de Victorias', fontsize=11, fontweight='bold')
        ax4.set_title('Total de Victorias Significativas', fontsize=12, fontweight='bold')
        ax4.set_xticks(range(len(df_ranking_global)))
        ax4.set_xticklabels(df_ranking_global['Modelo'], rotation=45, ha='right', fontsize=9)
        ax4.grid(True, alpha=0.3, axis='y')
        
        plt.tight_layout()
        plt.savefig(dir_salida / 'RANKING_GLOBAL.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        print(f"\n✅ Ranking global guardado: RANKING_GLOBAL.png")
        
        # ============================================================
        # RESUMEN POR ESCENARIO
        # ============================================================
        print("\n" + "="*80)
        print("🏆 MEJORES MODELOS POR ESCENARIO")
        print("="*80)
        
        resultados_por_escenario = []
        
        for escenario in ESCENARIOS:
            df_esc = self.df[self.df['Escenario'] == escenario]
            
            # Realizar DM test para este escenario
            df_comp_esc, _ = comparaciones_multiples_dm(df_esc, MODELOS, alpha=0.05)
            df_rank_esc, _ = calcular_ranking_dm(df_comp_esc, MODELOS)
            
            mejor = df_rank_esc.iloc[0]
            
            print(f"\n{escenario}:")
            print(f"  🥇 Mejor modelo: {mejor['Modelo']}")
            print(f"  📊 Score DM: {mejor['Score']} (V:{mejor['Victorias']}, D:{mejor['Derrotas']}, E:{mejor['Empates']})")
            print(f"  📈 % Victorias: {mejor['Pct_Victorias']:.1f}%")
            
            # Estadísticas descriptivas del mejor modelo
            media = df_esc[mejor['Modelo']].mean()
            std = df_esc[mejor['Modelo']].std()
            minimo = df_esc[mejor['Modelo']].min()
            maximo = df_esc[mejor['Modelo']].max()
            
            print(f"  📊 Rendimiento: {media:.4f} ± {std:.4f}")
            print(f"  📊 Rango: [{minimo:.4f}, {maximo:.4f}]")
            
            # Top 3
            print(f"\n  Top 3:")
            for i, row in df_rank_esc.head(3).iterrows():
                print(f"    {row['Rank']}. {row['Modelo']} (Score: {row['Score']}, {row['Pct_Victorias']:.1f}% victorias)")
            
            resultados_por_escenario.append({
                'Escenario': escenario,
                'Mejor_Modelo': mejor['Modelo'],
                'Rank': mejor['Rank'],
                'Score_DM': mejor['Score'],
                'Victorias': mejor['Victorias'],
                'Derrotas': mejor['Derrotas'],
                'Pct_Victorias': mejor['Pct_Victorias'],
                'Media': media,
                'Std': std,
                'Min': minimo,
                'Max': maximo
            })
        
        # Guardar resumen por escenario
        df_resumen_escenarios = pd.DataFrame(resultados_por_escenario)
        df_resumen_escenarios.to_excel(
            dir_salida / 'resumen_mejores_por_escenario.xlsx', 
            index=False
        )
        
        print("\n" + "="*80)
        print("✅ Análisis comparativo global completado")
        print("="*80 + "\n")
        
        # ============================================================
        # GENERAR EXCEL CONSOLIDADO
        # ============================================================
        self._generar_excel_consolidado()
        
    def _generar_excel_consolidado(self):
        """Genera un archivo Excel consolidado con todas las métricas y análisis"""
        print("\n" + "="*80)
        print("📊 GENERANDO EXCEL CONSOLIDADO")
        print("="*80 + "\n")
        
        dir_salida = Path("./Resultados")
        archivo_excel = dir_salida / "ANALISIS_CONSOLIDADO_COMPLETO.xlsx"
        
        with pd.ExcelWriter(archivo_excel, engine='openpyxl') as writer:
            
            # ============================================================
            # HOJA 1: RESUMEN GENERAL POR MODELO-ESCENARIO
            # ============================================================
            print("📋 Generando HOJA 1: Resumen General por Modelo-Escenario...")
            
            resumen_general = []
            
            # Obtener ranking global
            df_comparaciones_global, _ = comparaciones_multiples_dm(self.df, MODELOS, alpha=0.05)
            df_ranking_global, _ = calcular_ranking_dm(df_comparaciones_global, MODELOS)
            
            for escenario in ESCENARIOS:
                df_esc = self.df[self.df['Escenario'] == escenario]
                
                # Obtener ranking del escenario
                df_comp_esc, _ = comparaciones_multiples_dm(df_esc, MODELOS, alpha=0.05)
                df_rank_esc, _ = calcular_ranking_dm(df_comp_esc, MODELOS)
                
                for modelo in MODELOS:
                    # Estadísticas básicas
                    rendimiento_prom = df_esc[modelo].mean()
                    desv_est = df_esc[modelo].std()
                    coef_var = (desv_est / rendimiento_prom * 100) if rendimiento_prom != 0 else 0
                    
                    # Ranking global del modelo
                    rank_global = df_ranking_global[df_ranking_global['Modelo'] == modelo]['Rank'].values[0]
                    score_dm_global = df_ranking_global[df_ranking_global['Modelo'] == modelo]['Score'].values[0]
                    victorias_global = df_ranking_global[df_ranking_global['Modelo'] == modelo]['Victorias'].values[0]
                    pct_victorias_global = df_ranking_global[df_ranking_global['Modelo'] == modelo]['Pct_Victorias'].values[0]
                    
                    # Ranking en el escenario
                    rank_escenario = df_rank_esc[df_rank_esc['Modelo'] == modelo]['Rank'].values[0]
                    score_dm_escenario = df_rank_esc[df_rank_esc['Modelo'] == modelo]['Score'].values[0]
                    victorias_escenario = df_rank_esc[df_rank_esc['Modelo'] == modelo]['Victorias'].values[0]
                    pct_victorias_escenario = df_rank_esc[df_rank_esc['Modelo'] == modelo]['Pct_Victorias'].values[0]
                    
                    resumen_general.append({
                        'Escenario': escenario,
                        'Modelo': modelo,
                        'Rendimiento_Promedio': round(rendimiento_prom, 6),
                        'Desviacion_Estandar': round(desv_est, 6),
                        'Ranking_Global': int(rank_global),
                        'Score_DM_Global': int(score_dm_global),
                        'Victorias_Significativas_Global': int(victorias_global),
                        '%_Victorias_Global': round(pct_victorias_global, 2),
                        'Ranking_Escenario': int(rank_escenario),
                        'Score_DM_Escenario': int(score_dm_escenario),
                        'Victorias_Significativas_Escenario': int(victorias_escenario),
                        '%_Victorias_Escenario': round(pct_victorias_escenario, 2),
                        'Estabilidad_General': round(coef_var, 2),
                        'Minimo': round(df_esc[modelo].min(), 6),
                        'Maximo': round(df_esc[modelo].max(), 6),
                        'Mediana': round(df_esc[modelo].median(), 6),
                        'N_Observaciones': len(df_esc)
                    })
            
            df_hoja1 = pd.DataFrame(resumen_general)
            df_hoja1.to_excel(writer, sheet_name='Resumen_General', index=False)
            print("✓ Completado\n")
            
            # ============================================================
            # HOJA 2: RENDIMIENTO POR CARACTERÍSTICAS
            # ============================================================
            print("📋 Generando HOJA 2: Rendimiento por Características...")
            
            rendimiento_caracteristicas = []
            
            caracteristicas_cols = {
                'Distribución': 'Distribución',
                'Tipo_Generador': 'Tipo de Modelo',
                'Paso': 'Paso',
                'Varianza': 'Varianza error'
            }
            
            for escenario in ESCENARIOS:
                df_esc = self.df[self.df['Escenario'] == escenario]
                
                for modelo in MODELOS:
                    for caract_nombre, caract_col in caracteristicas_cols.items():
                        
                        stats = df_esc.groupby(caract_col)[modelo].agg([
                            'mean', 'std', 'count', 'min', 'max', 'median'
                        ])
                        
                        for categoria in stats.index:
                            rendimiento_caracteristicas.append({
                                'Escenario': escenario,
                                'Modelo': modelo,
                                'Caracteristica': caract_nombre,
                                'Categoria': str(categoria),
                                'Rendimiento_Promedio': round(stats.loc[categoria, 'mean'], 6),
                                'Desviacion_Estandar': round(stats.loc[categoria, 'std'], 6),
                                'N_Observaciones': int(stats.loc[categoria, 'count']),
                                'Minimo': round(stats.loc[categoria, 'min'], 6),
                                'Maximo': round(stats.loc[categoria, 'max'], 6),
                                'Mediana': round(stats.loc[categoria, 'median'], 6)
                            })
            
            df_hoja2 = pd.DataFrame(rendimiento_caracteristicas)
            df_hoja2.to_excel(writer, sheet_name='Rendimiento_Caracteristicas', index=False)
            print("✓ Completado\n")
            
            # ============================================================
            # HOJA 3: IMPORTANCIA DE CARACTERÍSTICAS
            # ============================================================
            print("📋 Generando HOJA 3: Importancia de Características...")
            
            importancia_caracteristicas = []
            
            for escenario in ESCENARIOS:
                df_esc = self.df[self.df['Escenario'] == escenario]
                
                for modelo in MODELOS:
                    # Calcular importancia basada en rango de variación
                    importancias = {}
                    correlaciones = {}
                    
                    # Distribución
                    dist_rango = df_esc.groupby('Distribución')[modelo].mean().max() - \
                                df_esc.groupby('Distribución')[modelo].mean().min()
                    importancias['Distribución'] = dist_rango
                    df_temp = df_esc.copy()
                    df_temp['Dist_num'] = pd.Categorical(df_temp['Distribución']).codes
                    correlaciones['Distribución'] = abs(df_temp['Dist_num'].corr(df_temp[modelo]))
                    
                    # Tipo de Generador
                    tipo_rango = df_esc.groupby('Tipo de Modelo')[modelo].mean().max() - \
                                df_esc.groupby('Tipo de Modelo')[modelo].mean().min()
                    importancias['Tipo_Generador'] = tipo_rango
                    df_temp['Tipo_num'] = pd.Categorical(df_temp['Tipo de Modelo']).codes
                    correlaciones['Tipo_Generador'] = abs(df_temp['Tipo_num'].corr(df_temp[modelo]))
                    
                    # Paso
                    paso_rango = df_esc.groupby('Paso')[modelo].mean().max() - \
                                df_esc.groupby('Paso')[modelo].mean().min()
                    importancias['Paso'] = paso_rango
                    correlaciones['Paso'] = abs(df_esc['Paso'].corr(df_esc[modelo]))
                    
                    # Varianza
                    var_rango = df_esc.groupby('Varianza error')[modelo].mean().max() - \
                            df_esc.groupby('Varianza error')[modelo].mean().min()
                    importancias['Varianza'] = var_rango
                    correlaciones['Varianza'] = abs(df_esc['Varianza error'].corr(df_esc[modelo]))
                    
                    # Normalizar importancias (0-100)
                    max_imp = max(importancias.values())
                    if max_imp > 0:
                        importancias_norm = {k: (v/max_imp)*100 for k, v in importancias.items()}
                    else:
                        importancias_norm = {k: 0 for k in importancias.keys()}
                    
                    # Crear ranking
                    items_ordenados = sorted(importancias_norm.items(), key=lambda x: x[1], reverse=True)
                    
                    for rank, (caract, imp_norm) in enumerate(items_ordenados, 1):
                        importancia_caracteristicas.append({
                            'Escenario': escenario,
                            'Modelo': modelo,
                            'Caracteristica': caract,
                            'Importancia_Relativa': round(imp_norm, 2),
                            'Correlacion_Absoluta': round(correlaciones[caract], 4),
                            'Rank_Importancia': rank,
                            'Rango_Variacion': round(importancias[caract], 6)
                        })
            
            df_hoja3 = pd.DataFrame(importancia_caracteristicas)
            df_hoja3.to_excel(writer, sheet_name='Importancia_Caracteristicas', index=False)
            print("✓ Completado\n")
            
            # ============================================================
            # HOJA 4: ESTABILIDAD Y OUTLIERS
            # ============================================================
            print("📋 Generando HOJA 4: Estabilidad y Outliers...")
            
            estabilidad_outliers = []
            
            for escenario in ESCENARIOS:
                df_esc = self.df[self.df['Escenario'] == escenario]
                
                for modelo in MODELOS:
                    # Coeficiente de variación global
                    media = df_esc[modelo].mean()
                    std = df_esc[modelo].std()
                    coef_var = (std / media * 100) if media != 0 else 0
                    
                    # Outliers usando IQR
                    Q1 = df_esc[modelo].quantile(0.25)
                    Q3 = df_esc[modelo].quantile(0.75)
                    IQR = Q3 - Q1
                    lower_bound = Q1 - 1.5 * IQR
                    upper_bound = Q3 + 1.5 * IQR
                    
                    outliers = df_esc[(df_esc[modelo] < lower_bound) | (df_esc[modelo] > upper_bound)]
                    n_outliers = len(outliers)
                    pct_outliers = (n_outliers / len(df_esc)) * 100 if len(df_esc) > 0 else 0
                    
                    # Estabilidad por dimensiones
                    estab_paso = df_esc.groupby('Paso')[modelo].std().mean()
                    estab_tipo = df_esc.groupby('Tipo de Modelo')[modelo].std().mean()
                    estab_dist = df_esc.groupby('Distribución')[modelo].std().mean()
                    estab_var = df_esc.groupby('Varianza error')[modelo].std().mean()
                    
                    estabilidad_outliers.append({
                        'Escenario': escenario,
                        'Modelo': modelo,
                        'Coef_Variacion_Global': round(coef_var, 2),
                        'N_Outliers': int(n_outliers),
                        '%_Outliers': round(pct_outliers, 2),
                        'Estabilidad_Paso': round(estab_paso, 6),
                        'Estabilidad_Tipo': round(estab_tipo, 6),
                        'Estabilidad_Distribucion': round(estab_dist, 6),
                        'Estabilidad_Varianza': round(estab_var, 6),
                        'Rango_IQR': round(IQR, 6),
                        'Q1': round(Q1, 6),
                        'Q3': round(Q3, 6),
                        'Limite_Inferior': round(lower_bound, 6),
                        'Limite_Superior': round(upper_bound, 6)
                    })
            
            df_hoja4 = pd.DataFrame(estabilidad_outliers)
            df_hoja4.to_excel(writer, sheet_name='Estabilidad_Outliers', index=False)
            print("✓ Completado\n")
            
            # ============================================================
            # HOJA 5: MEJORES/PEORES CONFIGURACIONES
            # ============================================================
            print("📋 Generando HOJA 5: Mejores/Peores Configuraciones...")
            
            configuraciones_extremas = []
            
            for escenario in ESCENARIOS:
                df_esc = self.df[self.df['Escenario'] == escenario]
                
                for modelo in MODELOS:
                    # Encontrar mejor configuración
                    idx_mejor = df_esc[modelo].idxmin()
                    mejor_config = df_esc.loc[idx_mejor]
                    mejor_configuracion = f"{mejor_config['Distribución']}+{mejor_config['Tipo de Modelo']}+Paso{mejor_config['Paso']}+Var{mejor_config['Varianza error']}"
                    rendimiento_mejor = df_esc[modelo].min()
                    
                    # Encontrar peor configuración
                    idx_peor = df_esc[modelo].idxmax()
                    peor_config = df_esc.loc[idx_peor]
                    peor_configuracion = f"{peor_config['Distribución']}+{peor_config['Tipo de Modelo']}+Paso{peor_config['Paso']}+Var{peor_config['Varianza error']}"
                    rendimiento_peor = df_esc[modelo].max()
                    
                    # Amplitud de rango
                    amplitud_rango = rendimiento_peor - rendimiento_mejor
                    
                    # Top 3 mejores y peores
                    df_sorted = df_esc.sort_values(by=modelo)
                    
                    top3_mejor = []
                    for i in range(min(3, len(df_sorted))):
                        row = df_sorted.iloc[i]
                        config = f"{row['Distribución']}+{row['Tipo de Modelo']}+Paso{row['Paso']}+Var{row['Varianza error']}"
                        top3_mejor.append(f"{config}({row[modelo]:.6f})")
                    
                    top3_peor = []
                    for i in range(max(0, len(df_sorted)-3), len(df_sorted)):
                        row = df_sorted.iloc[i]
                        config = f"{row['Distribución']}+{row['Tipo de Modelo']}+Paso{row['Paso']}+Var{row['Varianza error']}"
                        top3_peor.append(f"{config}({row[modelo]:.6f})")
                    
                    configuraciones_extremas.append({
                        'Escenario': escenario,
                        'Modelo': modelo,
                        'Mejor_Configuracion': mejor_configuracion,
                        'Rendimiento_Mejor': round(rendimiento_mejor, 6),
                        'Distribucion_Mejor': mejor_config['Distribución'],
                        'Tipo_Generador_Mejor': mejor_config['Tipo de Modelo'],
                        'Paso_Mejor': int(mejor_config['Paso']),
                        'Varianza_Mejor': mejor_config['Varianza error'],
                        'Peor_Configuracion': peor_configuracion,
                        'Rendimiento_Peor': round(rendimiento_peor, 6),
                        'Distribucion_Peor': peor_config['Distribución'],
                        'Tipo_Generador_Peor': peor_config['Tipo de Modelo'],
                        'Paso_Peor': int(peor_config['Paso']),
                        'Varianza_Peor': peor_config['Varianza error'],
                        'Amplitud_Rango': round(amplitud_rango, 6),
                        'Top3_Mejores': ' | '.join(top3_mejor),
                        'Top3_Peores': ' | '.join(top3_peor)
                    })
            
            df_hoja5 = pd.DataFrame(configuraciones_extremas)
            df_hoja5.to_excel(writer, sheet_name='Configuraciones_Extremas', index=False)
            print("✓ Completado\n")
            
            # ============================================================
            # HOJA 6: RESULTADOS DM DETALLADOS
            # ============================================================
            print("📋 Generando HOJA 6: Resultados DM Detallados...")
            
            resultados_dm_todos = []
            
            for escenario in ESCENARIOS:
                df_esc = self.df[self.df['Escenario'] == escenario]
                df_comp_esc, alpha_bonf = comparaciones_multiples_dm(df_esc, MODELOS, alpha=0.05)
                
                for _, row in df_comp_esc.iterrows():
                    resultados_dm_todos.append({
                        'Escenario': escenario,
                        'Modelo_1': row['Modelo_1'],
                        'Modelo_2': row['Modelo_2'],
                        'DM_Statistic': round(row['DM_Statistic'], 6),
                        'p_value': round(row['p_value'], 6),
                        'p_value_bonferroni': round(row['p_value_bonferroni'], 6),
                        'Significativo': row['Significativo'],
                        'Ganador': row['Ganador'],
                        'Diff_Media': round(row['Diff_Media'], 6),
                        'Interpretacion': row['Interpretacion']
                    })
            
            df_hoja6 = pd.DataFrame(resultados_dm_todos)
            df_hoja6.to_excel(writer, sheet_name='Resultados_DM_Detallados', index=False)
            print("✓ Completado\n")
            
            # ============================================================
            # HOJA 7: RANKING FINAL
            # ============================================================
            print("📋 Generando HOJA 7: Ranking Final...")
            
            ranking_final = []
            
            # Obtener ranking global
            df_comp_global, _ = comparaciones_multiples_dm(self.df, MODELOS, alpha=0.05)
            df_rank_global, _ = calcular_ranking_dm(df_comp_global, MODELOS)
            
            for modelo in MODELOS:
                # Datos globales
                rank_global_data = df_rank_global[df_rank_global['Modelo'] == modelo].iloc[0]
                
                # Calcular consistencia (% de escenarios donde está en top 3)
                top3_count = 0
                victorias_totales = 0
                score_total = 0
                rendimientos_ponderados = []
                
                for escenario in ESCENARIOS:
                    df_esc = self.df[self.df['Escenario'] == escenario]
                    df_comp_esc, _ = comparaciones_multiples_dm(df_esc, MODELOS, alpha=0.05)
                    df_rank_esc, _ = calcular_ranking_dm(df_comp_esc, MODELOS)
                    
                    rank_data = df_rank_esc[df_rank_esc['Modelo'] == modelo].iloc[0]
                    
                    if rank_data['Rank'] <= 3:
                        top3_count += 1
                    
                    victorias_totales += rank_data['Victorias']
                    score_total += rank_data['Score']
                    
                    # Rendimiento promedio en el escenario
                    rend_prom = df_esc[modelo].mean()
                    rendimientos_ponderados.append(rend_prom)
                
                consistencia = (top3_count / len(ESCENARIOS)) * 100
                rendimiento_promedio_ajustado = np.mean(rendimientos_ponderados)
                
                # Score final compuesto (normalizado)
                score_final = (rank_global_data['Score'] * 0.4 + 
                            consistencia * 0.3 + 
                            (100 - rank_global_data['Rank'] * 10) * 0.3)
                
                ranking_final.append({
                    'Rank_Global': int(rank_global_data['Rank']),
                    'Modelo': modelo,
                    'Score_Final': round(score_final, 2),
                    'Score_DM_Global': int(rank_global_data['Score']),
                    'Victorias_Totales': int(victorias_totales),
                    'Victorias_Global': int(rank_global_data['Victorias']),
                    'Derrotas_Global': int(rank_global_data['Derrotas']),
                    '%_Victorias_Global': round(rank_global_data['Pct_Victorias'], 2),
                    'Consistencia': round(consistencia, 2),
                    'Top3_Count': int(top3_count),
                    'Rendimiento_Promedio_Ajustado': round(rendimiento_promedio_ajustado, 6),
                    'Mejor_Escenario': self._get_mejor_escenario(modelo),
                    'Peor_Escenario': self._get_peor_escenario(modelo)
                })
            
            df_hoja7 = pd.DataFrame(ranking_final)
            df_hoja7 = df_hoja7.sort_values('Score_Final', ascending=False).reset_index(drop=True)
            df_hoja7.to_excel(writer, sheet_name='Ranking_Final', index=False)
            print("✓ Completado\n")
            
            # ============================================================
            # HOJA 8: ANÁLISIS DE SENSIBILIDAD COMPLETO
            # ============================================================
            print("📋 Generando HOJA 8: Análisis de Sensibilidad Completo...")
            
            sensibilidad_completa = []
            
            for escenario in ESCENARIOS:
                df_esc = self.df[self.df['Escenario'] == escenario]
                
                # Obtener ranking del escenario
                df_comp_esc, _ = comparaciones_multiples_dm(df_esc, MODELOS, alpha=0.05)
                df_rank_esc, _ = calcular_ranking_dm(df_comp_esc, MODELOS)
                
                for modelo in MODELOS:
                    # Estadísticas básicas
                    promedio = df_esc[modelo].mean()
                    mediana = df_esc[modelo].median()
                    desv_std = df_esc[modelo].std()
                    
                    # Ranking en el escenario
                    rank_escenario = df_rank_esc[df_rank_esc['Modelo'] == modelo]['Rank'].values[0]
                    
                    # ===== ANÁLISIS POR DISTRIBUCIÓN =====
                    dist_stats = df_esc.groupby('Distribución')[modelo].mean()
                    mejor_dist = dist_stats.idxmin()
                    peor_dist = dist_stats.idxmax()
                    sensibilidad_dist = dist_stats.max() - dist_stats.min()
                    
                    # ===== ANÁLISIS POR PASO =====
                    paso_stats = df_esc.groupby('Paso')[modelo].mean()
                    mejor_paso = paso_stats.idxmin()
                    peor_paso = paso_stats.idxmax()
                    sensibilidad_paso = paso_stats.max() - paso_stats.min()
                    
                    # ===== ANÁLISIS POR VARIANZA =====
                    var_stats = df_esc.groupby('Varianza error')[modelo].mean()
                    mejor_varianza = var_stats.idxmin()
                    peor_varianza = var_stats.idxmax()
                    sensibilidad_varianza = var_stats.max() - var_stats.min()
                    
                    # ===== ANÁLISIS POR TIPO DE GENERADOR =====
                    tipo_stats = df_esc.groupby('Tipo de Modelo')[modelo].mean()
                    mejor_tipo = tipo_stats.idxmin()
                    peor_tipo = tipo_stats.idxmax()
                    sensibilidad_tipo = tipo_stats.max() - tipo_stats.min()
                    
                    # ===== MEJOR Y PEOR COMBINACIÓN =====
                    idx_mejor_comb = df_esc[modelo].idxmin()
                    mejor_comb = df_esc.loc[idx_mejor_comb]
                    mejor_combinacion = (f"Dist:{mejor_comb['Distribución']}|"
                                    f"Tipo:{mejor_comb['Tipo de Modelo']}|"
                                    f"Paso:{mejor_comb['Paso']}|"
                                    f"Var:{mejor_comb['Varianza error']}")
                    mejor_comb_valor = df_esc[modelo].min()
                    
                    idx_peor_comb = df_esc[modelo].idxmax()
                    peor_comb = df_esc.loc[idx_peor_comb]
                    peor_combinacion = (f"Dist:{peor_comb['Distribución']}|"
                                    f"Tipo:{peor_comb['Tipo de Modelo']}|"
                                    f"Paso:{peor_comb['Paso']}|"
                                    f"Var:{peor_comb['Varianza error']}")
                    peor_comb_valor = df_esc[modelo].max()
                    
                    # ===== CATEGORÍA MÁS Y MENOS IMPORTANTE =====
                    sensibilidades = {
                        'Distribución': sensibilidad_dist,
                        'Paso': sensibilidad_paso,
                        'Varianza': sensibilidad_varianza,
                        'Tipo_Generador': sensibilidad_tipo
                    }
                    
                    categoria_mas_importante = max(sensibilidades, key=sensibilidades.get)
                    categoria_menos_importante = min(sensibilidades, key=sensibilidades.get)
                    
                    sensibilidad_completa.append({
                        'Escenario': escenario,
                        'Modelo': modelo,
                        'Promedio': round(promedio, 6),
                        'Mediana': round(mediana, 6),
                        'Desviacion_Estandar': round(desv_std, 6),
                        'Ranking_Escenario': int(rank_escenario),
                        'Mejor_Distribucion': mejor_dist,
                        'Mejor_Dist_Valor': round(dist_stats[mejor_dist], 6),
                        'Peor_Distribucion': peor_dist,
                        'Peor_Dist_Valor': round(dist_stats[peor_dist], 6),
                        'Sensibilidad_Distribucion': round(sensibilidad_dist, 6),
                        'Mejor_Paso': int(mejor_paso),
                        'Mejor_Paso_Valor': round(paso_stats[mejor_paso], 6),
                        'Peor_Paso': int(peor_paso),
                        'Peor_Paso_Valor': round(paso_stats[peor_paso], 6),
                        'Sensibilidad_Paso': round(sensibilidad_paso, 6),
                        'Mejor_Varianza': mejor_varianza,
                        'Mejor_Varianza_Valor': round(var_stats[mejor_varianza], 6),
                        'Peor_Varianza': peor_varianza,
                        'Peor_Varianza_Valor': round(var_stats[peor_varianza], 6),
                        'Sensibilidad_Varianza': round(sensibilidad_varianza, 6),
                        'Mejor_Tipo_Generador': mejor_tipo,
                        'Mejor_Tipo_Valor': round(tipo_stats[mejor_tipo], 6),
                        'Peor_Tipo_Generador': peor_tipo,
                        'Peor_Tipo_Valor': round(tipo_stats[peor_tipo], 6),
                        'Sensibilidad_Tipo_Generador': round(sensibilidad_tipo, 6),
                        'Mejor_Combinacion': mejor_combinacion,
                        'Mejor_Comb_Valor': round(mejor_comb_valor, 6),
                        'Peor_Combinacion': peor_combinacion,
                        'Peor_Comb_Valor': round(peor_comb_valor, 6),
                        'Categoria_Mas_Importante': categoria_mas_importante,
                        'Sensibilidad_Categoria_Max': round(sensibilidades[categoria_mas_importante], 6),
                        'Categoria_Menos_Importante': categoria_menos_importante,
                        'Sensibilidad_Categoria_Min': round(sensibilidades[categoria_menos_importante], 6)
                    })
            
            df_hoja8 = pd.DataFrame(sensibilidad_completa)
            df_hoja8 = df_hoja8.sort_values(['Escenario', 'Ranking_Escenario']).reset_index(drop=True)
            df_hoja8.to_excel(writer, sheet_name='Analisis_Sensibilidad', index=False)
            print("✓ Completado\n")
        
        print(f"✅ Excel consolidado generado: {archivo_excel}")
        print(f"   📊 8 hojas creadas con análisis completo\n")

    def _get_mejor_escenario(self, modelo):
            """Obtiene el escenario donde el modelo tiene mejor ranking"""
            mejor_rank = float('inf')
            mejor_esc = ""
            
            for escenario in ESCENARIOS:
                df_esc = self.df[self.df['Escenario'] == escenario]
                df_comp_esc, _ = comparaciones_multiples_dm(df_esc, MODELOS, alpha=0.05)
                df_rank_esc, _ = calcular_ranking_dm(df_comp_esc, MODELOS)
                
                rank = df_rank_esc[df_rank_esc['Modelo'] == modelo]['Rank'].values[0]
                
                if rank < mejor_rank:
                    mejor_rank = rank
                    mejor_esc = escenario
            
            return f"{mejor_esc} (Rank #{int(mejor_rank)})"
        
    def _get_peor_escenario(self, modelo):
            """Obtiene el escenario donde el modelo tiene peor ranking"""
            peor_rank = 0
            peor_esc = ""
            
            for escenario in ESCENARIOS:
                df_esc = self.df[self.df['Escenario'] == escenario]
                df_comp_esc, _ = comparaciones_multiples_dm(df_esc, MODELOS, alpha=0.05)
                df_rank_esc, _ = calcular_ranking_dm(df_comp_esc, MODELOS)
                
                rank = df_rank_esc[df_rank_esc['Modelo'] == modelo]['Rank'].values[0]
                
                if rank > peor_rank:
                    peor_rank = rank
                    peor_esc = escenario
            
            return f"{peor_esc} (Rank #{int(peor_rank)})"


# ============================================================================
# FUNCIÓN PRINCIPAL DE EJECUCIÓN
# ============================================================================

def main():
    """Función principal que ejecuta todo el análisis"""
    print("\n" + "█"*80)
    print("█" + " "*78 + "█")
    print("█" + " "*15 + "ANÁLISIS DETALLADO DE MODELOS DE PREDICCIÓN" + " "*21 + "█")
    print("█" + " "*20 + "CON GRÁFICAS INDIVIDUALES Y RANKING DM" + " "*21 + "█")
    print("█" + " "*78 + "█")
    print("█"*80 + "\n")
    
    try:
        # Crear instancia del analizador
        analizador = AnalizadorModelos(RUTA_DATOS)
        
        # Ejecutar análisis completo
        analizador.ejecutar_analisis_completo()
        
        print("\n" + "█"*80)
        print("█" + " "*78 + "█")
        print("█" + " "*25 + "ANÁLISIS COMPLETADO EXITOSAMENTE" + " "*22 + "█")
        print("█" + " "*78 + "█")
        print("█"*80 + "\n")
        
        print("📁 Los resultados se encuentran en:")
        print("   └── ./Resultados/")
        print("       ├── Estacionario_Lineal/")
        print("       │   ├── RANKING_Estacionario_Lineal.png  ⭐")
        print("       │   ├── LSPM/")
        print("       │   │   ├── 01_rendimiento_distribucion.png")
        print("       │   │   ├── 02_rendimiento_tipo_generador.png")
        print("       │   │   ├── 03_rendimiento_paso.png")
        print("       │   │   ├── 04_rendimiento_varianza.png")
        print("       │   │   ├── 05-10_interacciones (6 gráficas)")
        print("       │   │   ├── 11_importancia_caracteristicas.png")
        print("       │   │   ├── 12_distribucion_general.png")
        print("       │   │   ├── 13_analisis_outliers.png")
        print("       │   │   ├── 14_heatmap_configuraciones.png")
        print("       │   │   ├── 15_analisis_estabilidad.png")
        print("       │   │   └── estadisticas_completas.txt")
        print("       │   └── ... (todos los modelos)")
        print("       ├── No_Lineal_Estacionario/")
        print("       │   └── RANKING_No_Lineal_Estacionario.png  ⭐")
        print("       ├── No_Estacionario_Lineal/")
        print("       │   └── RANKING_No_Estacionario_Lineal.png  ⭐")
        print("       ├── Comparativo_Global/")
        print("       │   ├── RANKING_GLOBAL.png  ⭐⭐⭐")
        print("       │   ├── ranking_dm_global.xlsx")
        print("       │   └── resumen_mejores_por_escenario.xlsx")
        print("       └── ANALISIS_CONSOLIDADO_COMPLETO.xlsx  📊📊📊")
        print("           ├── Hoja 1: Resumen General")
        print("           ├── Hoja 2: Rendimiento por Características")
        print("           ├── Hoja 3: Importancia de Características")
        print("           ├── Hoja 4: Estabilidad y Outliers")
        print("           ├── Hoja 5: Configuraciones Extremas")
        print("           ├── Hoja 6: Resultados DM Detallados")
        print("           └── Hoja 7: Ranking Final")
        print("\n" + "="*80)
        print("NOTA: Cada modelo en cada escenario tiene 15 gráficas individuales")
        print("      + 1 ranking por escenario + 1 ranking global")
        print("      + 1 Excel consolidado con 7 hojas de análisis completo")
        print("="*80 + "\n")
        
    except FileNotFoundError:
        print(f"\n❌ ERROR: No se encontró el archivo {RUTA_DATOS}")
        print("   Por favor, verifica que el archivo existe y la ruta es correcta.\n")
    except Exception as e:
        print(f"\n❌ ERROR INESPERADO: {str(e)}")
        import traceback
        traceback.print_exc()


if __name__ == "__main__":
    main()


████████████████████████████████████████████████████████████████████████████████
█                                                                              █
█               ANÁLISIS DETALLADO DE MODELOS DE PREDICCIÓN                     █
█                    CON GRÁFICAS INDIVIDUALES Y RANKING DM                     █
█                                                                              █
████████████████████████████████████████████████████████████████████████████████

✓ Datos cargados: 2000 filas, 15 columnas

Escenarios encontrados: ['Estacionario_Lineal' 'No_Estacionario_Lineal' 'No_Lineal_Estacionario']
Modelos a analizar: 9

INICIANDO ANÁLISIS COMPLETO DE MODELOS


[1/27] Procesando...

ANÁLISIS: AREPD en escenario Estacionario_Lineal

📊 Generando: Rendimiento por Distribución...
✓ Completado

📊 Generando: Rendimiento por Tipo de Generador...
✓ Completado

📊 Generando: Rendimiento por Paso...
✓ Completado

📊 Generando: Rendimiento por Varianza...
✓ Completado

📊 Ge

## Analisis General Corregido*

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from scipy import stats
from itertools import combinations
import warnings
warnings.filterwarnings('ignore')

# Configuración de estilo
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# ============================================================================
# CONFIGURACIÓN GLOBAL
# ============================================================================

RUTA_DATOS = "./Datos/datos_combinados.xlsx"
DIR_SALIDA = "./resultados_base_completa"

MODELOS = ['AREPD', 'AV-MCPS', 'Block Bootstrapping', 'DeepAR', 
           'EnCQR-LSTM', 'LSPM', 'LSPMW', 'MCPS', 'Sieve Bootstrap']

# Colores únicos para 9 modelos
COLORES_MODELOS = {
    'AREPD': '#e41a1c',
    'AV-MCPS': '#377eb8', 
    'Block Bootstrapping': '#4daf4a',
    'DeepAR': '#984ea3',
    'EnCQR-LSTM': '#ff7f00',
    'LSPM': '#ffff33',
    'LSPMW': '#a65628',
    'MCPS': '#f781bf',
    'Sieve Bootstrap': '#999999'
}

# ============================================================================
# FUNCIONES AUXILIARES - TEST DIEBOLD-MARIANO
# ============================================================================

def diebold_mariano_test(errores1, errores2, h=1, alternative='two-sided'):
    """Test de Diebold-Mariano para comparar precisión de pronósticos"""
    e1 = np.asarray(errores1)
    e2 = np.asarray(errores2)
    
    if len(e1) != len(e2):
        raise ValueError("Los vectores de errores deben tener la misma longitud")
    
    n = len(e1)
    d = e1 - e2
    d_mean = np.mean(d)
    
    # Varianza con corrección de autocorrelación
    gamma_0 = np.var(d, ddof=1)
    gamma_sum = 0
    for k in range(1, h):
        if k < n:
            gamma_k = np.mean((d[:-k] - d_mean) * (d[k:] - d_mean))
            gamma_sum += 2 * gamma_k
    
    var_d = (gamma_0 + gamma_sum) / n
    
    # Corrección de Harvey-Leybourne-Newbold
    hlnc = np.sqrt((n + 1 - 2*h + h*(h-1)/n) / n)
    
    if var_d > 0:
        dm_stat = d_mean / np.sqrt(var_d)
        dm_stat_corrected = dm_stat * hlnc
    else:
        dm_stat = 0
        dm_stat_corrected = 0
    
    # P-valor
    if alternative == 'two-sided':
        p_value = 2 * (1 - stats.t.cdf(abs(dm_stat_corrected), df=n-1))
    elif alternative == 'less':
        p_value = stats.t.cdf(dm_stat_corrected, df=n-1)
    elif alternative == 'greater':
        p_value = 1 - stats.t.cdf(dm_stat_corrected, df=n-1)
    else:
        raise ValueError("alternative debe ser 'two-sided', 'less' o 'greater'")
    
    return {
        'dm_statistic': dm_stat,
        'dm_statistic_corrected': dm_stat_corrected,
        'p_value': p_value,
        'mean_diff': d_mean,
        'modelo1_mejor': d_mean < 0,
        'n': n
    }


def comparaciones_multiples_dm(df, modelos, alpha=0.05):
    """Comparaciones múltiples con corrección de Bonferroni"""
    n_comparaciones = len(list(combinations(modelos, 2)))
    alpha_bonferroni = alpha / n_comparaciones
    
    resultados = []
    
    for modelo1, modelo2 in combinations(modelos, 2):
        try:
            dm_result = diebold_mariano_test(
                df[modelo1].values, 
                df[modelo2].values,
                h=1,
                alternative='two-sided'
            )
            
            significativo = dm_result['p_value'] < alpha_bonferroni
            
            if significativo:
                if dm_result['mean_diff'] < 0:
                    ganador = modelo1
                else:
                    ganador = modelo2
            else:
                ganador = "No hay diferencia"
            
            resultados.append({
                'Modelo_1': modelo1,
                'Modelo_2': modelo2,
                'DM_Statistic': dm_result['dm_statistic_corrected'],
                'p_value': dm_result['p_value'],
                'p_value_bonferroni': alpha_bonferroni,
                'Significativo': significativo,
                'Ganador': ganador,
                'Diff_Media': dm_result['mean_diff']
            })
            
        except Exception as e:
            continue
    
    return pd.DataFrame(resultados), alpha_bonferroni


def calcular_ranking_dm(df_comparaciones, modelos):
    """Calcula ranking basado en resultados DM"""
    n = len(modelos)
    matriz = pd.DataFrame(np.zeros((n, n)), index=modelos, columns=modelos)
    
    for _, row in df_comparaciones.iterrows():
        m1, m2 = row['Modelo_1'], row['Modelo_2']
        if row['Significativo']:
            if row['Ganador'] == m1:
                matriz.loc[m1, m2] = 1
                matriz.loc[m2, m1] = -1
            elif row['Ganador'] == m2:
                matriz.loc[m2, m1] = 1
                matriz.loc[m1, m2] = -1
    
    ranking_data = []
    for modelo in modelos:
        victorias = (matriz.loc[modelo] == 1).sum()
        derrotas = (matriz.loc[modelo] == -1).sum()
        empates = (matriz.loc[modelo] == 0).sum() - 1
        score = victorias - derrotas
        total_comparaciones = victorias + derrotas + empates
        pct_victorias = (victorias / total_comparaciones * 100) if total_comparaciones > 0 else 0
        
        ranking_data.append({
            'Modelo': modelo,
            'Victorias': int(victorias),
            'Derrotas': int(derrotas),
            'Empates': int(empates),
            'Score': int(score),
            'Pct_Victorias': round(pct_victorias, 2)
        })
    
    df_ranking = pd.DataFrame(ranking_data)
    df_ranking = df_ranking.sort_values('Score', ascending=False).reset_index(drop=True)
    df_ranking['Rank'] = range(1, len(df_ranking) + 1)
    
    return df_ranking, matriz


# ============================================================================
# CLASE PRINCIPAL DE ANÁLISIS
# ============================================================================

class AnalizadorBaseCompleta:
    """Análisis completo de la base de datos en 8 dimensiones"""
    
    def __init__(self, ruta_datos):
        """Inicializa el analizador"""
        print("\n" + "="*80)
        print("INICIANDO ANÁLISIS COMPLETO DE BASE DE DATOS")
        print("="*80 + "\n")
        
        self.df = pd.read_excel(ruta_datos)
        self.modelos = MODELOS
        self.dir_salida = Path(DIR_SALIDA)
        self.dir_salida.mkdir(parents=True, exist_ok=True)
        
        # Extraer características del escenario
        self._extraer_caracteristicas()
        
        print(f"✓ Datos cargados: {self.df.shape[0]} filas, {self.df.shape[1]} columnas")
        print(f"✓ Modelos a analizar: {len(self.modelos)}")
        print(f"✓ Directorio de salida: {self.dir_salida}")
        print("\n" + "="*80 + "\n")
    
    def _extraer_caracteristicas(self):
        """Extrae características individuales del escenario"""
        # Crear columnas binarias/categóricas
        self.df['Estacionario'] = self.df['Escenario'].apply(
            lambda x: 'Estacionario' if 'Estacionario' in x and 'No_Estacionario' not in x else 'No Estacionario'
        )
        
        self.df['Lineal'] = self.df['Escenario'].apply(
            lambda x: 'Lineal' if 'Lineal' in x and 'No_Lineal' not in x else 'No Lineal'
        )
        
        print("✓ Características extraídas:")
        print(f"  - Estacionariedad: {self.df['Estacionario'].unique()}")
        print(f"  - Linealidad: {self.df['Lineal'].unique()}")
        print(f"  - Tipos de Modelo: {self.df['Tipo de Modelo'].unique()}")
        print(f"  - Distribuciones: {self.df['Distribución'].unique()}")
        print(f"  - Varianzas: {sorted(self.df['Varianza error'].unique())}")
        print(f"  - Pasos: {sorted(self.df['Paso'].unique())}")
    
    def ejecutar_analisis_completo(self):
        """Ejecuta todos los análisis"""
        print("\n" + "🔬"*40 + "\n")
        
        # 1. Impacto de Estacionariedad
        print("1️⃣  Analizando impacto de Estacionariedad...")
        self._analisis_estacionariedad()
        
        # 2. Impacto de Linealidad
        print("2️⃣  Analizando impacto de Linealidad...")
        self._analisis_linealidad()
        
        # 3. Efecto del Modelo Generador
        print("3️⃣  Analizando efecto del Modelo Generador...")
        self._analisis_modelo_generador()
        
        # 4. Influencia de Distribución
        print("4️⃣  Analizando influencia de Distribución...")
        self._analisis_distribucion()
        
        # 5. Impacto de Varianza
        print("5️⃣  Analizando impacto de Varianza...")
        self._analisis_varianza()
        
        # 6. Deterioro por Horizonte
        print("6️⃣  Analizando deterioro por Horizonte...")
        self._analisis_horizonte()
        
        # 7. Robustez y Estabilidad
        print("7️⃣  Analizando Robustez y Estabilidad...")
        self._analisis_robustez()
        
        # 8. Diferencias Estadísticamente Significativas
        print("8️⃣  Analizando Diferencias Estadísticamente Significativas...")
        self._analisis_significancia()
        
        # Resumen ejecutivo
        print("\n9️⃣  Generando Resumen Ejecutivo...")
        self._generar_resumen_ejecutivo()
        
        print("\n" + "="*80)
        print("✅ ANÁLISIS COMPLETO FINALIZADO")
        print(f"📁 Resultados guardados en: {self.dir_salida}")
        print("="*80 + "\n")
    
    # ========================================================================
    # 1. IMPACTO DE ESTACIONARIEDAD
    # ========================================================================
    
    def _analisis_estacionariedad(self):
        """Analiza el impacto de la estacionariedad"""
        
        # Calcular estadísticas por estacionariedad
        stats_est = []
        for modelo in self.modelos:
            for est in ['Estacionario', 'No Estacionario']:
                df_subset = self.df[self.df['Estacionario'] == est]
                stats_est.append({
                    'Modelo': modelo,
                    'Estacionariedad': est,
                    'Media': df_subset[modelo].mean(),
                    'Std': df_subset[modelo].std(),
                    'Mediana': df_subset[modelo].median()
                })
        
        df_stats = pd.DataFrame(stats_est)
        
        # FIGURA 1.1: Barras comparativas
        fig, ax = plt.subplots(figsize=(14, 9))
        pivot_media = df_stats.pivot(index='Modelo', columns='Estacionariedad', values='Media')
        pivot_media = pivot_media.sort_values('Estacionario')
        
        x = np.arange(len(pivot_media))
        width = 0.35
        
        ax.bar(x - width/2, pivot_media['Estacionario'], width, 
               label='Estacionario', color='lightblue', edgecolor='black', linewidth=1.5)
        ax.bar(x + width/2, pivot_media['No Estacionario'], width, 
               label='No Estacionario', color='lightcoral', edgecolor='black', linewidth=1.5)
        
        ax.set_xlabel('Modelos', fontweight='bold', fontsize=12)
        ax.set_ylabel('Rendimiento Promedio', fontweight='bold', fontsize=12)
        ax.set_title('Impacto de Estacionariedad: Rendimiento Comparativo', 
                     fontweight='bold', fontsize=14, pad=20)
        ax.set_xticks(x)
        ax.set_xticklabels(pivot_media.index, rotation=45, ha='right', fontsize=11)
        ax.legend(fontsize=11, loc='best')
        ax.grid(True, alpha=0.3, axis='y')
        
        plt.tight_layout()
        plt.savefig(self.dir_salida / '1_1_estacionariedad_comparativo.png', 
                   dpi=300, bbox_inches='tight')
        plt.close()
        
        # FIGURA 1.2: Cambio relativo (barras horizontales) - CORREGIDO
        fig, ax = plt.subplots(figsize=(12, 9))
        cambio_rel = ((pivot_media['No Estacionario'] - pivot_media['Estacionario']) / 
                      pivot_media['Estacionario'] * 100)
        cambio_rel = cambio_rel.sort_values()
        
        colors = ['green' if x < 0 else 'red' for x in cambio_rel.values]
        bars = ax.barh(cambio_rel.index, cambio_rel.values, color=colors, 
                       alpha=0.7, edgecolor='black', linewidth=1.5)
        ax.axvline(0, color='black', linestyle='-', linewidth=2)
        ax.set_xlabel('Cambio Relativo (%)', fontweight='bold', fontsize=12)
        ax.set_ylabel('Modelos', fontweight='bold', fontsize=12)
        ax.set_title('Deterioro en Datos No Estacionarios\n(Negativo = Mejora, Positivo = Deterioro)', 
                     fontweight='bold', fontsize=14, pad=20)
        ax.grid(True, alpha=0.3, axis='x')
        
        for i, (bar, val) in enumerate(zip(bars, cambio_rel.values)):
            ax.text(val + (3 if val > 0 else -3), i, f'{val:.1f}%', 
                   va='center', ha='left' if val > 0 else 'right',
                   fontweight='bold', fontsize=10)
        
        plt.tight_layout()
        plt.savefig(self.dir_salida / '1_2_estacionariedad_cambio_relativo.png', 
                   dpi=300, bbox_inches='tight')
        plt.close()
        
        print("   ✓ 2 figuras generadas para estacionariedad\n")
    
    # ========================================================================
    # 2. IMPACTO DE LINEALIDAD
    # ========================================================================
    
    def _analisis_linealidad(self):
        """Analiza el impacto de la linealidad"""
        
        # Calcular estadísticas
        stats_lin = []
        for modelo in self.modelos:
            for lin in ['Lineal', 'No Lineal']:
                df_subset = self.df[self.df['Lineal'] == lin]
                stats_lin.append({
                    'Modelo': modelo,
                    'Linealidad': lin,
                    'Media': df_subset[modelo].mean(),
                    'Std': df_subset[modelo].std(),
                    'Mediana': df_subset[modelo].median()
                })
        
        df_stats = pd.DataFrame(stats_lin)
        
        # FIGURA 2.1: Barras comparativas
        fig, ax = plt.subplots(figsize=(14, 9))
        pivot_media = df_stats.pivot(index='Modelo', columns='Linealidad', values='Media')
        pivot_media = pivot_media.sort_values('Lineal')
        
        x = np.arange(len(pivot_media))
        width = 0.35
        
        ax.bar(x - width/2, pivot_media['Lineal'], width, 
              label='Lineal', color='lightgreen', edgecolor='black', linewidth=1.5)
        ax.bar(x + width/2, pivot_media['No Lineal'], width, 
              label='No Lineal', color='orange', edgecolor='black', linewidth=1.5)
        
        ax.set_xlabel('Modelos', fontweight='bold', fontsize=12)
        ax.set_ylabel('Rendimiento Promedio', fontweight='bold', fontsize=12)
        ax.set_title('Impacto de Linealidad: Rendimiento Comparativo', 
                     fontweight='bold', fontsize=14, pad=20)
        ax.set_xticks(x)
        ax.set_xticklabels(pivot_media.index, rotation=45, ha='right', fontsize=11)
        ax.legend(fontsize=11, loc='best')
        ax.grid(True, alpha=0.3, axis='y')
        
        plt.tight_layout()
        plt.savefig(self.dir_salida / '2_1_linealidad_comparativo.png', 
                   dpi=300, bbox_inches='tight')
        plt.close()
        
        # FIGURA 2.2: Cambio relativo - CORREGIDO
        fig, ax = plt.subplots(figsize=(14, 10))
        cambio_rel = ((pivot_media['No Lineal'] - pivot_media['Lineal']) / 
                      pivot_media['Lineal'] * 100)
        cambio_rel = cambio_rel.sort_values()
        
        colors = ['green' if x < 0 else 'red' for x in cambio_rel.values]
        bars = ax.barh(cambio_rel.index, cambio_rel.values, color=colors, 
                       alpha=0.7, edgecolor='black', linewidth=1.5)
        ax.axvline(0, color='black', linestyle='-', linewidth=2)
        ax.set_xlabel('Cambio Relativo (%)', fontweight='bold', fontsize=12)
        ax.set_ylabel('Modelos', fontweight='bold', fontsize=12)
        ax.set_title('Deterioro en Datos No Lineales\n(Negativo = Mejora, Positivo = Deterioro)', 
                     fontweight='bold', fontsize=14, pad=20)
        ax.grid(True, alpha=0.3, axis='x')
        
        # Ajustar márgenes del eje x para dar espacio a las etiquetas
        x_min = min(cambio_rel.values)
        x_max = max(cambio_rel.values)
        x_range = x_max - x_min
        ax.set_xlim(x_min - x_range * 0.15, x_max + x_range * 0.15)
        
        for i, (bar, val) in enumerate(zip(bars, cambio_rel.values)):
            offset = x_range * 0.02
            ax.text(val + (offset if val > 0 else -offset), i, f'{val:.1f}%', 
                   va='center', ha='left' if val > 0 else 'right',
                   fontweight='bold', fontsize=10)
        
        plt.tight_layout()
        plt.savefig(self.dir_salida / '2_2_linealidad_cambio_relativo.png', 
                   dpi=300, bbox_inches='tight')
        plt.close()
        
        print("   ✓ 2 figuras generadas para linealidad\n")
    
    # ========================================================================
    # 3. EFECTO DEL MODELO GENERADOR
    # ========================================================================
    
    def _analisis_modelo_generador(self):
        """Analiza el efecto del modelo generador de datos"""
        
        pivot_media = self.df.groupby('Tipo de Modelo')[self.modelos].mean()
        tipos = self.df['Tipo de Modelo'].unique()
        
        # FIGURA 3.2: Heatmap normalizado (Z-scores)
        fig, ax = plt.subplots(figsize=(14, 10))
        
        pivot_norm = pivot_media.T.sub(pivot_media.T.mean(axis=1), axis=0).div(pivot_media.T.std(axis=1), axis=0)
        
        sns.heatmap(pivot_norm, annot=True, fmt='.2f', cmap='RdBu_r', center=0,
                   ax=ax, cbar_kws={'label': 'Z-Score'},
                   linewidths=0.5, linecolor='gray', vmin=-2, vmax=2)
        ax.set_xlabel('Tipo de Modelo Generador', fontweight='bold', fontsize=12)
        ax.set_ylabel('Modelo de Predicción', fontweight='bold', fontsize=12)
        ax.set_title('Rendimiento Relativo (Z-Score por Modelo)', 
                     fontweight='bold', fontsize=14, pad=20)
        ax.tick_params(axis='x', rotation=45, labelsize=10)
        ax.tick_params(axis='y', rotation=0, labelsize=10)
        
        plt.tight_layout()
        plt.savefig(self.dir_salida / '3_2_modelo_generador_zscore.png', 
                   dpi=300, bbox_inches='tight')
        plt.close()
        
        # FIGURA 3.3: Variabilidad por tipo
        fig, ax = plt.subplots(figsize=(12, 8))
        
        rankings = []
        for tipo in tipos:
            df_tipo = self.df[self.df['Tipo de Modelo'] == tipo]
            medias = df_tipo[self.modelos].mean().sort_values()
            rankings.append({
                'Tipo': tipo,
                'Mejor_Modelo': medias.index[0],
                'Mejor_Rendimiento': medias.values[0],
                'Peor_Modelo': medias.index[-1],
                'Peor_Rendimiento': medias.values[-1],
                'Rango': medias.values[-1] - medias.values[0]
            })
        
        df_rankings = pd.DataFrame(rankings).sort_values('Rango', ascending=False)
        
        y_pos = np.arange(len(df_rankings))
        bars = ax.barh(y_pos, df_rankings['Rango'].values, 
                       color='steelblue', alpha=0.7, edgecolor='black', linewidth=1.5)
        ax.set_yticks(y_pos)
        ax.set_yticklabels(df_rankings['Tipo'].values, fontsize=10)
        ax.set_xlabel('Rango de Rendimiento (Max - Min)', fontweight='bold', fontsize=12)
        ax.set_title('Variabilidad por Tipo de Generador', 
                     fontweight='bold', fontsize=14, pad=20)
        ax.grid(True, alpha=0.3, axis='x')
        
        for i, (bar, val) in enumerate(zip(bars, df_rankings['Rango'].values)):
            ax.text(val + 0.001, i, f'{val:.3f}', va='center', fontweight='bold', fontsize=10)
        
        plt.tight_layout()
        plt.savefig(self.dir_salida / '3_3_modelo_generador_variabilidad.png', 
                   dpi=300, bbox_inches='tight')
        plt.close()
        
        print("   ✓ 2 figuras generadas para modelo generador\n")
    
    # ========================================================================
    # 4. INFLUENCIA DE LA DISTRIBUCIÓN
    # ========================================================================
    
    def _analisis_distribucion(self):
        """Analiza la influencia de la distribución de errores"""
        
        pivot_media = self.df.groupby('Distribución')[self.modelos].mean()
        pivot_std = self.df.groupby('Distribución')[self.modelos].std()
        
        # FIGURA 4.1: Heatmap de rendimiento
        fig, ax = plt.subplots(figsize=(14, 10))
        
        sns.heatmap(pivot_media.T, annot=True, fmt='.3f', cmap='RdYlGn_r', 
                   ax=ax, cbar_kws={'label': 'Rendimiento Promedio'},
                   linewidths=0.5, linecolor='gray')
        ax.set_xlabel('Distribución de Errores', fontweight='bold', fontsize=12)
        ax.set_ylabel('Modelo de Predicción', fontweight='bold', fontsize=12)
        ax.set_title('Rendimiento por Distribución de Errores', 
                     fontweight='bold', fontsize=14, pad=20)
        ax.tick_params(axis='x', rotation=45, labelsize=10)
        ax.tick_params(axis='y', rotation=0, labelsize=10)
        
        plt.tight_layout()
        plt.savefig(self.dir_salida / '4_1_distribucion_heatmap_rendimiento.png', 
                   dpi=300, bbox_inches='tight')
        plt.close()
        
        # FIGURA 4.2: Heatmap de variabilidad
        fig, ax = plt.subplots(figsize=(14, 10))
        
        sns.heatmap(pivot_std.T, annot=True, fmt='.3f', cmap='YlOrRd', 
                   ax=ax, cbar_kws={'label': 'Desviación Estándar'},
                   linewidths=0.5, linecolor='gray')
        ax.set_xlabel('Distribución de Errores', fontweight='bold', fontsize=12)
        ax.set_ylabel('Modelo de Predicción', fontweight='bold', fontsize=12)
        ax.set_title('Variabilidad por Distribución de Errores', 
                     fontweight='bold', fontsize=14, pad=20)
        ax.tick_params(axis='x', rotation=45, labelsize=10)
        ax.tick_params(axis='y', rotation=0, labelsize=10)
        
        plt.tight_layout()
        plt.savefig(self.dir_salida / '4_2_distribucion_heatmap_variabilidad.png', 
                   dpi=300, bbox_inches='tight')
        plt.close()
        
        print("   ✓ 2 figuras generadas para distribución\n")
    
    # ========================================================================
    # 5. IMPACTO DE VARIANZA
    # ========================================================================
    
    def _analisis_varianza(self):
        """Analiza el impacto del nivel de varianza (ruido)"""
        
        varianzas = sorted(self.df['Varianza error'].unique())
        
        # FIGURA 5.1: Líneas de tendencia - CORREGIDO (colores únicos)
        fig, ax = plt.subplots(figsize=(14, 8))
        
        for modelo in self.modelos:
            medias = [self.df[self.df['Varianza error'] == v][modelo].mean() 
                     for v in varianzas]
            ax.plot(varianzas, medias, marker='o', label=modelo, 
                   linewidth=2.5, markersize=8, color=COLORES_MODELOS[modelo])
        
        ax.set_xlabel('Nivel de Varianza', fontweight='bold', fontsize=12)
        ax.set_ylabel('Rendimiento Promedio', fontweight='bold', fontsize=12)
        ax.set_title('Deterioro con Aumento de Varianza', 
                     fontweight='bold', fontsize=14, pad=20)
        ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)
        ax.grid(True, alpha=0.3)
        ax.set_xticks(varianzas)
        
        plt.tight_layout()
        plt.savefig(self.dir_salida / '5_1_varianza_tendencias.png', 
                   dpi=300, bbox_inches='tight')
        plt.close()
        
        # FIGURA 5.2: Tasa de crecimiento
        fig, ax = plt.subplots(figsize=(12, 8))
        
        tasas_crecimiento = {}
        for modelo in self.modelos:
            medias = [self.df[self.df['Varianza error'] == v][modelo].mean() 
                     for v in varianzas]
            if len(medias) > 1:
                pendiente = (medias[-1] - medias[0]) / (varianzas[-1] - varianzas[0])
                tasas_crecimiento[modelo] = pendiente
        
        tc_sorted = dict(sorted(tasas_crecimiento.items(), key=lambda x: x[1]))
        
        colors_tc = ['green' if v < np.median(list(tc_sorted.values())) else 'red' 
                    for v in tc_sorted.values()]
        bars = ax.barh(range(len(tc_sorted)), list(tc_sorted.values()), 
                       color=colors_tc, alpha=0.7, edgecolor='black', linewidth=1.5)
        ax.set_yticks(range(len(tc_sorted)))
        ax.set_yticklabels(list(tc_sorted.keys()), fontsize=10)
        ax.set_xlabel('Tasa de Crecimiento del Error', fontweight='bold', fontsize=12)
        ax.set_title('Sensibilidad al Ruido\n(Menor = Más Robusto)', 
                     fontweight='bold', fontsize=14, pad=20)
        ax.axvline(np.median(list(tc_sorted.values())), color='black', 
                  linestyle='--', linewidth=2, label='Mediana')
        ax.grid(True, alpha=0.3, axis='x')
        ax.legend(fontsize=11)
        
        for i, (bar, val) in enumerate(zip(bars, tc_sorted.values())):
            ax.text(val + (0.0001 if val > 0 else -0.0001), i, f'{val:.4f}', 
                   va='center', ha='left' if val > 0 else 'right',
                   fontweight='bold', fontsize=9)
        
        plt.tight_layout()
        plt.savefig(self.dir_salida / '5_2_varianza_tasa_crecimiento.png', 
                   dpi=300, bbox_inches='tight')
        plt.close()
        
        print("   ✓ 2 figuras generadas para varianza\n")
    
    # ========================================================================
    # 6. DETERIORO POR HORIZONTE
    # ========================================================================
    
    def _analisis_horizonte(self):
        """Analiza el deterioro del rendimiento con el horizonte de predicción"""
        
        pasos = sorted(self.df['Paso'].unique())
        
        # FIGURA 6.1: Evolución paso a paso - CORREGIDO (colores únicos)
        fig, ax = plt.subplots(figsize=(14, 8))
        
        for modelo in self.modelos:
            medias = [self.df[self.df['Paso'] == p][modelo].mean() for p in pasos]
            ax.plot(pasos, medias, marker='o', label=modelo, 
                   linewidth=2.5, markersize=8, color=COLORES_MODELOS[modelo])
        
        ax.set_xlabel('Horizonte de Predicción (Paso)', fontweight='bold', fontsize=12)
        ax.set_ylabel('Rendimiento Promedio', fontweight='bold', fontsize=12)
        ax.set_title('Evolución del Rendimiento por Horizonte', 
                     fontweight='bold', fontsize=14, pad=20)
        ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)
        ax.grid(True, alpha=0.3)
        ax.set_xticks(pasos)
        
        plt.tight_layout()
        plt.savefig(self.dir_salida / '6_1_horizonte_evolucion.png', 
                   dpi=300, bbox_inches='tight')
        plt.close()
        
        # FIGURA 6.2: Tasa de deterioro
        fig, ax = plt.subplots(figsize=(12, 8))
        
        tasas_deterioro = {}
        for modelo in self.modelos:
            medias = [self.df[self.df['Paso'] == p][modelo].mean() for p in pasos]
            if len(medias) > 1:
                pendiente = (medias[-1] - medias[0]) / (pasos[-1] - pasos[0])
                tasas_deterioro[modelo] = pendiente
        
        td_sorted = dict(sorted(tasas_deterioro.items(), key=lambda x: x[1]))
        
        colors_td = ['green' if v < np.median(list(td_sorted.values())) else 'red' 
                    for v in td_sorted.values()]
        bars = ax.barh(range(len(td_sorted)), list(td_sorted.values()), 
                       color=colors_td, alpha=0.7, edgecolor='black', linewidth=1.5)
        ax.set_yticks(range(len(td_sorted)))
        ax.set_yticklabels(list(td_sorted.keys()), fontsize=10)
        ax.set_xlabel('Tasa de Deterioro por Paso', fontweight='bold', fontsize=12)
        ax.set_title('Velocidad de Deterioro\n(Menor = Más Estable)', 
                     fontweight='bold', fontsize=14, pad=20)
        ax.axvline(0, color='black', linestyle='-', linewidth=2)
        ax.grid(True, alpha=0.3, axis='x')
        
        for i, (bar, val) in enumerate(zip(bars, td_sorted.values())):
            ax.text(val + (0.0001 if val > 0 else -0.0001), i, f'{val:.4f}', 
                   va='center', ha='left' if val > 0 else 'right',
                   fontweight='bold', fontsize=9)
        
        plt.tight_layout()
        plt.savefig(self.dir_salida / '6_2_horizonte_tasa_deterioro.png', 
                   dpi=300, bbox_inches='tight')
        plt.close()
        
        print("   ✓ 2 figuras generadas para horizonte\n")
    
    # ========================================================================
    # 7. ROBUSTEZ Y ESTABILIDAD
    # ========================================================================
    
    def _analisis_robustez(self):
        """Analiza la robustez y estabilidad de los modelos"""
        
        # Calcular métricas de robustez
        metricas_robustez = []
        
        for modelo in self.modelos:
            std_global = self.df[modelo].std()
            cv = (self.df[modelo].std() / self.df[modelo].mean()) * 100
            q75, q25 = self.df[modelo].quantile([0.75, 0.25])
            iqr = q75 - q25
            std_entre_escenarios = self.df.groupby('Escenario')[modelo].mean().std()
            std_entre_dist = self.df.groupby('Distribución')[modelo].mean().std()
            std_entre_var = self.df.groupby('Varianza error')[modelo].mean().std()
            
            metricas_robustez.append({
                'Modelo': modelo,
                'Std_Global': std_global,
                'CV': cv,
                'IQR': iqr,
                'Std_Escenarios': std_entre_escenarios,
                'Std_Distribuciones': std_entre_dist,
                'Std_Varianzas': std_entre_var
            })
        
        df_robustez = pd.DataFrame(metricas_robustez)
        
        # FIGURA 7.2: Coeficiente de variación
        fig, ax = plt.subplots(figsize=(12, 8))
        
        df_sorted = df_robustez.sort_values('CV')
        colors = plt.cm.RdYlGn(np.linspace(0.8, 0.2, len(df_sorted)))
        bars = ax.barh(df_sorted['Modelo'], df_sorted['CV'], 
                       color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)
        ax.set_xlabel('Coeficiente de Variación (%)', fontweight='bold', fontsize=12)
        ax.set_title('Variabilidad Relativa\n(Menor = Más Consistente)', 
                     fontweight='bold', fontsize=14, pad=20)
        ax.grid(True, alpha=0.3, axis='x')
        
        for i, (bar, val) in enumerate(zip(bars, df_sorted['CV'].values)):
            ax.text(val + 1, i, f'{val:.1f}%', va='center', fontweight='bold', fontsize=9)
        
        plt.tight_layout()
        plt.savefig(self.dir_salida / '7_2_robustez_coef_variacion.png', 
                   dpi=300, bbox_inches='tight')
        plt.close()
        
        # Guardar para usar después
        self.df_robustez = df_robustez
        
        print("   ✓ 1 figura generada para robustez\n")
    
    # ========================================================================
    # 8. DIFERENCIAS ESTADÍSTICAMENTE SIGNIFICATIVAS
    # ========================================================================
    
    def _analisis_significancia(self):
        """Análisis de diferencias estadísticamente significativas con Test DM"""
        print("\n" + "="*80)
        print("REALIZANDO TEST DE DIEBOLD-MARIANO")
        print("="*80 + "\n")
        
        # Realizar comparaciones múltiples
        df_comparaciones, alpha_bonf = comparaciones_multiples_dm(
            self.df, self.modelos, alpha=0.05
        )
        
        print(f"   Número de comparaciones: {len(df_comparaciones)}")
        print(f"   Alpha corregido (Bonferroni): {alpha_bonf:.6f}")
        print(f"   Comparaciones significativas: {df_comparaciones['Significativo'].sum()}")
        
        # Calcular ranking
        df_ranking, matriz_sup = calcular_ranking_dm(df_comparaciones, self.modelos)
        
        # FIGURA 8.2: Matriz de superioridad
        fig, ax = plt.subplots(figsize=(14, 12))
        
        sns.heatmap(matriz_sup, annot=True, fmt='.0f', cmap='RdYlGn', 
                   center=0, ax=ax, cbar_kws={'label': 'Superioridad'},
                   vmin=-1, vmax=1, linewidths=1, linecolor='gray',
                   annot_kws={'fontsize': 10, 'fontweight': 'bold'})
        ax.set_title('Matriz de Superioridad\n(1=Superior, -1=Inferior, 0=Sin diferencia)', 
                     fontweight='bold', fontsize=14, pad=20)
        ax.set_xlabel('Modelo Comparado', fontsize=12, fontweight='bold')
        ax.set_ylabel('Modelo', fontsize=12, fontweight='bold')
        ax.tick_params(labelsize=10)
        
        plt.tight_layout()
        plt.savefig(self.dir_salida / '8_2_significancia_matriz_superioridad.png', 
                   dpi=300, bbox_inches='tight')
        plt.close()
        
        # Guardar para usar después
        self.df_ranking = df_ranking
        self.df_comparaciones = df_comparaciones
        
        print(f"\n   ✓ Ranking guardado: Top 3")
        for i, row in df_ranking.head(3).iterrows():
            print(f"      {row['Rank']}. {row['Modelo']} - Score: {row['Score']} "
                  f"(V:{row['Victorias']}, D:{row['Derrotas']}, E:{row['Empates']})")
        
        print("\n   ✓ 1 figura generada para significancia\n")
    
    # ========================================================================
    # 9. RESUMEN EJECUTIVO
    # ========================================================================
    
    def _generar_resumen_ejecutivo(self):
        """Genera un resumen ejecutivo consolidado"""
        print("\n" + "="*80)
        print("GENERANDO RESUMEN EJECUTIVO")
        print("="*80 + "\n")
        
        pasos = sorted(self.df['Paso'].unique())
        distribuciones = self.df['Distribución'].unique()
        varianzas = sorted(self.df['Varianza error'].unique())
        
        # FIGURA 9.2: Comparación multidimensional - CORREGIDO
        fig, ax = plt.subplots(figsize=(14, 10))
        
        top5_modelos = self.df_ranking.head(5)['Modelo'].tolist()
        
        # Nuevas dimensiones
        caracteristicas = ['Ranking DM', 'Estabilidad por Paso', 
                          'Estabilidad por Distribución', 'Estabilidad por Varianza']
        matriz_resumen = []
        
        for modelo in top5_modelos:
            fila = []
            
            # 1. Ranking DM (normalizado)
            rank_pos = self.df_ranking[self.df_ranking['Modelo'] == modelo].index[0]
            rank_norm = 100 * (1 - rank_pos / (len(self.df_ranking) - 1))
            fila.append(rank_norm)
            
            # 2. Estabilidad por Paso (inversa de la std)
            stds_paso = [self.df[self.df['Paso'] == p][modelo].std() for p in pasos]
            est_paso = 100 * (1 - (np.mean(stds_paso) - min([np.mean([self.df[self.df['Paso'] == p][m].std() for p in pasos]) for m in self.modelos])) / 
                             (max([np.mean([self.df[self.df['Paso'] == p][m].std() for p in pasos]) for m in self.modelos]) - 
                              min([np.mean([self.df[self.df['Paso'] == p][m].std() for p in pasos]) for m in self.modelos])))
            fila.append(est_paso)
            
            # 3. Estabilidad por Distribución
            stds_dist = [self.df[self.df['Distribución'] == d][modelo].std() for d in distribuciones]
            est_dist = 100 * (1 - (np.mean(stds_dist) - min([np.mean([self.df[self.df['Distribución'] == d][m].std() for d in distribuciones]) for m in self.modelos])) / 
                             (max([np.mean([self.df[self.df['Distribución'] == d][m].std() for d in distribuciones]) for m in self.modelos]) - 
                              min([np.mean([self.df[self.df['Distribución'] == d][m].std() for d in distribuciones]) for m in self.modelos])))
            fila.append(est_dist)
            
            # 4. Estabilidad por Varianza
            stds_var = [self.df[self.df['Varianza error'] == v][modelo].std() for v in varianzas]
            est_var = 100 * (1 - (np.mean(stds_var) - min([np.mean([self.df[self.df['Varianza error'] == v][m].std() for v in varianzas]) for m in self.modelos])) / 
                            (max([np.mean([self.df[self.df['Varianza error'] == v][m].std() for v in varianzas]) for m in self.modelos]) - 
                             min([np.mean([self.df[self.df['Varianza error'] == v][m].std() for v in varianzas]) for m in self.modelos])))
            fila.append(est_var)
            
            matriz_resumen.append(fila)
        
        df_heatmap = pd.DataFrame(matriz_resumen, columns=caracteristicas, index=top5_modelos)
        
        sns.heatmap(df_heatmap, annot=True, fmt='.1f', cmap='RdYlGn', 
                   ax=ax, cbar_kws={'label': 'Score Normalizado (0-100)'},
                   linewidths=2, linecolor='white', vmin=0, vmax=100,
                   annot_kws={'fontsize': 11, 'fontweight': 'bold'})
        ax.set_title('Perfil Multidimensional - Top 5 Modelos', 
                     fontweight='bold', fontsize=14, pad=20)
        ax.set_xlabel('Dimensión de Evaluación', fontweight='bold', fontsize=12)
        ax.set_ylabel('Modelo', fontweight='bold', fontsize=12)
        ax.tick_params(labelsize=11)
        
        plt.tight_layout()
        plt.savefig(self.dir_salida / '9_2_resumen_perfil_multidimensional.png', 
                   dpi=300, bbox_inches='tight')
        plt.close()
        
        # FIGURA 9.3: Impacto de características - NORMALIZADO AL 100%
        fig, ax = plt.subplots(figsize=(12, 8))
        
        impactos = []
        
        # Estacionariedad
        est_data = []
        for modelo in self.modelos:
            est = self.df[self.df['Estacionario'] == 'Estacionario'][modelo].mean()
            no_est = self.df[self.df['Estacionario'] == 'No Estacionario'][modelo].mean()
            cambio = ((no_est - est) / est * 100) if est != 0 else 0
            est_data.append(cambio)
        impactos.append(('Estacionariedad', np.mean(np.abs(est_data))))
        
        # Linealidad
        lin_data = []
        for modelo in self.modelos:
            lin = self.df[self.df['Lineal'] == 'Lineal'][modelo].mean()
            no_lin = self.df[self.df['Lineal'] == 'No Lineal'][modelo].mean()
            cambio = ((no_lin - lin) / lin * 100) if lin != 0 else 0
            lin_data.append(cambio)
        impactos.append(('Linealidad', np.mean(np.abs(lin_data))))
        
        # Varianza
        var_data = []
        for modelo in self.modelos:
            corr = abs(self.df[['Varianza error', modelo]].corr().iloc[0, 1])
            var_data.append(corr * 100)
        impactos.append(('Varianza', np.mean(var_data)))
        
        # Horizonte
        hor_data = []
        pasos = sorted(self.df['Paso'].unique())
        for modelo in self.modelos:
            medias = [self.df[self.df['Paso'] == p][modelo].mean() for p in pasos]
            if len(medias) > 1 and medias[0] != 0:
                cambio = abs(((medias[-1] - medias[0]) / medias[0]) * 100)
                hor_data.append(cambio)
        impactos.append(('Horizonte', np.mean(hor_data)))
        
        # Distribución
        dist_data = []
        for modelo in self.modelos:
            medias_dist = self.df.groupby('Distribución')[modelo].mean()
            rango = medias_dist.max() - medias_dist.min()
            dist_data.append(rango / medias_dist.mean() * 100 if medias_dist.mean() != 0 else 0)
        impactos.append(('Distribución', np.mean(dist_data)))
        
        # Tipo de Modelo
        tipo_data = []
        for modelo in self.modelos:
            medias_tipo = self.df.groupby('Tipo de Modelo')[modelo].mean()
            rango = medias_tipo.max() - medias_tipo.min()
            tipo_data.append(rango / medias_tipo.mean() * 100 if medias_tipo.mean() != 0 else 0)
        impactos.append(('Tipo Generador', np.mean(tipo_data)))
        
        # NORMALIZAR AL 100%
        total_impacto = sum([x[1] for x in impactos])
        impactos_norm = [(nombre, (valor / total_impacto) * 100) for nombre, valor in impactos]
        
        # Ordenar por impacto
        impactos_sorted = sorted(impactos_norm, key=lambda x: x[1], reverse=True)
        nombres_imp = [x[0] for x in impactos_sorted]
        valores_imp = [x[1] for x in impactos_sorted]
        
        colors_imp = plt.cm.Reds(np.linspace(0.3, 0.9, len(impactos_sorted)))
        bars = ax.barh(nombres_imp, valores_imp, color=colors_imp, alpha=0.8, 
                       edgecolor='black', linewidth=1.5)
        ax.set_xlabel('Impacto Normalizado (%)', fontweight='bold', fontsize=12)
        ax.set_title('Impacto de Características en el Rendimiento\n(Total = 100%)', 
                     fontweight='bold', fontsize=14, pad=20)
        ax.grid(True, alpha=0.3, axis='x')
        ax.set_xlim(0, max(valores_imp) * 1.15)
        
        for i, (bar, val) in enumerate(zip(bars, valores_imp)):
            ax.text(val + 1, i, f' {val:.1f}%', va='center', fontweight='bold', fontsize=11)
        
        # Agregar suma total
        ax.text(0.98, 0.02, f'Suma Total: {sum(valores_imp):.1f}%', 
                transform=ax.transAxes, fontsize=11, fontweight='bold',
                ha='right', va='bottom',
                bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.5))
        
        plt.tight_layout()
        plt.savefig(self.dir_salida / '9_3_resumen_impacto_caracteristicas.png', 
                   dpi=300, bbox_inches='tight')
        plt.close()
        
        print("   ✓ 2 figuras generadas para resumen ejecutivo")
        print()


# ============================================================================
# FUNCIÓN PRINCIPAL
# ============================================================================

def main():
    """Función principal de ejecución"""
    print("\n" + "█"*80)
    print("█" + " "*78 + "█")
    print("█" + " "*15 + "ANÁLISIS COMPLETO DE BASE DE DATOS - VERSIÓN CORREGIDA" + " "*8 + "█")
    print("█" + " "*78 + "█")
    print("█"*80 + "\n")
    
    try:
        # Crear instancia del analizador
        analizador = AnalizadorBaseCompleta(RUTA_DATOS)
        
        # Ejecutar análisis completo
        analizador.ejecutar_analisis_completo()
        
        print("\n" + "█"*80)
        print("█" + " "*78 + "█")
        print("█" + " "*20 + "✅ ANÁLISIS COMPLETADO EXITOSAMENTE" + " "*23 + "█")
        print("█" + " "*78 + "█")
        print("█"*80 + "\n")
        
        print("📊 TOTAL DE FIGURAS GENERADAS: 15 imágenes PNG")
        print("\n📁 ESTRUCTURA DE RESULTADOS:")
        print(f"   {DIR_SALIDA}/")
        print("   ├── 1.1: Estacionariedad - Comparativo")
        print("   ├── 1.2: Estacionariedad - Cambio Relativo")
        print("   ├── 2.1: Linealidad - Comparativo")
        print("   ├── 2.2: Linealidad - Cambio Relativo")
        print("   ├── 3.2: Modelo Generador - Z-Score")
        print("   ├── 3.3: Modelo Generador - Variabilidad")
        print("   ├── 4.1: Distribución - Heatmap Rendimiento")
        print("   ├── 4.2: Distribución - Heatmap Variabilidad")
        print("   ├── 5.1: Varianza - Tendencias")
        print("   ├── 5.2: Varianza - Tasa de Crecimiento")
        print("   ├── 6.1: Horizonte - Evolución")
        print("   ├── 6.2: Horizonte - Tasa de Deterioro")
        print("   ├── 7.2: Robustez - Coeficiente de Variación")
        print("   ├── 8.2: Significancia - Matriz de Superioridad")
        print("   ├── 9.2: Resumen - Perfil Multidimensional")
        print("   └── 9.3: Resumen - Impacto de Características (normalizado)")
        print("\n" + "="*80 + "\n")
        
    except FileNotFoundError:
        print(f"\n❌ ERROR: No se encontró el archivo {RUTA_DATOS}")
        print("   Por favor, verifica que el archivo existe y la ruta es correcta.\n")
    except Exception as e:
        print(f"\n❌ ERROR INESPERADO: {str(e)}")
        import traceback
        traceback.print_exc()


if __name__ == "__main__":
    main()


████████████████████████████████████████████████████████████████████████████████
█                                                                              █
█               ANÁLISIS COMPLETO DE BASE DE DATOS - VERSIÓN CORREGIDA        █
█                                                                              █
████████████████████████████████████████████████████████████████████████████████


INICIANDO ANÁLISIS COMPLETO DE BASE DE DATOS

✓ Características extraídas:
  - Estacionariedad: ['Estacionario' 'No Estacionario']
  - Linealidad: ['Lineal' 'No Lineal']
  - Tipos de Modelo: ['AR(1)' 'AR(2)' 'MA(1)' 'MA(2)' 'ARMA(1,1)' 'ARMA(2,2)' 'ARIMA(0,1,0)'
 'ARIMA(1,1,0)' 'ARIMA(2,1,0)' 'ARIMA(0,1,1)' 'ARIMA(0,1,2)'
 'ARIMA(1,1,1)' 'ARIMA(2,1,2)' 'SETAR(2,1)' 'TAR(2,1)' 'EXPAR(2,1)'
 'BILINEAR(1)' 'SETAR(2,2)' 'TAR(2,2)' 'SETAR(2,3)']
  - Distribuciones: ['normal' 'uniform' 'exponential' 't-student' 'mixture']
  - Varianzas: [np.float64(0.2), np.float64(0.5), np.float64(1.0), np.f